# Predicting The Results of NFL Games

## 1. Introduction to sportsipy

#### Installing open source modules (please activate any modules that are needed to be installed)

In [ ]:
!pip install sportsipy
# !pip install matplotlib
# !pip install pandas
# !pip install numpy
# !pip install datetime
# !pip install seaborn
# !pip install geojsonio --upgrade
# !pip install folium --upgrade
# !pip install ipython
# !pip install branca
# !pip install scipy
# !pip install --user decorator==4.3.0
# !pip install networkx
# !pip install scikit-learn
# !pip install xgboost
# !pip install tensorflow
# !pip install yellowbrick

# !pip install nltk
# import nltk
# nltk.download('all-corpora')

# !pip install requests
# !pip install tweepy
# !pip install gensim==3.8.3

#### Importing some of the necessary packages

In [1]:
# sportsipy is an open source library to extract the game statistics
from sportsipy.nfl.boxscore import Boxscores as game_info
from sportsipy.nfl.boxscore import Boxscore as game_stats_info

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime           
%matplotlib inline

In [3]:
# To hide all the warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
# Increase resolutions of all the graphs in the notebook
# plt.rcParams['figure.dpi'] = 80 

#### Game information and stats methods from sportsipy
1. game_info(week_number, year).games['string'] takes argument as string as 'week_number-year' format and returns a json object (having all games in that week of that year)
2. We then pull out game_id from the json object
3. game_stats_info() takes game_id (here game_str) as an argument and returns the game information

In [51]:
game_str = game_info(4,2022).games['4-2022'][13]['boxscore']
one_game_stats = game_stats_info(game_str)
one_game_stats.dataframe

,attendance,away_first_downs,away_fourth_down_attempts,away_fourth_down_conversions,away_fumbles,away_fumbles_lost,away_interceptions,away_net_pass_yards,away_pass_attempts,away_pass_completions,away_pass_touchdowns,away_pass_yards,away_penalties,away_points,away_rush_attempts,away_rush_touchdowns,away_rush_yards,away_third_down_attempts,away_third_down_conversions,away_time_of_possession,away_times_sacked,away_total_yards,away_turnovers,away_yards_from_penalties,away_yards_lost_from_sacks,date,datetime,duration,home_first_downs,home_fourth_down_attempts,home_fourth_down_conversions,home_fumbles,home_fumbles_lost,home_interceptions,home_net_pass_yards,home_pass_attempts,home_pass_completions,home_pass_touchdowns,home_pass_yards,home_penalties,home_points,home_rush_attempts,home_rush_touchdowns,home_rush_yards,home_third_down_attempts,home_third_down_conversions,home_time_of_possession,home_times_sacked,home_total_yards,home_turnovers,home_yards_from_penalties,home_yards_lost_from_sacks,losing_abbr,losing_name,over_under,roof,stadium,surface,time,vegas_line,weather,winner,winning_abbr,winning_name,won_toss
202210020rai,62332,12,2,1,1,1,0,214,25,17,2,237,7,23,20,1,85,11,3,25:08,3,299,1,50,23,"Sunday Oct 2, 2022",2022-10-02 16:25:00,3:09,25,0,0,0,0,0,173,34,21,0,188,4,32,38,2,212,14,7,34:52,2,385,0,25,15,DEN,Denver Broncos,45.0 (over),Dome,Allegiant Stadium,Grass,4:25pm,Las Vegas Raiders -3.0,None,Home,RAI,Las Vegas Raiders,Raiders


In [6]:
week_scores = game_info(2,2021)
game_str = week_scores.games['2-2021'][0]['boxscore']
one_game_stats = game_stats_info(game_str)
g_df = pd.DataFrame(week_scores.games['2-2021'][0], index = [0])
#g_data(g_df,one_game_stats)[0]
#week_scores.games['2-2021'][0], index = [0]

In [ ]:
g_df

In [120]:
df = pd.read_csv('https://projects.fivethirtyeight.com/nfl-api/nfl_elo_latest.csv')

In [ ]:
df

In [ ]:
pd.DataFrame(week_scores.games['2-2021'][0], index = [0])

## 2. Data Gathering and Exploration

### 2.1. Extracting the schedule of past 11 years (2010 - 2021)

Building a function using the open source sportsipy package that loops through each week and each game inside each week

In [5]:
def schedule(year):
    weeks_list = list(range(1,19))
    schedule_df = pd.DataFrame()
    for w in range(len(weeks_list)):
        date = '{}-{}'.format(weeks_list[w], year)
        w_scores = game_info(weeks_list[w],year)
        w_games_df = pd.DataFrame()
        for g in range(len(w_scores.games[date])):
            game_df = pd.DataFrame(w_scores.games[date][g], index = [0])[['away_name',
                                                                          'away_abbr',
                                                                          'home_name',
                                                                          'home_abbr',
                                                                          'winning_name',
                                                                          'winning_abbr']]
            game_df['week'] = weeks_list[w]
            w_games_df = pd.concat([w_games_df,game_df])
        schedule_df = pd.concat([schedule_df, w_games_df]).reset_index().drop(columns = 'index') 
    return schedule_df

In [6]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
schedule(2022)

1. Importing the schedule from the year 2010 to 2020 and exporting it to a csv file (so that we don't need to extract the data from the open source, hence increasing the speed)
2. Changing names of the teams that have changed their name in the last 10 years to maintain homogeneity 
3. Concating the schedule of current year (2021) to the schedule from 2010 to 2020

In [9]:
# Setting up the current year variable and passing the number of weeks to a list
current_y = 2022
weeks_list = list(range(1,19))

In [ ]:
full_schedule = pd.DataFrame()
for n in range(2010, 2022):
    schedule_ = schedule(n)
    schedule_['year'] = n
    full_schedule = pd.concat([full_schedule, schedule_])
full_schedule.reset_index(drop = True, inplace = True)

In [11]:
full_schedule = full_schedule.replace('Oakland Raiders', 'Las Vegas Raiders')
full_schedule = full_schedule.replace('San Diego Chargers', 'Los Angeles Chargers')
full_schedule = full_schedule.replace('St. Louis Rams', 'Los Angeles Rams')
full_schedule = full_schedule.replace('Washington Redskins', 'Washington Commanders')

In [12]:
full_schedule = full_schedule.replace('Washington Football Team', 'Washington Commanders')

In [ ]:
full_schedule.to_csv('full_schedule5.csv')

In [6]:
full_schedule = pd.read_csv('full_schedule3.csv')
full_schedule = full_schedule.loc[:, ~full_schedule.columns.str.contains('^Unnamed')]

In [ ]:
full_schedule.tail(500)

In [7]:
full_schedule = full_schedule.loc[full_schedule['year']!=2022]

In [7]:
full_schedule = full_schedule.drop_duplicates()

In [10]:
full_schedule_1 = schedule(current_y)
full_schedule_1['year'] = current_y
full_schedule = pd.concat([full_schedule, full_schedule_1])
full_schedule.reset_index(drop = True, inplace = True)

In [9]:
completed_games_df = full_schedule[(full_schedule['winning_name'].notna()) &
                                       (full_schedule['year'] == current_y)]
    

NameError: name 'current_y' is not defined

In [ ]:
current_y


In [ ]:
completed_games_df

### CHANGING CODE TO ADJUST FOR 2022

In [16]:
if completed_games_df.empty:
    latest_completed_w = 1
else:
    latest_completed_w = completed_games_df.values.max()

scheduled_games_df = full_schedule[(full_schedule['winning_name'].isnull()) &
                                           (full_schedule['year'] == current_y) &
                                           (full_schedule['week'] >= latest_completed_w)]

In [ ]:
latest_completed_w

In [ ]:
scheduled_games_df

### 2.2. Setting up the current week
1. Current week (current_w) is the most important argument (parameter) in the functions
2. We should make sure that the current_w really matches the real current week of the NFL season while also considering whether or not the library has updated its database
3. We have run our code and built our model in the week 14 of 2021 NFL season (Dec 12, 09.00 AM)
4. In other words, all the visualizations, analysis, model accuracy, model selection, presentation, and report all have been produced from running the code on that time 
5. Our model is an adapting model, meaning that it will adapt and take into accounts the new available data (the new games that just been played during the current week)
6. Please note that the results may be changed and different from our submission (presentation and report) if the code is being run again after our submission time since the current week and data may be updated by the library developer
7. The results may also be changed even when the current week is still the same if there are new game data that are being updated in the database

In [17]:
# Using full_schedule method to determine the correct current week as an input
def determine_current_week(full_schedule, current_y):
    current_w = int()
    completed_games_df = full_schedule[(full_schedule['winning_name'].notna()) &
                                       (full_schedule['year'] == current_y)]
    
    if len(completed_games_df) == 0:
        current_w = 1
    else:
        latest_completed_w = completed_games_df['week'].values.max()
        scheduled_games_df = full_schedule[(full_schedule['winning_name'].isnull()) &
                                           (full_schedule['year'] == current_y) &
                                           (full_schedule['week'] >= latest_completed_w)]
#         if scheduled_games_df.empty:
#             earliest_scheduled_w = 1
#         else:
#             latest_completed_w = completed_games_df.values.max()

        earliest_scheduled_w = scheduled_games_df['week'].values.min()
        #print(completed_games_df)
        current_w = earliest_scheduled_w
    
    print('The current week is: ', current_w)
    
    return current_w

In [16]:
# Setting up the current week
current_w = determine_current_week(full_schedule, current_y)
print('The current week value being assigned: ', current_w)

The current week is:  1
The current week value being assigned:  1


### 2.3. Full schedule dataframe exploration

In [ ]:
full_schedule.head()

In [ ]:
full_schedule.tail()

In [ ]:
full_schedule.info()

In [44]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
full_schedule_null = full_schedule[full_schedule['winning_name'].isnull()]
len(full_schedule_null)

In [ ]:
full_schedule_null

In [17]:
full_schedule = full_schedule.drop_duplicates()

In [ ]:
full_schedule

1. We can see that there are two columns that contain null values: winning_name and winning_abbr
2. There are 87 null values
3. The null values consists of NaN and None
4. NaN = The result of the game is a draw -> to be handled (excluded) later
5. None = The match (or game) has not completed yet, only scheduled

In [ ]:
# We can find which team recorded the most wins from 2010 up until now
def plot_team_rank_based_on_winning_count(full_schedule):
    winner_group = full_schedule.groupby('winning_name')
    from matplotlib import cm
    plt.rcParams["figure.dpi"] = 80
    colors = cm.inferno_r(np.linspace(.2, .8, 32))
    winner_group.size().sort_values().plot(kind='barh', figsize=(6,8), xlabel='Team Name',
        title='Team Ranking Based on Total Number of Games Won 2010-2021', color=colors)

plot_team_rank_based_on_winning_count(full_schedule)

1. Team with the most wins is New England Patriots, followed by Green Bay Packers and Pittsburgh Steelers
2. On the other hand, team with the least wins is Jacksonville Jaguars, followed by Cleveland Browns and New York Jets
3. We can create the same chart but only for a specific year (cross-section analysis)
4. For example, in 2020, the team with the most wins is Kansas City Chiefs, followed by Buffalo Bills and Green Bay Packers

In [ ]:
# Find the team with the most wins in a given year
def plot_team_rank_for_specific_year(full_schedule, year):
    full_schedule_filtered = full_schedule[full_schedule['year'] == year]
    winner_group = full_schedule_filtered.groupby('winning_name')
    from matplotlib import cm
    plt.rcParams["figure.dpi"] = 80
    colors = cm.inferno_r(np.linspace(.2, .8, 32))
    winner_group.size().sort_values().plot(kind='barh', figsize=(6,8), x='Games Won', xlabel='Team Name',
        title=f'Team Ranking Based on Total Number of Games Won in {year}', color=colors)

plot_team_rank_for_specific_year(full_schedule, 2021)

In [ ]:
# We can also find out how each team performs each year (time-series)
def plot_team_performance(full_schedule, team_names):
    team_performance_df = full_schedule.groupby(['winning_name','year']).size().unstack().T
    team_performance_df = team_performance_df.fillna(0)
    team_performance_df.columns.name = 'Team'
    plt.rcParams["figure.dpi"] = 80
    team_performance_df.loc[:,team_names].plot(figsize=(10,5), grid=False, xlabel='Year',
        ylabel='Games Won', xticks = team_performance_df.index, colormap='coolwarm',
        lw=4, title='Number of Games Won by Each Team Annually')
    return team_performance_df

team_names = ['New England Patriots']
plot_team_performance(full_schedule, team_names)

1. As seen above, even though New England Patriots recorded the most wins in the last 10 years, its performance was not that good last year
2. Using the same function, we can compare the trend for several teams in one graph like the one below

In [ ]:
team_names = ['Arizona Cardinals','Detroit Lions','Miami Dolphins','New York Jets','Atlanta Falcons']
plot_team_performance(full_schedule, team_names)

1. Another time-series analysis we can do with the full schedule dataframe is comparing the number of wins between away and home team
2. From the graph below, we can see that home team scored more wins from 2010 to 2019
3. In 2020, however, away team managed to record more wins
4. The season is still on-going for the 2021 NFL but away team is currently leading the metric

In [ ]:
# We can check how many times a home team wins vs an away team wins each year
def plot_home_vs_away(full_schedule):
    home_team_wins_df = full_schedule[full_schedule['winning_name']==full_schedule['home_name']]
    home_team_wins_df = pd.DataFrame(home_team_wins_df.groupby('year').size())
    away_team_wins_df = full_schedule[full_schedule['winning_name']==full_schedule['away_name']]
    away_team_wins_df = pd.DataFrame(away_team_wins_df.groupby('year').size())
    combined_df = pd.merge(home_team_wins_df, away_team_wins_df, left_index=True, right_index=True)
    combined_df = combined_df.rename(columns = {'0_x':'Home','0_y':'Away'})
    plt.rcParams["figure.dpi"] = 80
    combined_df.plot(kind ='bar',figsize=(8,4), grid=False, xlabel='Year', ylabel='Games Won',
                    title='Total Number of Games Won by Home Team vs. Away Team',
                    colormap='coolwarm')

plot_home_vs_away(full_schedule)

### 2.4. Extracting the game data (stats) of past 11 years (2010 - 2021)

#### Column manipulation functions

Creating the supporting functions that will help the main functions for ease of processing

In [12]:
# 'column_name_manipulation' function changes the feature names (it removes home/away from the start of the feature name)
def column_name_manipulation(obj):
    if type(obj) is not list:
        columns =  list(obj.columns)
    else:
        columns =  obj
    new_columns_list = []

    for column_name in columns:
        if 'away' in column_name:
            column_name = column_name.split('_')
            column_name.remove('away')
            column_name = column_name = '_'.join(column_name)
            new_columns_list.append(column_name)
            
        elif 'home' in column_name:
            column_name = column_name.split('_')
            column_name.remove('home')
            column_name = column_name = '_'.join(column_name)
            new_columns_list.append(column_name)
            
        else:
            new_columns_list.append(column_name)
    if type(obj) is not list:
        obj.columns = new_columns_list
    else:       
        obj = new_columns_list
        
    return obj

In [13]:
# 'column_name_manipulation' function changes the feature names (it adds home/away from the start of the feature name)
def column_name_manipulation_reverse(obj, type_ = 'home'):
    if type(obj) is not list:
        columns =  list(obj.columns)
    else:
        columns =  obj
        
    new_columns_list = []

    for column_name in columns:
        column_name = type_ + '_' + column_name 
        new_columns_list.append(column_name)
            
    if type(obj) is not list:
        obj.columns = new_columns_list
    else:       
        obj = new_columns_list
        
    return obj

1. Dropping the redundant columns
2. Listing the required columns

In [14]:
columns_required = ['first_downs', 'fourth_down_attempts', 'fourth_down_conversions', 
                    'fumbles', 'fumbles_lost', 'interceptions','net_pass_yards', 
                    'pass_attempts','pass_completions','pass_touchdowns','pass_yards',
                    'penalties','points','rush_attempts','rush_touchdowns','rush_yards',
                    'third_down_attempts','third_down_conversions','time_of_possession',
                    'times_sacked','total_yards','turnovers','yards_from_penalties','yards_lost_from_sacks']

In [ ]:
columns_required_home = column_name_manipulation_reverse(columns_required, 'home')

In [ ]:
columns_required2 = ['vegas_odds','over_under']

In [306]:
columns_required2 = ['vegas_line', 'over_under', 'weather']

In [ ]:
#columns_required2_home = column_name_manipulation_reverse(columns_required2, 'home')
columns_required_away = column_name_manipulation_reverse(columns_required, 'away')

In [ ]:
columns_required2_away

In [ ]:
a_stats_df_vegas_lines = one_game_stats.dataframe[columns_required2]

In [ ]:
a_stats_df_vegas_lines

In [ ]:
column_name_manipulation(g_df[['away_name', 'away_abbr', 'away_score']])

In [ ]:
g_df[['away_name', 'away_abbr', 'away_score']]

In [ ]:
columns_required_home = column_name_manipulation_reverse(columns_required, 'home')
columns_required_away = column_name_manipulation_reverse(columns_required, 'away')

In [ ]:
columns_required_away

In [ ]:
y = one_game_stats.dataframe[columns_required_home]
x = one_game_stats.dataframe[columns_required2]
pd.concat([x,y], axis=1)


In [ ]:
week_scores = game_info(2,2021)
game_str = week_scores.games['2-2021'][0]['boxscore']
one_game_stats = game_stats_info(game_str)
one_game_stats
one_game_stats.dataframe[columns_required_home].reset_index().drop(columns = 'index')

In [ ]:
column_name_manipulation_reverse(columns_required, 'home')

In [ ]:
x = one_game_stats.dataframe[columns_required2]
x

#### Function to extract each game info

1. 'g_data' function takes 2 arguments as inputs, schedule and game info
2. It returns 2 dataframes, home_df and away_df
3. home_df and away_df contain statistics of home team and away team for that particular game
4. It also adds 2 more columns to the dataframes, game_won and game_lost, which are binary. If score of away team is greater than home team, that means away team won and hence away_df would have game_won flagged as 1 and game_lost flagged as 0. Same for home team.
5. The prefixes home/away are removed from each of the dataframe features
6. We do (5) to build past records of each team for each game and use it for modelling and analyses
7. The function also transforms the value of time_of_possession from a time duration format (%M:%S) to integer duration (expressed in seconds)

In [15]:
def g_data(g_df,one_game_stats):

    columns_required_home = column_name_manipulation_reverse(columns_required, 'home')
    columns_required_away = column_name_manipulation_reverse(columns_required, 'away')
    
    #columns_required2_home = column_name_manipulation_reverse(columns_required2, 'home')
    #columns_required2_away = column_name_manipulation_reverse(columns_required2, 'away')
    
    #vegas_odds_df= one_game_stats.dataframe[columns_required2]
    columns_required2 = ['vegas_line','over_under','weather']
    x = one_game_stats.dataframe[columns_required2].reset_index().drop(columns = 'index')
    
    
    try:
        a_team_df = column_name_manipulation(g_df[['away_name', 'away_abbr', 'away_score']]).rename(columns = {
            'name' : 'team_name', 'abbr': 'team_abbr'})
        h_team_df = column_name_manipulation(g_df[['home_name','home_abbr', 'home_score']]).rename(columns = {
            'name' : 'team_name', 'abbr': 'team_abbr'})
       # columns_required2 = ['vegas_line', 'over_under', 'team_name']
       # vegas_odds_df= one_game_stats.dataframe[columns_required2]

        try:
            if g_df.loc[0,'away_score'] != g_df.loc[0,'home_score']:
                a_team_df['game_won'] = int(g_df.loc[0,'away_score'] > g_df.loc[0,'home_score'])
                a_team_df['game_lost'] = 1- a_team_df['game_won']
                h_team_df['game_won'] = int(g_df.loc[0,'away_score'] < g_df.loc[0,'home_score'])
                h_team_df['game_lost'] = 1- h_team_df['game_won']

            else:
                a_team_df['game_won'] = a_team_df['game_lost'] = h_team_df['game_won'] = h_team_df['game_lost'] = 0

        except TypeError:
                a_team_df['game_won'] = a_team_df['game_lost'] = h_team_df['game_won'] = h_team_df['game_lost'] = np.nan
                
        a_stats_df = one_game_stats.dataframe[columns_required_away].reset_index().drop(columns ='index')
        
        a_stats_df = column_name_manipulation(a_stats_df)
       # a_stats_df = pd.concat([x,a_stats_df], axis=1)
        
        
        #a_stats_df_vegas_lines = one_game_stats.dataframe[columns_required2].reset_index().drop(columns ='index')
        #a_stats_df_vegas_lines= column_name_manipulation(vegas_odds_df)
        #a_stats_df = pd.merge(a_stats_df, a_stats_df_vegas_lines, left_index = True, right_index = True)
        
        h_stats_df = one_game_stats.dataframe[columns_required_home].reset_index().drop(columns = 'index')
        #x = one_game_stats.dataframe[columns_required2]
        h_stats_df = column_name_manipulation(h_stats_df)
       # h_stats_df = pd.concat([x,h_stats_df], axis=1)
        #h_stats_df_vegas_lines = one_game_stats.dataframe[columns_required2].reset_index().drop(columns ='index')
        #h_stats_df_vegas_lines= column_name_manipulation(vegas_odds_df)
        #h_stats_df = pd.merge(h_stats_df, h_stats_df_vegas_lines, left_index = True, right_index = True)

        a_team_df = pd.merge(a_team_df, a_stats_df, left_index = True, right_index = True)
        
        #a_team_df =  pd.merge(a_team_df, x, left_index = True, right_index = True)
        h_team_df = pd.merge(h_team_df, h_stats_df, left_index = True, right_index = True)
        
        #a_team_df = a_team_df.merge(vegas_odds_df, on='team_name')
       # h_team_df = pd.merge(h_team_df, vegas_odds_df, left_index = True, right_index = True)
        
        try:
            time_a_team = datetime.datetime.strptime(a_team_df['time_of_possession'][0],'%M:%S')
            time_h_team = datetime.datetime.strptime(h_team_df['time_of_possession'][0],'%M:%S')
            a_team_df['time_of_possession'] = int(time_a_team.minute* 60) + int(time_a_team.second)
            h_team_df['time_of_possession'] = int(time_h_team.minute* 60) + int(time_h_team.second)
        except TypeError:
            a_team_df['time_of_possession'] = np.nan
            h_team_df['time_of_possession'] = np.nan
            
    except TypeError:
        a_team_df = pd.DataFrame()
        h_team_df = pd.DataFrame()
    h_team_df.insert(1,'vegas_odds', x['vegas_line'])
    a_team_df.insert(1,'vegas_odds', x['vegas_line'])
    h_team_df.insert(1,'over_under', x['over_under'])
    a_team_df.insert(1,'over_under', x['over_under'])
    h_team_df.insert(1,'weather', x['weather'])
    a_team_df.insert(1,'weather', x['weather'])
    #new_df = pd.concat([new_df, over_df['Actual Total']], axis=1)
    return a_team_df, h_team_df

In [17]:
g_df = pd.DataFrame(week_scores.games['2-2021'][0], index = [0])
g_df.set_index('boxscore')
#g_df2.rename(index={1: 'boxscore'})
#g_df2.index.names = ['boxscore']
g_df2 = one_game_stats.dataframe
g_df.set_index('boxscore')

,away_name,away_abbr,away_score,home_name,home_abbr,home_score,winning_name,winning_abbr,losing_name,losing_abbr
boxscore,,,,,,,,,,
202109160was,New York Giants,nyg,29,Washington Football Team,was,30,Washington Football Team,was,New York Giants,nyg


In [66]:
week_scores = game_info(2,2021)
game_str = week_scores.games['2-2021'][0]['boxscore']
one_game_stats = game_stats_info(game_str)
g_df = pd.DataFrame(week_scores.games['2-2021'][0], index = [0])

g_data(g_df,one_game_stats)[0]


,team_name,weather,over_under,vegas_odds,team_abbr,score,game_won,game_lost,first_downs,fourth_down_attempts,fourth_down_conversions,fumbles,fumbles_lost,interceptions,net_pass_yards,pass_attempts,pass_completions,pass_touchdowns,pass_yards,penalties,points,rush_attempts,rush_touchdowns,rush_yards,third_down_attempts,third_down_conversions,time_of_possession,times_sacked,total_yards,turnovers,yards_from_penalties,yards_lost_from_sacks
0,New York Giants,"71 degrees, relative humidity 94%, no wind",40.5 (over),Washington Football Team -3.5,nyg,29,0,1,21,0,0,0,0,0,228,32,22,1,249,11,29,28,1,163,12,4,1904,4,391,0,81,21


In [24]:
columns_required_home = column_name_manipulation_reverse(columns_required, 'home')
one_game_stats.dataframe[columns_required2]

,vegas_line,over_under
202109160was,Washington Football Team -3.5,40.5 (over)


In [1]:
year

NameError: name 'year' is not defined

In [ ]:
(g_df[['home_name','home_abbr', 'home_score']]).rename(columns = {
            'name' : 'team_name', 'abbr': 'team_abbr'})

In [ ]:
g_df[['home_name','home_abbr', 'home_score']]

In [18]:
def g_data_till_week(weeks_list, year):
    weeksgames_df = pd.DataFrame()
    for w in range(len(weeks_list)):
        date = '{}-{}'.format(weeks_list[w], year)
        w_scores = game_info(weeks_list[w],year)
        w_games_df = pd.DataFrame()
        for g in range(len(w_scores.games[date])):
            game_string = w_scores.games[date][g]['boxscore']
            try:
                one_game_stats = game_stats_info(game_string)
                g_df = pd.DataFrame(w_scores.games[date][g], index = [0])
                a_team_df, h_team_df = g_data(g_df,one_game_stats)
                a_team_df['week'] = h_team_df['week'] = weeks_list[w]
                w_games_df = pd.concat([w_games_df,a_team_df])
                w_games_df = pd.concat([w_games_df,h_team_df])
            except:
                one_game_stats = None
        weeksgames_df = pd.concat([weeksgames_df,w_games_df])
       
    return weeksgames_df


In [19]:
weeks_games_df=g_data_till_week(weeks_list, 2022)

In [53]:
weeks_games_df=weeks_games_df.reset_index(drop=True)

In [ ]:
list1=[]
list2=[]
for i in range(0,len(weeks_games_df),2):
    print(weeks_games_df['score'][i+1])
#     list1.append((weeks_games_df['score'][i+1])+(weeks_games_df['score'][i]))
#     list2.append((weeks_games_df['score'][i+1])-(weeks_games_df['score'][i]))
#     #list7.append((df_new['1st'][i+1])+(df_new['1st'][i])+(df_new['2nd'][i+1])+(df_new['2nd'][i]))
#     #list8.append((df_new['3rd'][i+1])+(df_new['3rd'][i])+(df_new['4th'][i+1])+(df_new['4th'][i]))
#    # list6.append(df_new['Team'][i+1])
# list1 = list(np.repeat(list1, 2))
# list2 = list(np.repeat(list2, 2))
# #print(list2)
# #for i in range(0,len(df_new),2):

# weeks_games_df['Actual Total']=list1
# weeks_games_df['Actual Score Differential']=list2
# #weeksgames_df['Overs']= np.where(((weeksgames_df['Actual Total'])>= ((weeksgames_df['total']))),1, 0)
# weeks_games_df['Actual Score Differential'] = weeks_games_df['Actual Score Differential'].apply(float)
# # weeksgames_df['game_won']=full_game_data['game_won'].apply(int)

#### Apply g_data function in for loop to gather all the game statistics given number of weeks and year number

1. g_data_till_week function takes 2 arguments : weeks_list and year (weeks_list is the list of all the weeks for which we want data) 
2. It returns team statistics for each game in that particular week for that particular year
3. In our case, we take data for all the weeks in a season 

In [20]:
def g_data_till_week(weeks_list, year):
    weeksgames_df = pd.DataFrame()
    for w in range(len(weeks_list)):
        date = '{}-{}'.format(weeks_list[w], year)
        w_scores = game_info(weeks_list[w],year)
        #print(w_scores)
        w_games_df = pd.DataFrame()
        for g in range(len(w_scores.games[date])):
            game_string = w_scores.games[date][g]['boxscore']
            try:
                one_game_stats = game_stats_info(game_string)
                g_df = pd.DataFrame(w_scores.games[date][g], index = [0])
                a_team_df, h_team_df = g_data(g_df,one_game_stats)
                a_team_df['week'] = h_team_df['week'] = weeks_list[w]
                w_games_df = pd.concat([w_games_df,a_team_df])
                w_games_df = pd.concat([w_games_df,h_team_df])
            except:
                one_game_stats = None
        weeksgames_df = pd.concat([weeksgames_df,w_games_df]).reset_index(drop=True)
#     weeksgames_df['spread']= weeksgames_df['vegas_odds'].str[-4:]
#     weeksgames_df['wind']= weeksgames_df['weather'].str[-8:]
#     weeksgames_df['total']= weeksgames_df['over_under'].str[:4]
#     weeksgames_df['over or under']= weeksgames_df['over_under'].str[4:]
#     weeksgames_df['spread']=np.where(weeksgames_df['spread'] =='Pick',0, weeksgames_df['spread'])
#    # weeksgames_df['spread']=np.where(weeksgames_df['spread'] ==None,0, weeksgames_df['spread'])
#     weeksgames_df['spread'] = weeksgames_df['spread'].apply(float)
#     weeksgames_df['total'] = weeksgames_df['total'].apply(float)
#     lst = []
#     weeksgames_df=weeksgames_df.reset_index()
#     for i in range(len(weeksgames_df)):
#         #print(weeksgames_df['vegas_odds'][i][:-5])
#         if weeksgames_df['team_name'][i].replace(" ","") == weeksgames_df['vegas_odds'][i][:-5].replace(" ",""):
#             lst.append((weeksgames_df['spread'][i])* 1)
#             #weeksgames_df['new spread']=((weeksgames_df['spread'][i])* 1)
#         else:
#             lst.append((weeksgames_df['spread'][i])* -1)
#     weeksgames_df['new spread']=lst
    list1=[] 
    list2=[]
    list6=[]
    list7=[]
    list8=[]
    for i in range(0,len(weeksgames_df),2):
        list1.append((weeksgames_df['score'][i+1])+(weeksgames_df['score'][i]))
        list2.append((weeksgames_df['score'][i+1])-(weeksgames_df['score'][i]))
        #list7.append((df_new['1st'][i+1])+(df_new['1st'][i])+(df_new['2nd'][i+1])+(df_new['2nd'][i]))
        #list8.append((df_new['3rd'][i+1])+(df_new['3rd'][i])+(df_new['4th'][i+1])+(df_new['4th'][i]))
       # list6.append(df_new['Team'][i+1])
    list1 = list(np.repeat(list1, 2))
    list2 = list(np.repeat(list2, 2))
    #print(list2)
    #for i in range(0,len(df_new),2):

    weeksgames_df['Actual Total']=list1
    weeksgames_df['Actual Score Differential']=list2
    #weeksgames_df['Overs']= np.where(((weeksgames_df['Actual Total'])>= ((weeksgames_df['total']))),1, 0)
    weeksgames_df['Actual Score Differential'] = weeksgames_df['Actual Score Differential'].apply(float)
   # weeksgames_df['game_won']=full_game_data['game_won'].apply(int)
#     weeksgames_df['Actual Covering']= np.where(((((weeksgames_df['Actual Score Differential'])>= (np.abs(weeksgames_df['new spread'])))&(weeksgames_df['new spread'])<0)),1, 0)
#     weeksgames_df['Actual Covering']= np.where(((((weeksgames_df['Actual Covering'])==1)& (weeksgames_df['game_won'])==1)),1, 0)
    
    return weeksgames_df

In [52]:
weeksgames_df.head()

NameError: name 'weeksgames_df' is not defined

In [21]:
weeks_list = list(range(1,19))

In [ ]:
game_stats_info(game_info(1,2021).games['1-2021'][0]['boxscore'])

In [314]:
weeksgames_df_test = g_data_till_week(weeks_list, 2021)

NFL games for week 1
NFL games for week 2
NFL games for week 3
NFL games for week 4
NFL games for week 5
NFL games for week 6
NFL games for week 7
NFL games for week 8
NFL games for week 9
NFL games for week 10
NFL games for week 11
NFL games for week 12
NFL games for week 13
NFL games for week 14
NFL games for week 15
NFL games for week 16
NFL games for week 17
NFL games for week 18


In [326]:
weeksgames_df_test = weeksgames_df_test.fillna(value=np.nan)
weeksgames_df_test = weeksgames_df_test.fillna(0)
weeksgames_df_test= weeksgames_df_test.replace('no wind ', 0)

In [ ]:
list1 = []
#weeksgames_df=weeksgames_df.reset_index()
for i in range(len(weeksgames_df_test)):
    print(i)
    if weeksgames_df_test['wind'][i] == ' no wind':
        list1.append(0)
        #weeksgames_df['new spread']=((weeksgames_df['spread'][i])* 1)
    else:
        list1.append(weeksgames_df_test['wind'][i])
list1


In [338]:
list1[1][-6:-3]

' 9 '

In [349]:
list2 = []
for i in range (len(list1)):
    if isinstance(list1[i], str) == True:
        list2.append(list1[i][-6:-3])
    else:
        list2.append(list1[i])
#print(list1)
list3=[]
for i in range (len(list2)):
    if isinstance(list2[i], str) == True:
        list3.append(list2[i].strip(' '))
    else:
        list3.append(list2[i])
#print(list1)       

floats = [float(i) for i in list3]
weeksgames_df_test['wind']=floats

In [ ]:
weeksgames_df_test 

In [ ]:
weeksgames_df['spread']= weeksgames_df['vegas_odds'].str[-5:]

In [ ]:
weeksgames_df['total']= weeksgames_df['over_under'].str[:4]

In [ ]:
weeksgames_df['spread']=np.where(weeksgames_df['spread'] =='Pick',0, weeksgames_df['spread'])
weeksgames_df['spread'] = weeksgames_df['spread'].apply(float)
weeksgames_df['total'] = weeksgames_df['total'].apply(float)

In [ ]:
#weeksgames_df['vegas_odds'][i].str[:-4]

In [ ]:
weeksgames_df.head(10)

In [ ]:
#weeksgames_df=weeksgames_df.reset_index()
lst = []
for i in range(len(weeksgames_df)):
    #print(weeksgames_df['vegas_odds'][i][:-5])
    #print(weeksgames_df['vegas_odds'][i][:-5].replace(" ",""))
    if weeksgames_df['team_name'][i].replace(" ","") == weeksgames_df['vegas_odds'][i][:-5].replace(" ",""):
        lst.append((weeksgames_df['spread'][i])* 1)
        #weeksgames_df['new spread']=((weeksgames_df['spread'][i])* 1)
    else:
        lst.append((weeksgames_df['spread'][i])* -1)
weeksgames_df['new spread']=lst
    

In [ ]:
weeksgames_df['vegas_odds'][0][:-5].replace(" ","")

In [ ]:
weeksgames_df['vegas_odds'].str[:-4]

In [ ]:
game_info(weeks_list[3],2022).games['4-2022'][0]['boxscore']

In [ ]:
aggregate_games_df

1. Importing the team statistics for each game from the year 2010 to 2020 using the g_data function and exporting it to a csv file (so that we don't need to extract the data from the open source, hence increasing the speed)
2. Changing names of the teams that have changed their name in the last 10 years to maintain homogeneity 
3. Concating the team statistics for each game of the current year (2021) to the similar team stats from 2010 to 2020

In [164]:
full_game_data = pd.DataFrame()
for n in range(2019, 2020):
    game_data_ = g_data_till_week(list(range(1, 18)), n)
    game_data_['year'] = n
    full_game_data = pd.concat([full_game_data, game_data_])
full_game_data.reset_index(drop = True, inplace = True)

In [148]:
full_game_data.to_csv('2010-2018_vegas_odds_full_game_data.csv')

In [151]:
full_game_data_2010_2019=pd.read_csv('2010-2018_vegas_odds_full_game_data.csv')

In [168]:
full_game_data = pd.concat([full_game_data_2010_2019, full_game_data])

In [154]:
full_game_data = full_game_data.loc[:, ~full_game_data.columns.str.contains('^Unnamed')]

In [172]:
full_game_data=full_game_data.drop(columns=['index'])

In [63]:
full_game_data=pd.read_csv('final_2010-2022_full_game_data_with_spread_no_wind_revised2.csv')

In [173]:
full_game_data.to_csv('2010-2019_vegas_odds_full_game_datarev1.csv')

In [17]:
full_game_data = full_game_data.loc[:, ~full_game_data.columns.str.contains('^Unnamed')]

In [179]:
full_game_data_2010_2019=pd.read_csv('2010-2019_vegas_odds_full_game_datarev1.csv')

In [16]:
full_game_data = pd.read_csv('final_2010-2022_full_game_data_with_spread_no_wind.csv')

In [18]:
full_game_data = full_game_data.replace('Oakland Raiders', 'Las Vegas Raiders')
full_game_data = full_game_data.replace('Washington Football Team', 'Washington Commanders')
#full_game_data = full_game_data

In [ ]:
import numpy as np
full_game_data
list1=[] 
list2=[]
list6=[]
list7=[]
list8=[]
for i in range(0,len(full_game_data),2):
    list1.append((full_game_data['score'][i+1])+(full_game_data['score'][i]))
    list2.append((full_game_data['score'][i+1])-(full_game_data['score'][i]))
    #list7.append((df_new['1st'][i+1])+(df_new['1st'][i])+(df_new['2nd'][i+1])+(df_new['2nd'][i]))
    #list8.append((df_new['3rd'][i+1])+(df_new['3rd'][i])+(df_new['4th'][i+1])+(df_new['4th'][i]))
   # list6.append(df_new['Team'][i+1])
list1 = list(np.repeat(list1, 2))
list2 = list(np.repeat(list2, 2))
print(list2)
#for i in range(0,len(df_new),2):

full_game_data['Actual Total']=list1
full_game_data['Actual Score Differential']=list2
full_game_data['Overs']= np.where(((full_game_data['Actual Total'])>= ((full_game_data['total']))),1, 0)
full_game_data['Actual Score Differential'] = full_game_data['Actual Score Differential'].apply(float)
full_game_data['game_won']=full_game_data['game_won'].apply(int)
full_game_data['Actual Covering']= np.where(((((full_game_data['Actual Score Differential'])>= (np.abs(full_game_data['new spread'])))&(full_game_data['new spread'])<0)),1, 0)
full_game_data['Actual Covering']= np.where(((((full_game_data['Actual Covering'])==1)& (full_game_data['game_won'])==1)),1, 0)
full_game_data.dtypes

In [ ]:
full_game_data.to_csv('full_game_data4.csv')

In [264]:
full_game_data = pd.read_csv('full_game_data6.csv')
full_game_data = full_game_data.loc[:, ~full_game_data.columns.str.contains('^Unnamed')]

In [26]:
full_game_data=full_game_data.reset_index()

In [ ]:
full_game_data.head(100)

In [21]:
full_game_data.to_csv('full_game_data6.csv')
full_game_data= pd.read_csv('full_game_data6.csv')

NameError: name 'full_game_data' is not defined

In [34]:
g_data_till_week(weeks_list, 2022)

,index,team_name,weather,over_under,vegas_odds,team_abbr,score,game_won,game_lost,first_downs,...,week,spread,wind,total,over or under,new spread,Actual Total,Actual Score Differential,Overs,Actual Covering
0,0,Buffalo Bills,None,52.0 (under),Buffalo Bills -2.5,buf,31,0,0,23,...,1,-2.5,None,52.0,(under),-2.5,41,-21.0,0,0
1,0,Los Angeles Rams,None,52.0 (under),Buffalo Bills -2.5,ram,10,1,1,19,...,1,-2.5,None,52.0,(under),2.5,41,-21.0,0,0
2,0,New Orleans Saints,None,43.5 (over),Atlanta Falcons -5.5,nor,27,1,0,18,...,1,-5.5,None,43.5,(over),5.5,53,-1.0,1,0
3,0,Atlanta Falcons,None,43.5 (over),Atlanta Falcons -5.5,atl,26,0,1,26,...,1,-5.5,None,43.5,(over),-5.5,53,-1.0,1,0
4,0,Cleveland Browns,None,42.0 (over),Carolina Panthers -1.5,cle,26,0,0,23,...,1,-1.5,None,42.0,(over),1.5,50,-2.0,1,0
5,0,Carolina Panthers,None,42.0 (over),Carolina Panthers -1.5,car,24,1,1,15,...,1,-1.5,None,42.0,(over),-1.5,50,-2.0,1,0
6,0,San Francisco 49ers,None,38.0 (under),Chicago Bears -6.0,sfo,10,1,1,17,...,1,-6.0,None,38.0,(under),6.0,29,9.0,0,0
7,0,Chicago Bears,None,38.0 (under),Chicago Bears -6.0,chi,19,0,0,15,...,1,-6.0,None,38.0,(under),-6.0,29,9.0,0,0
8,0,Pittsburgh Steelers,None,44.0 (under),Cincinnati Bengals -7.0,pit,23,1,0,13,...,1,-7.0,None,44.0,(under),7.0,43,-3.0,0,0
9,0,Cincinnati Bengals,None,44.0 (under),Cincinnati Bengals -7.0,cin,20,0,1,32,...,1,-7.0,None,44.0,(under),-7.0,43,-3.0,0,0


### full_game_data populate

In [34]:
full_game_data = pd.read_csv('final_2010-2022_full_game_data_wk2_updatedrev3.csv')
full_game_data = full_game_data.loc[:, ~full_game_data.columns.str.contains('^Unnamed')]

In [36]:
full_game_data_1 = g_data_till_week(weeks_list, current_y)
full_game_data_1['year'] = current_y
#full_game_data_1 = full_game_data_1.drop(columns=['index'])
full_game_data = pd.concat([full_game_data, full_game_data_1])
full_game_data.reset_index(drop = True, inplace = True)

In [180]:
full_game_data=pd.concat([full_game_data_2010_2019, full_game_data])

In [35]:
full_game_data = full_game_data.loc[full_game_data['year']!=2022]

In [22]:
full_game_data = full_game_data.loc[:, ~full_game_data.columns.str.contains('^Unnamed')]

In [ ]:
full_game_data=full_game_data.drop(columns=['weather','wind'])

In [24]:
full_game_data=full_game_data.drop(columns=['Unnamed: 0'])

In [33]:
full_game_data.to_csv('final_2010-2022_week3_updatedrev2.csv')

In [37]:
full_game_data = full_game_data.replace('Oakland Raiders', 'Las Vegas Raiders')
full_game_data = full_game_data.replace('San Diego Chargers', 'Los Angeles Chargers')
full_game_data = full_game_data.replace('St. Louis Rams', 'Los Angeles Rams')
full_game_data = full_game_data.replace('Washington Football Team', 'Washington Commanders')
full_game_data = full_game_data.replace('Washington Redskins', 'Washington Commanders')

In [39]:
full_game_data.tail(100)

,team_name,over_under,vegas_odds,team_abbr,score,game_won,game_lost,first_downs,fourth_down_attempts,fourth_down_conversions,fumbles,fumbles_lost,interceptions,net_pass_yards,pass_attempts,pass_completions,pass_touchdowns,pass_yards,penalties,points,rush_attempts,rush_touchdowns,rush_yards,third_down_attempts,third_down_conversions,time_of_possession,times_sacked,total_yards,turnovers,yards_from_penalties,yards_lost_from_sacks,week,total,over or under,new spread,Actual Total,Actual Score Differential,Overs,Actual Covering,year,Actual Covering2,weather,wind
6202,Chicago Bears,41.5 (under),Green Bay Packers -10.0,chi,10,0,1,11,1,0,1,0,1,48,11,7,0,70,7,10,27,1,180,7,1,1365,3,228,1,50,22,2,NaN,NaN,NaN,37,17.0,NaN,NaN,2022,NaN,None,NaN
6203,Green Bay Packers,41.5 (under),Green Bay Packers -10.0,gnb,27,1,0,26,0,0,3,1,0,211,25,19,2,234,3,27,38,1,203,9,5,2235,3,414,1,25,23,2,NaN,NaN,NaN,37,17.0,NaN,NaN,2022,NaN,None,NaN
6204,Tennessee Titans,47.5 (over),Buffalo Bills -10.0,oti,7,0,1,12,1,1,3,2,2,107,24,12,0,123,9,7,27,1,80,12,4,1706,2,187,4,87,16,2,NaN,NaN,NaN,48,34.0,NaN,NaN,2022,NaN,None,NaN
6205,Buffalo Bills,47.5 (over),Buffalo Bills -10.0,buf,41,1,0,23,3,2,0,0,0,313,40,26,4,317,8,41,24,0,101,13,5,1894,1,414,0,49,4,2,NaN,NaN,NaN,48,34.0,NaN,NaN,2022,NaN,None,NaN
6206,Minnesota Vikings,50.0 (under),Philadelphia Eagles -2.5,min,7,0,1,20,0,0,1,0,3,202,46,27,1,221,5,7,11,0,62,12,4,1426,2,264,3,25,19,2,NaN,NaN,NaN,31,17.0,NaN,NaN,2022,NaN,None,NaN
6207,Philadelphia Eagles,50.0 (under),Philadelphia Eagles -2.5,phi,24,1,0,25,0,0,1,0,1,323,31,26,1,333,8,24,34,2,163,13,7,2174,3,486,1,60,10,2,NaN,NaN,NaN,31,17.0,NaN,NaN,2022,NaN,None,NaN
6208,Pittsburgh Steelers,38.0 (over),Cleveland Browns -4.5,pit,17,0,1,20,0,0,2,1,0,204,32,20,0,207,5,17,22,2,104,9,1,1431,1,308,1,25,3,3,NaN,NaN,NaN,46,12.0,NaN,NaN,2022,NaN,None,NaN
6209,Cleveland Browns,38.0 (over),Cleveland Browns -4.5,cle,29,1,0,22,4,3,0,0,0,205,31,21,2,220,5,29,38,1,171,16,6,2169,2,376,0,24,15,3,NaN,NaN,NaN,46,12.0,NaN,NaN,2022,NaN,None,NaN
6210,New Orleans Saints,41.0 (under),New Orleans Saints -2.5,nor,14,0,1,19,0,0,3,1,2,342,41,25,1,353,7,14,22,1,84,13,5,1790,1,426,3,58,11,3,NaN,NaN,NaN,36,8.0,NaN,NaN,2022,NaN,None,NaN
6211,Carolina Panthers,41.0 (under),New Orleans Saints -2.5,car,22,1,0,12,1,0,0,0,0,148,25,12,1,170,4,22,31,0,145,14,4,1810,3,293,0,40,22,3,NaN,NaN,NaN,36,8.0,NaN,NaN,2022,NaN,None,NaN


In [198]:
len(full_game_data.drop_duplicates())

1024

In [ ]:
full_game_data = pd.concat([full_game_data, full_game_data_1])
full_game_data.reset_index(drop = True, inplace = True)

### 2.5. Full game data (stats) dataframe exploration

In [ ]:
full_game_data.head()

In [ ]:
full_game_data.tail()

In [ ]:
full_game_data.info()

1. We can see that the team stats data from sportsipy NFL package are complete (no null/missing values)
2. There are also no duplicate values (as indicated below) since each row is attributed to each team's performance for a specific match
2. This dataframe is not the final dataframe that will be used in the machine learning application
3. However, we can analyze and extract some insights from this dataframe before creating the final dataframe

In [ ]:
full_game_data.duplicated().unique()

In [ ]:
# We first can do a single variable (univariat) analysis by plotting its histogram
# Since the game is played with each team trying to score to win, we'll use this variable
def create_histogram_for_one_variable(full_game_data, variable_name):
    full_game_data[variable_name].hist(bins=15, figsize=(5,4), grid=False, color='midnightblue')
    plt.xlabel(variable_name)
    plt.ylabel('Frequency')
    plt.title('Histogram of ' + variable_name)
    print(variable_name)

variable_name = 'score'
create_histogram_for_one_variable(full_game_data, variable_name)

1. We can see that it's not that balanced and there are some extremes to the right side
3. From the histogram, we can conclude that the teams usually score between 15-30
2. The data also shows that there are less and less observations when a team scores more than 40 in a match

In [ ]:
# We can further stack the histogram by the year
def create_stacked_histogram_for_one_variable(full_game_data, variable_name):
    from matplotlib import cm
    colors = cm.inferno_r(np.linspace(.3, .7, 12))
    full_game_data.pivot(columns='year')['score'].plot(kind = 'hist', bins = 15,
                                                       figsize=(5,4), grid=False,
                                                       stacked=True, color=colors)
    plt.xlabel(variable_name)
    plt.ylabel('Frequency')
    plt.title('Histogram of ' + variable_name + ' Grouped by Year')

variable_name = 'score'
create_stacked_histogram_for_one_variable(full_game_data, variable_name)

1. From the graph above, we see that there is no significant difference between each year in each bar
2. The frequency tends to be divided fairly every season
3. It may be easier to see the cumulative histogram to see that the height of each bar is fairly divided

In [ ]:
# We can plot the cumulative histogram of the above graph
def create_stacked_histogram_for_one_variable_cumulative(full_game_data, variable_name):
    from matplotlib import cm
    colors = cm.inferno_r(np.linspace(.3, .7, 12))
    full_game_data.pivot(columns='year')['score'].plot(kind = 'hist', bins = 15,
                                                       figsize=(5,5), grid=False,
                                                       stacked=True, color=colors,
                                                       cumulative=True)
    plt.xlabel(variable_name)
    plt.ylabel('Frequency')
    plt.title('Cumulative Histogram of ' + variable_name + ' Grouped by Year')

variable_name = 'score'
create_stacked_histogram_for_one_variable_cumulative(full_game_data, variable_name)

In [ ]:
# We are interested in the relationship between the score and all other team stats
def plot_scatter_for_score_vs_other_stats(full_game_data, no_of_cols):
    team_stats = full_game_data.drop(columns = ['team_name', 'team_abbr', 'game_won',
                                            'game_lost', 'week', 'year'])
    no_of_rows = (len(team_stats.columns)//no_of_cols)+1
    fig = plt.figure(figsize=(20,30))
    for i, col in enumerate(team_stats.iloc[:,1:].columns):
        ax = fig.add_subplot(no_of_rows,no_of_cols, i+1)
        ax.scatter(team_stats[col], team_stats['score'], c='dimgrey')
        ax.set_ylabel('score')
        ax.set_xlabel(col)
        ax.set_title('{} vs. {}'.format(col, 'score'), color='firebrick')
    fig.tight_layout()  
    plt.show()

plot_scatter_for_score_vs_other_stats(full_game_data, 4)

1. From the scatter plots above, we see that there are some stats that are positively correlated with score (first_downs, net_pass_yards, pass_touchdowns, pass_yards, points, rush_attempts, rush_touchdowns, rush_yards, time_of_possession, and total_yards)
2. However, there are also stats that are negatively correlated with score (fourth_down_attempts, fumbles_lost, interceptions, times_scaked, turnovers, and yards_lost_from_sacks)
3. There are also stats that seem to be inconclusive based on these charts, in other words, the dots are too scattered (fourth_down_conversions, fumbles, pass_attempts, pass_completions, penalties, third_down_attempts, third_down_conversions, and yards_from_penalties)
4. We can then further investigate the relationship between other variables

In [ ]:
# Plotting scatter plots between several variables in a list
def plot_scatter_by_choosing_variables_no1(full_game_data, list_of_var):
    import seaborn as sns
    sns.set_style("whitegrid", {'axes.grid' : False})
    sns.pairplot(full_game_data[list_of_var])

list_of_var = ['first_downs', 'rush_attempts', 'rush_yards', 'total_yards']
plot_scatter_by_choosing_variables_no1(full_game_data, list_of_var)

1. We suspect that variables that are positively correlated to the score will also have the same behavior between each other
2. The plot above shows that there are indeed a positive correlation between variables that contribute to scoring higher
3. It signals multicolinearity and we will confirm again and handle it later in our final dataframe
4. For the graph below, we plot the same thing, but now each dot represents the season of NFL

In [ ]:
# Plotting scatter plots between several variables in a list and differentiate each point based on the season (year)
def plot_scatter_by_choosing_variables_no2(full_game_data, list_of_var):
    full_game_data['season'] = full_game_data['year'].apply(lambda x: str(x))
    list_of_var.append('season')
    import seaborn as sns
    sns.set_style("whitegrid", {'axes.grid' : False})
    sns.pairplot(full_game_data[list_of_var], hue='season', palette='icefire', corner=True)

list_of_var = ['first_downs', 'rush_attempts', 'rush_yards', 'total_yards']
plot_scatter_by_choosing_variables_no2(full_game_data, list_of_var)

In [ ]:
# Plotting scatter plots between several variables in a list and differentiate each point based on the team name
def plot_scatter_by_choosing_variables_no3(full_game_data, list_of_var):
    list_of_var.append('team_name')
    import seaborn as sns
    sns.set_style("whitegrid", {'axes.grid' : False})
    sns.pairplot(full_game_data[list_of_var], hue='team_name', palette='coolwarm', corner=True)

list_of_var = ['interceptions', 'times_sacked', 'turnovers', 'yards_lost_from_sacks']
plot_scatter_by_choosing_variables_no3(full_game_data, list_of_var)

1. For the variables that are negatively correlated with the score, we can also see that some of them have linear relationship between each other based on the graphs above (turnovers and interceptions, yards_lost_from_sacks and times_sacked)
2. It further confirms that multicolinearity exists
3. For these above graphs, each point represents each team

In [ ]:
# Plotting histograms of a variable grouped by the winning and losing team
def create_feature_histogram_winning_vs_losing_team(full_game_data, variable_name):
    full_game_data.groupby('game_won')[variable_name].hist(bins=10, figsize=(5,3),
                                                           histtype='stepfilled',
                                                           alpha=0.4, grid=False)
    plt.xlabel(variable_name)
    plt.ylabel('Frequency')
    plt.title('Histogram of ' + variable_name + ' Grouped by Winning and Losing Team')
    plt.legend(['0.0: Losing Team','1.0: Winning Team'])

variable_name = 'first_downs'
create_feature_histogram_winning_vs_losing_team(full_game_data, variable_name)

In [ ]:
variable_name = 'rush_attempts'
create_feature_histogram_winning_vs_losing_team(full_game_data, variable_name)

In [ ]:
variable_name = 'rush_touchdowns'
create_feature_histogram_winning_vs_losing_team(full_game_data, variable_name)

1. The three histograms comparison above confirms that for a variable that is correlated positively to the score, the winning team histogram will be more on the right side
2. On the other hand, for the negative correlated variables (with score), as shown below, the losing team histogram tends to be more on the right side

In [ ]:
variable_name = 'turnovers'
create_feature_histogram_winning_vs_losing_team(full_game_data, variable_name)

In [ ]:
variable_name = 'times_sacked'
create_feature_histogram_winning_vs_losing_team(full_game_data, variable_name)

1. Based on some of the findings above and information that we have about the NFL team statistics, we believe that there are two types of team statistics: the good stats (stats that will increase the chance of winning) and the bad ones (stats that don't help the team)
2. Intuitively, we think that the teams that won the game should have better good stats compared to the teams that lost the game
3. On the contrary, the winning team would have worse bad stats compared to the losing team
4. In other words, if we take the stats difference (average of all macthes) between the winning team and the losing team, the good stats difference should have positive values (first graph below), and the bad stats difference should be negative (second graph below)
5. Later on, after performing the machine learning model, we can compare this hypothesis to the decomposed features' importance graph

#### Hypothesized good features (stats):
1. first_downs
2. fourth_down_conversions
3. time_of_possession
4. third_down_attempts
5. net_pass_yards
6. pass_attempts
7. pass_completions
8. pass_touchdowns
9. pass_yards
10. points
11. rush_attempts
12. rush_touchdowns
13. rush_yards
14. third_down_conversions
15. total_yards

#### Hypothesized bad features (stats):
1. fourth_down_attempts
2. fumbles
3. fumbles_lost
4. interceptions
5. penalties
6. times_sacked
7. turnovers
8. yards_from_penalties
9. yards_lost_from_sacks

In [ ]:
# Creating the good features plot between winning team vs. losing team: expected to be positive,
# and the bad features plot: expected to be negative
def plot_good_and_bad_features_comparison(full_game_data):
    winning_df = full_game_data[full_game_data["game_won"] == 1].drop(
        columns = ['team_name', 'team_abbr', 'score', 'game_won', 'game_lost',
        'week', 'year'])
    losing_df = full_game_data[full_game_data["game_lost"] == 1].drop(
        columns = ['team_name', 'team_abbr', 'score', 'game_won', 'game_lost',
        'week', 'year'])

    winning_good_df = winning_df[['first_downs','fourth_down_conversions','time_of_possession',
                         'third_down_attempts','net_pass_yards','pass_attempts',
                         'pass_completions','pass_touchdowns','pass_yards','points',
                         'rush_attempts','rush_touchdowns','rush_yards',
                         'third_down_conversions','total_yards']]
    winning_bad_df = winning_df[['fourth_down_attempts','fumbles','fumbles_lost','interceptions',
                          'penalties','times_sacked','turnovers','yards_from_penalties',
                          'yards_lost_from_sacks']]
    losing_good_df = losing_df[['first_downs','fourth_down_conversions','time_of_possession',
                         'third_down_attempts','net_pass_yards','pass_attempts',
                         'pass_completions','pass_touchdowns','pass_yards','points',
                         'rush_attempts','rush_touchdowns','rush_yards',
                         'third_down_conversions','total_yards']]
    losing_bad_df = losing_df[['fourth_down_attempts','fumbles','fumbles_lost','interceptions',
                          'penalties','times_sacked','turnovers','yards_from_penalties',
                          'yards_lost_from_sacks']]

    good_features = winning_good_df.mean() - losing_good_df.mean()
    bad_features = winning_bad_df.mean() - losing_bad_df.mean()
    
    plt.rcParams["figure.dpi"] = 80
    
    good_features.plot(kind='barh', figsize = (10, 6), color='midnightblue')
    plt.ylabel('Features')
    plt.legend(['Average Good Features of Winning Team - Average Good Features of Losing Team'], loc='best')
    plt.title('Good Features Comparison')
    plt.show()
    
    bad_features.plot(kind='barh', figsize = (10, 4), color='firebrick')
    plt.ylabel('Features')
    plt.legend(['Average Bad Features of Winning Team - Average Bad Features of Losing Team'], loc='best')
    plt.title('Bad Features Comparison')
    plt.show()

plot_good_and_bad_features_comparison(full_game_data)

### 2.6. More on visualization: folium map

1. We are also interested in creating folium map to summarize each team's performance (team stats)
2. To complete this, we create two maps: map that can show the summary from 2010 until the current week of current season and map that can show the summary for a specific year

In [ ]:
# Creating a folium map summarizing team stats from 2010 to 2021 (up until current week)
def create_team_stats_map(full_game_data):
    import folium
    import geojsonio
    import json
    import branca
    from IPython.display import IFrame
            
    team_loc = pd.read_csv('team_information.csv')
    team_stats = full_game_data.drop(columns=['team_abbr', 'game_won', 'game_lost', 'week',
                                            'year']).groupby('team_name').mean().reset_index()
    team_won_and_lost = full_game_data[['team_name', 'game_won', 'game_lost'
                                       ]].groupby('team_name').sum().reset_index()
    temp_df = pd.merge(team_loc, team_won_and_lost, left_on='team_name',
                       right_on='team_name')
    df = pd.merge(temp_df, team_stats, left_on='team_name',
                  right_on='team_name').rename(columns = {'game_won': 'total_game_won',
                                                 'game_lost': 'total_game_lost',
                                                 'score': 'average_score',
                                                 'first_downs': 'average_first_downs',
                                                 'fourth_down_attempts': 'average_fourth_down_attempts',
                                                 'fourth_down_conversions': 'average_fourth_down_conversions',
                                                 'fumbles': 'average_fumbles',
                                                 'fumbles_lost': 'average_fumbles_lost',
                                                 'interceptions': 'average_interceptions',
                                                 'net_pass_yards': 'average_net_pass_yards',
                                                 'pass_attempts': 'average_pass_attempts',
                                                 'pass_completions': 'average_pass_completions',
                                                 'pass_touchdowns': 'average_pass_touchdowns',
                                                 'pass_yards': 'average_pass_yards',
                                                 'penalties': 'average_penalties',
                                                 'points': 'average_points',
                                                 'rush_attempts': 'average_rush_attempts',
                                                 'rush_touchdowns': 'average_rush_touchdowns',
                                                 'rush_yards': 'average_rush_yards',
                                                 'third_down_attempts': 'average_third_down_attempts',
                                                 'third_down_conversions': 'average_third_down_conversions',
                                                 'time_of_possession': 'average_time_of_possession',
                                                 'times_sacked': 'average_times_sacked',
                                                 'total_yards': 'average_total_yards',
                                                 'turnovers': 'average_turnovers',
                                                 'yards_from_penalties': 'average_yards_from_penalties',
                                                 'yards_lost_from_sacks': 'average_yards_lost_from_sacks'})    
    
    usa_center = (37.0902, -95.7129)
    team_map = folium.Map(location=usa_center, zoom_start=4)
    
    for i in df.index:
        popup_dict = dict()
        for var in df.iloc[:,10:].columns:
            popup_dict[var] = f'{df.iloc[:,10:].loc[i, var]:.2f}'
            popup_df = pd.DataFrame(list(popup_dict.items()), columns=[['Attributes', 'Values']])
            popup_html = popup_df.to_html(index=False)
            iframe = branca.element.IFrame(html=popup_html, width=400, height=300)
            popup = folium.Popup(iframe, max_width=2650)
        
        icon = folium.features.CustomIcon(df.loc[i,'team_logo_wikipedia'], icon_size=(36, 36))
        folium.Marker(location=[df.loc[i,'lat'], df.loc[i,'long']],
                      popup=popup,
                      tooltip=df.loc[i, 'team_name'],
                      icon=icon).add_to(team_map)
    
    title = 'Summary of Team Stats 2010-2021'
    title_html = '''
             <h3 align="center" style="font-size:20px"><b>{title}</b></h3>
             '''.format(title=title)
    team_map.get_root().html.add_child(folium.Element(title_html))
    
    return team_map

create_team_stats_map(full_game_data)

1. With the map, we can just click the team logo to get the summarized stats
2. Below is the function to create the same map but only the stats of a given year will be shown

In [ ]:
# Creating a folium map summarizing team stats in a given (specific) year
def create_yearly_team_stats_map(full_game_data, year):
    import folium
    import geojsonio
    import json
    import branca
    from IPython.display import IFrame
    
    full_game_data_year = full_game_data[full_game_data['year'] == year]
            
    team_loc = pd.read_csv('team_information.csv')
    team_stats = full_game_data_year.drop(columns=['team_abbr', 'game_won', 'game_lost', 'week',
                                            'year']).groupby('team_name').mean().reset_index()
    team_won_and_lost = full_game_data_year[['team_name', 'game_won', 'game_lost'
                                       ]].groupby('team_name').sum().reset_index()
    temp_df = pd.merge(team_loc, team_won_and_lost, left_on='team_name',
                       right_on='team_name')
    df = pd.merge(temp_df, team_stats, left_on='team_name',
                  right_on='team_name').rename(columns = {'game_won': 'total_game_won',
                                                 'game_lost': 'total_game_lost',
                                                 'score': 'average_score',
                                                 'first_downs': 'average_first_downs',
                                                 'fourth_down_attempts': 'average_fourth_down_attempts',
                                                 'fourth_down_conversions': 'average_fourth_down_conversions',
                                                 'fumbles': 'average_fumbles',
                                                 'fumbles_lost': 'average_fumbles_lost',
                                                 'interceptions': 'average_interceptions',
                                                 'net_pass_yards': 'average_net_pass_yards',
                                                 'pass_attempts': 'average_pass_attempts',
                                                 'pass_completions': 'average_pass_completions',
                                                 'pass_touchdowns': 'average_pass_touchdowns',
                                                 'pass_yards': 'average_pass_yards',
                                                 'penalties': 'average_penalties',
                                                 'points': 'average_points',
                                                 'rush_attempts': 'average_rush_attempts',
                                                 'rush_touchdowns': 'average_rush_touchdowns',
                                                 'rush_yards': 'average_rush_yards',
                                                 'third_down_attempts': 'average_third_down_attempts',
                                                 'third_down_conversions': 'average_third_down_conversions',
                                                 'time_of_possession': 'average_time_of_possession',
                                                 'times_sacked': 'average_times_sacked',
                                                 'total_yards': 'average_total_yards',
                                                 'turnovers': 'average_turnovers',
                                                 'yards_from_penalties': 'average_yards_from_penalties',
                                                 'yards_lost_from_sacks': 'average_yards_lost_from_sacks'})    
    
    usa_center = (37.0902, -95.7129)
    team_map = folium.Map(location=usa_center, zoom_start=4, tiles='cartodbpositron')
    
    for i in df.index:
        popup_dict = dict()
        for var in df.iloc[:,10:].columns:
            popup_dict[var] = f'{df.iloc[:,10:].loc[i, var]:.2f}'
            popup_df = pd.DataFrame(list(popup_dict.items()), columns=[['Attributes', 'Values']])
            popup_html = popup_df.to_html(index=False)
            iframe = branca.element.IFrame(html=popup_html, width=400, height=300)
            popup = folium.Popup(iframe, max_width=2650)
        
        icon = folium.features.CustomIcon(df.loc[i,'team_logo_wikipedia'], icon_size=(36, 36))
        folium.Marker(location=[df.loc[i,'lat'], df.loc[i,'long']],
                      popup=popup,
                      tooltip=df.loc[i, 'team_name'],
                      icon=icon).add_to(team_map)
    
    title = 'Summary of Team Stats in ' + str(year)
    title_html = '''
             <h3 align="center" style="font-size:20px"><b>{title}</b></h3>
             '''.format(title=title)
    team_map.get_root().html.add_child(folium.Element(title_html))
    
    return team_map

create_yearly_team_stats_map(full_game_data, 2020)

1. Having the team stats performace from 2010-2021 and plot them on a folium map is a great thing
2. With the map and stats, we can directly compare the teams to identify how strong a team is compared to the others (relative team strength based on the team stats)
3. However, one might be curious about which teams have the same overall strength based on the team stats and how 'close' each team is with the other teams based on their summarized stats
4. This is where we can apply network analysis

### 2.7. More on visualization: network

1. Using the overall stats performance of each team like what we have used in the folium map, we will calculate the 'distance' between each team
2. Distance, in this case, is calculated by using the euclidian method
3. Distance represents how close or how far each team is, based on the overall team stats performance
4. In other words, two teams that are close in distance will have similar team stats performance from 2010-2021
5. On the other hand, two teams that are far from each other will have opposite team stats performance
6. However, using distance can be a bit confusing since the 'smaller' the distance actually means the 'greater' the similarity between two teams
7. Therefore, for easier interpretation, we'll create similarity_distance which equals to '1 divided by the distance'
8. Hence, the smaller the distance -> the greater the similarity_distance, and the greater the similarity (in terms of performance stats) between the teams

In [ ]:
def create_euclidean_distance_df(full_game_data):
    team_stats = full_game_data.drop(columns=['game_won', 'game_lost', 'week',
        'year']).groupby(['team_name', 'team_abbr']).mean().reset_index()
    team_won_and_lost = full_game_data[['team_name', 'team_abbr', 'game_won', 'game_lost'
        ]].groupby(['team_name', 'team_abbr']).sum().reset_index()

    df = pd.merge(team_won_and_lost, team_stats, left_on=['team_name','team_abbr'], right_on=['team_name',
        'team_abbr']).rename(columns = {'game_won': 'total_game_won',
                                        'game_lost': 'total_game_lost',
                                        'score': 'average_score',
                                        'first_downs': 'average_first_downs',
                                        'fourth_down_attempts': 'average_fourth_down_attempts',
                                        'fourth_down_conversions': 'average_fourth_down_conversions',
                                        'fumbles': 'average_fumbles',
                                        'fumbles_lost': 'average_fumbles_lost',
                                        'interceptions': 'average_interceptions',
                                        'net_pass_yards': 'average_net_pass_yards',
                                        'pass_attempts': 'average_pass_attempts',
                                        'pass_completions': 'average_pass_completions',
                                        'pass_touchdowns': 'average_pass_touchdowns',
                                        'pass_yards': 'average_pass_yards',
                                        'penalties': 'average_penalties',
                                        'points': 'average_points',
                                        'rush_attempts': 'average_rush_attempts',
                                        'rush_touchdowns': 'average_rush_touchdowns',
                                        'rush_yards': 'average_rush_yards',
                                        'third_down_attempts': 'average_third_down_attempts',
                                        'third_down_conversions': 'average_third_down_conversions',
                                        'time_of_possession': 'average_time_of_possession',
                                        'times_sacked': 'average_times_sacked',
                                        'total_yards': 'average_total_yards',
                                        'turnovers': 'average_turnovers',
                                        'yards_from_penalties': 'average_yards_from_penalties',
                                        'yards_lost_from_sacks': 'average_yards_lost_from_sacks'})
    
    from scipy.spatial.distance import pdist
    import itertools

    distance_df = pd.DataFrame(itertools.combinations(df['team_abbr'].values, 2), columns=['team_1','team_2'])
    distance_df['distance'] = pdist(df.iloc[:,2:].values, 'euclid')
    distance_df['similarity_distance'] = 1 / distance_df['distance']
    
    return distance_df

In [ ]:
distance_df = create_euclidean_distance_df(full_game_data)

In [ ]:
distance_df

1. The distance_df has 496 rows because there are 32 teams and a distance represents a pair of 2 teams' distance (32 choose 2 combinations)
2. The descriptive stats of distance is shown below

In [ ]:
distance_df.describe()

1. Before drawing the network, we want to make sure that the network won't be too cluttered
2. Therefore, we will have a threshold to remove the edges (pair of two teams) where the similarity_distance is below this threshold number (less significant)
3. If we don't remove edges with large distance, every node will have 31 edges and it will be cluttered
4. We also want to identify the relative closeness of distance by giving thicker edges to smaller distance between two nodes (scaling the edge thickness to indicate the closeness in distance: thicker means closer)
5. We also want to identify the most connected nodes by adjusting the size of nodes (scale the size of nodes based on its degree to indicate which teams have the greatest number of closeness in distance with other teams)
6. We'll use the 'Fruchterman-Reingold' layout algorithm which will set the positions of each node by minimizing the distance between the nodes with high similarity_distance

In [ ]:
# Define the network by previously removing edges that are below a threshold
def create_network_and_remove_edges_below_mean(distance_df):
    
    distance_df = distance_df[['team_1', 'team_2', 'similarity_distance']]
    
    # Set the threshold to be equal to the mean
    threshold_distance = distance_df.describe().loc['mean', 'similarity_distance']
    
    import networkx as nx
    # Create a graph from edge list
    G_team = nx.from_pandas_edgelist(distance_df, 'team_1', 'team_2', edge_attr=['similarity_distance'])

    # List to store edges to remove
    edges_removed_list = []

    # Loop through edges in G and find distance which are below the threshold
    for team_1, team_2 in G_team.edges():
        edge_distance = G_team[team_1][team_2]['similarity_distance']
        if edge_distance < threshold_distance:
            edges_removed_list.append((team_1, team_2))

    # Remove edges contained in the remove list
    G_team.remove_edges_from(edges_removed_list)
    
    print(str(len(edges_removed_list)) + " edges have been removed")
    
    return G_team

In [ ]:
# Define the G_team and get information on how many edges have been removed
G_team = create_network_and_remove_edges_below_mean(distance_df)

In [ ]:
# Draw the network with Fruchterman Reingold layout
def draw_network_of_nfl_team(G_team):
    import networkx as nx
    
    # Set node size
    def assign_node_size(value, scaling_factor=2):
        return value**2 * scaling_factor
    
    node_size = []
    for key, value in dict(G_team.degree).items():
        node_size.append(assign_node_size(value))
    
    # Set edge thickness
    def assign_edge_thickness(value, scaling_factor=10):
        return (value*100)**2 / scaling_factor

    edge_width = []
    for key, value in nx.get_edge_attributes(G_team, 'similarity_distance').items():
        edge_width.append(assign_edge_thickness(value))
        
    import seaborn as sns
    sns.set(rc={'figure.figsize': (9, 9)})
    font_dict = {'fontsize': 18}
    
    nx.draw(G_team, pos=nx.fruchterman_reingold_layout(G_team),
            with_labels=True, node_size=node_size,
            node_color="#e1575c", edge_color='#363847',
            width=edge_width)

    plt.title("NFL Team Network Based on Team Stats Euclidian Distance", fontdict=font_dict)
    plt.show()

draw_network_of_nfl_team(G_team)

1. Based on the network above, we first can easily identify the teams with bigger size of node, which means they are the most connected teams
2. These teams, for instance, are 'clt' (Indianapolis Colts), 'cin' (Cincinnati Bengals), and 'min' (Minnesota Vikings)
3. Nodes that are positioned closer to each other also show thicker edges, meaning that we manage to signify the closeness of these nodes (teams)
4. For example, 'crd' (Arizona Cardinals) is closer to 'ram' (Los Angeles Rams), rather than to 'mia' (Miami Dolphins), and it is shown by the position and also the thickness level of their edges

### Team abbreviation
1. crd: Arizona Cardinals
2. atl: Atlanta Falcons
3. rav: Baltimore Ravens
4. buf: Buffalo Bills
5. car: Carolina Panthers
6. chi: Chicago Bears
7. cin: Cincinnati Bengals
8. cle: Cleveland Browns
9. dal: Dallas Cowboys
10. den: Denver Broncos
11. det: Detroit Lions
12. gnb: Green Bay Packers
13. htx: Houston Texans
14. clt: Indianapolis Colts
15. jax: Jacksonville Jaguars
16. kan: Kansas City Chiefs
17. rai: Las Vegas Raiders
18. sdg: Los Angeles Chargers
19. ram: Los Angeles Rams
20. mia: Miami Dolphins
21. min: Minnesota Vikings
22. nwe: New England Patriots
23. nor: New Orleans Saints
24. nyg: New York Giants
25. nyj: New York Jets
26. phi: Philadelphia Eagles
27. pit: Pittsburgh Steelers
28. sfo: San Francisco 49ers
29. sea: Seattle Seahawks
30. tam: Tampa Bay Buccaneers
31.	oti: Tennessee Titans
32. was: Washington Football Team

#### Drawing minimum spanning tree (MST)
1. Since the layout already positions the teams based on their closeness, we don't need the edges to represent closeness as well
2. We'll now use the MST to reduce the edges down to those necessary to connect all the teams
3. By doing this, we will have a better clustered network
4. We don't need the size of the node to be scaled right now since it will be meaningless in MST

In [ ]:
# Draw minimum spanning tree (MST) network
def draw_minimum_spanning_tree_of_nfl_team(G_team):
    import networkx as nx
    G_team_mst = nx.minimum_spanning_tree(G_team)
    
    import seaborn as sns
    sns.set(rc={'figure.figsize': (9, 9)})
    font_dict = {'fontsize': 18}
        
    # Draw minimum spanning tree, but we have to set node size and width to constant
    nx.draw(G_team_mst, with_labels=True,
            pos=nx.fruchterman_reingold_layout(G_team_mst),
            node_size=500, node_color="#e1575c",
            edge_color='#363847', width = 1.2)

    plt.title("NFL Team Network Based on Team Stats Euclidian Distance - Minimum Spanning Tree",
              fontdict=font_dict)
    plt.show()

draw_minimum_spanning_tree_of_nfl_team(G_team)

1. With the MST network above, we can better see how the teams are clustered
2. We can identify the centroids of the clusters: 'buf' (Buffalo Bills), 'crd' (Arizona Cardinals), and 'atl' (Atlanta Falcons)
3. There is also 'clt' (Indianapolis Colts) that connects two clusters

## 3. Data Finalization

#### Creating Dataframe to build and train the model

1. For schedule of each game from (2011-2021), we aggregate the statistics of each team playing the game and determine the impact of features on the result of the game
2. Statistics are calculated using the weighted average of statistics of games played by a team in that season and the previous season. For a given season, statistics up to the game played are taken into account.
3. For each game, we take statistics from past 17 weeks of team's performance
4. For example, for game in week 7 of a particular season, we use the statistics from the first 6 weeks of that season and last 11 weeks from the previous season
5. Weighted average is assigned to account for team changes in each season, momentum in each season, hence giving more weightage to performance of the current season than the last season
6. The differential statistics between each team are then calculated
7. We add a 'nan' value for the result if the game hasn't been played yet

'aggregate_weekly_data' function takes 5 arguments as inputs:
1. schedule dataframe -> full_schedule dataframe
2. weeksgames_df (game statistics of given week of a given year) -> full_game_data dataframe
3. current week
4. current year
5. weeks list
6. m (weightage given to statistics of past season)

In [ ]:
weeks_list

In [ ]:
schedule_df

In [380]:
current_y=2022

In [223]:
def aggregate_weekly_data(schedule_df, weeksgames_df, current_w, current_y, weeks_list, m):
    aggregate_games_df = pd.DataFrame()
    for n_year in range(2010, current_y + 1):
        weeksgames_df_1 = weeksgames_df[weeksgames_df.year == n_year]
        weeksgames_df_0 = weeksgames_df[weeksgames_df.year == n_year - 1]
        if n_year != current_y:
            schedule_df_1 = schedule_df[schedule_df.year == n_year]
            schedule_df_0 = schedule_df[schedule_df.year == n_year - 1]
        else:
            schedule_df_1 = schedule_df[schedule_df.year == n_year]
            schedule_df_1 = schedule_df_1[schedule_df_1.week <= current_w]
            schedule_df_0 = schedule_df[schedule_df.year == n_year - 1]
            
        for w in range(0, len(weeks_list)):
            games_df = schedule_df_1[schedule_df_1.week == weeks_list[w]]
            games_df = games_df.drop('year', axis = 1)

            if w == 0:
                aggregate_weekly_df = weeksgames_df_0[weeksgames_df_0.week >= weeks_list[w]].drop(columns = [
                    'score','week','game_won', 'game_lost']).groupby(by=["team_name", "team_abbr"]).mean().reset_index()      
                win_loss_df = weeksgames_df_0[weeksgames_df_0.week >= weeks_list[w]][["team_name",
                    "team_abbr",'game_won', 'game_lost']].groupby(by=["team_name", "team_abbr"]).sum().reset_index()
                win_loss_df['win_perc'] = win_loss_df['game_won'] / (win_loss_df['game_won'] + win_loss_df['game_lost'])
                win_loss_df = win_loss_df.drop(columns = ['game_won', 'game_lost'])
                try:
                    aggregate_weekly_df['fourth_down_perc'] = aggregate_weekly_df[
                        'fourth_down_conversions'] / agg_weekly_df['fourth_down_attempts']
                except:
                    aggregate_weekly_df['fourth_down_perc'] = 0
                aggregate_weekly_df['fourth_down_perc'] = aggregate_weekly_df['fourth_down_perc'].fillna(0)
                
                try:
                    aggregate_weekly_df['third_down_perc'] = aggregate_weekly_df[
                        'third_down_conversions'] / aggregate_weekly_df['third_down_attempts']
                except:
                    aggregate_weekly_df['third_down_perc'] = 0
                aggregate_weekly_df['third_down_perc'] = aggregate_weekly_df['third_down_perc'].fillna(0)
                
                aggregate_weekly_df = aggregate_weekly_df.drop(columns = ['fourth_down_attempts',
                                                                          'fourth_down_conversions',
                                                                          'third_down_attempts',
                                                                          'third_down_conversions'])
                
            else:
                aggregate_weekly_df_1 = weeksgames_df_1[weeksgames_df_1.week < weeks_list[w]].drop(columns = [
                    'score','week','game_won', 'game_lost']).groupby(by=["team_name", "team_abbr"]).mean().reset_index()       
                win_loss_df_1 = weeksgames_df_1[weeksgames_df_1.week < weeks_list[w]][["team_name",
                    "team_abbr",'game_won', 'game_lost']].groupby(by=["team_name", "team_abbr"]).sum().reset_index()
                win_loss_df_1['win_perc'] = win_loss_df_1['game_won'] / (win_loss_df_1['game_won'] + win_loss_df_1[
                    'game_lost'])
                win_loss_df_1 = win_loss_df_1.drop(columns = ['game_won', 'game_lost'])
                
                try:
                    aggregate_weekly_df_1['fourth_down_perc'] = aggregate_weekly_df_1[
                        'fourth_down_conversions'] / aggregate_weekly_df_1['fourth_down_attempts']
                except:
                    aggregate_weekly_df_1['fourth_down_perc'] = 0
                aggregate_weekly_df_1['fourth_down_perc'] = aggregate_weekly_df_1['fourth_down_perc'].fillna(0)
                
                try:
                    aggregate_weekly_df_1['third_down_perc'] = aggregate_weekly_df_1[
                        'third_down_conversions'] / aggregate_weekly_df_1['third_down_attempts']
                except:
                    aggregate_weekly_df_1['third_down_perc'] = 0
                aggregate_weekly_df_1['third_down_perc'] = aggregate_weekly_df_1['third_down_perc'].fillna(0)
                
                aggregate_weekly_df_1 = aggregate_weekly_df_1.drop(columns = ['fourth_down_attempts',
                                                                              'fourth_down_conversions',
                                                                              'third_down_attempts',
                                                                              'third_down_conversions'])
                
                aggregate_weekly_df_0 = weeksgames_df_0[weeksgames_df_0.week >= weeks_list[w]].drop(columns = [
                    'score','week','game_won', 'game_lost']).groupby(by=["team_name", "team_abbr"]).mean().reset_index()      
                win_loss_df_0 = weeksgames_df_0[weeksgames_df_0.week >= weeks_list[w]][["team_name",
                    "team_abbr",'game_won', 'game_lost']].groupby(by=["team_name", "team_abbr"]).sum().reset_index()
                win_loss_df_0['win_perc'] = win_loss_df_0['game_won'] / (win_loss_df_0['game_won'] + win_loss_df_0[
                    'game_lost'])
                win_loss_df_0 = win_loss_df_0.drop(columns = ['game_won', 'game_lost'])
                
                try:
                    aggregate_weekly_df_0['fourth_down_perc'] = aggregate_weekly_df_0[
                        'fourth_down_conversions'] / aggregate_weekly_df_0['fourth_down_attempts']
                except:
                    aggregate_weekly_df_0['fourth_down_perc'] = 0
                aggregate_weekly_df_0['fourth_down_perc'] = aggregate_weekly_df_0['fourth_down_perc'].fillna(0)

                try:
                    aggregate_weekly_df_0['third_down_perc'] = aggregate_weekly_df_0[
                        'third_down_conversions'] / aggregate_weekly_df_0['third_down_attempts']
                except:
                    aggregate_weekly_df_0['third_down_perc'] = 0
                aggregate_weekly_df_0['third_down_perc'] = aggregate_weekly_df_0['third_down_perc'].fillna(0)
                
                aggregate_weekly_df_0 = aggregate_weekly_df_0.drop(columns = ['fourth_down_attempts',
                                                                              'fourth_down_conversions',
                                                                              'third_down_attempts',
                                                                              'third_down_conversions'])
                
                name_abb_df = aggregate_weekly_df_1[['team_name', 'team_abbr']]
                aggregate_weekly_df = aggregate_weekly_df_1.select_dtypes(exclude=['object',
                    'datetime']) * (1-m) + aggregate_weekly_df_0.select_dtypes(exclude=['object', 'datetime']) * m
                win_loss_df = win_loss_df_1.select_dtypes(exclude=['object',
                    'datetime']) * (1-m) + win_loss_df_0.select_dtypes(exclude=['object', 'datetime']) * m
                
                aggregate_weekly_df = pd.concat([name_abb_df, aggregate_weekly_df], axis=1)
                win_loss_df = pd.concat([name_abb_df, win_loss_df], axis=1)
            
            aggregate_weekly_df = aggregate_weekly_df.drop('year', axis=1)
            aggregate_weekly_df = pd.merge(win_loss_df,aggregate_weekly_df,left_on = ['team_name',
                'team_abbr'], right_on = ['team_name', 'team_abbr'])
            away_df = pd.merge(games_df,aggregate_weekly_df,how = 'inner', left_on = ['away_name',
                'away_abbr'], right_on = ['team_name', 'team_abbr']).drop(columns = ['team_name', 'team_abbr'])
            
            list_column_change = list(away_df.columns[7:])
            list_new_away = column_name_manipulation_reverse(list_column_change, 'away')
            list_new_home = column_name_manipulation_reverse(list_column_change, 'home')

            away_df.columns = list(away_df.columns)[:7] + list_new_away
            
            home_df = pd.merge(games_df,aggregate_weekly_df,how = 'inner', left_on = ['home_name',
                'home_abbr'], right_on = ['team_name', 'team_abbr']).drop(columns = ['team_name', 'team_abbr'])
            home_df.columns = list(home_df.columns)[:7] + list_new_home
            
            aggregate_weekly_df = pd.merge(away_df,home_df,left_on = ['away_name', 'away_abbr', 'home_name',
                'home_abbr', 'winning_name', 'winning_abbr', 'week'], right_on = ['away_name', 'away_abbr',
                'home_name', 'home_abbr', 'winning_name', 'winning_abbr', 'week'])
            
            for n in range(len(list_column_change)):
                column_new = list_column_change[n] + '_' + 'dif'
                aggregate_weekly_df[column_new] = aggregate_weekly_df[list_new_away[n]] - aggregate_weekly_df[
                    list_new_home[n]]
                aggregate_weekly_df[column_new] = aggregate_weekly_df[column_new].fillna(0)
            
            aggregate_weekly_df = aggregate_weekly_df.drop(columns = list_new_away + list_new_home + [
                'fumbles_lost_dif'])
            
            if (aggregate_weekly_df['winning_name'].isnull().values.any() and n_year == current_y and weeks_list[w
                ] == current_w):
                conditions = [aggregate_weekly_df['winning_name'] == aggregate_weekly_df['away_name'],
                              aggregate_weekly_df['winning_name'] == aggregate_weekly_df['home_name']]
                choices = [1,0]
                aggregate_weekly_df['result'] = np.select(conditions,choices,default=np.nan)
                    
            elif aggregate_weekly_df['winning_name'].isnull().values.any():
                aggregate_weekly_df = aggregate_weekly_df.dropna()
                aggregate_weekly_df['result'] = (aggregate_weekly_df['winning_name'] == aggregate_weekly_df['away_name'])
                aggregate_weekly_df['result'] = aggregate_weekly_df['result'].astype('int')
                
            else:
                aggregate_weekly_df['result'] = (aggregate_weekly_df['winning_name'] == aggregate_weekly_df['away_name'])
                aggregate_weekly_df['result'] = aggregate_weekly_df['result'].astype('int')
            
            aggregate_weekly_df = aggregate_weekly_df.drop(columns = ['winning_name', 'winning_abbr'])
            aggregate_weekly_df['year'] = [n_year for n in range(len(aggregate_weekly_df))]
            
            new_columns_arrangement = list(aggregate_weekly_df.columns)[:5] + [
                list(aggregate_weekly_df.columns)[-1]] + list(aggregate_weekly_df.columns)[5:-1]
            aggregate_weekly_df = aggregate_weekly_df.reindex(columns=new_columns_arrangement)
            aggregate_weekly_df['year'] = aggregate_weekly_df.year.astype('int64')
            aggregate_games_df = pd.concat([aggregate_games_df, aggregate_weekly_df], axis = 0)
            aggregate_games_df = aggregate_games_df.reset_index().drop(columns = 'index')
            
    return aggregate_games_df

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [40]:
def aggregate_weekly_data(schedule_df, weeksgames_df, current_w, current_y, weeks_list, m):
    aggregate_games_df = pd.DataFrame()
    for n_year in range(2011, current_y + 1):
        weeksgames_df_1 = weeksgames_df[weeksgames_df.year == n_year]
        weeksgames_df_0 = weeksgames_df[weeksgames_df.year == n_year - 1]
        #over_df = weeksgames_df[['over_under','Overs']]
        if n_year != current_y:
            schedule_df_1 = schedule_df[schedule_df.year == n_year]
            schedule_df_0 = schedule_df[schedule_df.year == n_year - 1]
        else:
            schedule_df_1 = schedule_df[schedule_df.year == n_year]
            schedule_df_1 = schedule_df_1[schedule_df_1.week <= current_w]
            schedule_df_0 = schedule_df[schedule_df.year == n_year - 1]
        #print(schedule_df_0)
            
        for w in range(0, len(weeks_list)):
            games_df = schedule_df_1[schedule_df_1.week == weeks_list[w]]
            games_df = games_df.drop('year', axis = 1)
#             print(games_df)

            if w == 0:
                aggregate_weekly_df = weeksgames_df_0[weeksgames_df_0.week >= weeks_list[w]].drop(columns = [
                    'score','week','game_won', 'game_lost']).groupby(by=["team_name", "team_abbr"]).mean().reset_index()      
                win_loss_df = weeksgames_df_0[weeksgames_df_0.week >= weeks_list[w]][["team_name",
                    "team_abbr",'game_won', 'game_lost']].groupby(by=["team_name", "team_abbr"]).sum().reset_index()
                win_loss_df['win_perc'] = win_loss_df['game_won'] / (win_loss_df['game_won'] + win_loss_df['game_lost'])
                win_loss_df = win_loss_df.drop(columns = ['game_won', 'game_lost'])
                try:
                    aggregate_weekly_df['fourth_down_perc'] = aggregate_weekly_df[
                        'fourth_down_conversions'] / agg_weekly_df['fourth_down_attempts']
                except:
                    aggregate_weekly_df['fourth_down_perc'] = 0
                aggregate_weekly_df['fourth_down_perc'] = aggregate_weekly_df['fourth_down_perc'].fillna(0)
                
                try:
                    aggregate_weekly_df['third_down_perc'] = aggregate_weekly_df[
                        'third_down_conversions'] / aggregate_weekly_df['third_down_attempts']
                except:
                    aggregate_weekly_df['third_down_perc'] = 0
                aggregate_weekly_df['third_down_perc'] = aggregate_weekly_df['third_down_perc'].fillna(0)
                
                aggregate_weekly_df = aggregate_weekly_df.drop(columns = ['fourth_down_attempts',
                                                                          'fourth_down_conversions',
                                                                          'third_down_attempts',
                                                                          'third_down_conversions'])
                
            else:
                aggregate_weekly_df_1 = weeksgames_df_1[weeksgames_df_1.week < weeks_list[w]].drop(columns = [
                   'score', 'week','game_won', 'game_lost']).groupby(by=["team_name", "team_abbr"]).mean().reset_index()       
                win_loss_df_1 = weeksgames_df_1[weeksgames_df_1.week < weeks_list[w]][["team_name",
                    "team_abbr",'game_won', 'game_lost']].groupby(by=["team_name", "team_abbr"]).sum().reset_index()
                win_loss_df_1['win_perc'] = win_loss_df_1['game_won'] / (win_loss_df_1['game_won'] + win_loss_df_1[
                    'game_lost'])
                win_loss_df_1 = win_loss_df_1.drop(columns = ['game_won', 'game_lost'])
                
                try:
                    aggregate_weekly_df_1['fourth_down_perc'] = aggregate_weekly_df_1[
                        'fourth_down_conversions'] / aggregate_weekly_df_1['fourth_down_attempts']
                except:
                    aggregate_weekly_df_1['fourth_down_perc'] = 0
                aggregate_weekly_df_1['fourth_down_perc'] = aggregate_weekly_df_1['fourth_down_perc'].fillna(0)
                
                try:
                    aggregate_weekly_df_1['third_down_perc'] = aggregate_weekly_df_1[
                        'third_down_conversions'] / aggregate_weekly_df_1['third_down_attempts']
                except:
                    aggregate_weekly_df_1['third_down_perc'] = 0
                aggregate_weekly_df_1['third_down_perc'] = aggregate_weekly_df_1['third_down_perc'].fillna(0)
                
                aggregate_weekly_df_1 = aggregate_weekly_df_1.drop(columns = ['fourth_down_attempts',
                                                                              'fourth_down_conversions',
                                                                              'third_down_attempts',
                                                                              'third_down_conversions'])
                
                aggregate_weekly_df_0 = weeksgames_df_0[weeksgames_df_0.week >= weeks_list[w]].drop(columns = [
                    'score','week','game_won', 'game_lost']).groupby(by=["team_name", "team_abbr"]).mean().reset_index()      
                win_loss_df_0 = weeksgames_df_0[weeksgames_df_0.week >= weeks_list[w]][["team_name",
                    "team_abbr",'game_won', 'game_lost']].groupby(by=["team_name", "team_abbr"]).sum().reset_index()
                win_loss_df_0['win_perc'] = win_loss_df_0['game_won'] / (win_loss_df_0['game_won'] + win_loss_df_0[
                    'game_lost'])
                win_loss_df_0 = win_loss_df_0.drop(columns = ['game_won', 'game_lost'])
                
                try:
                    aggregate_weekly_df_0['fourth_down_perc'] = aggregate_weekly_df_0[
                        'fourth_down_conversions'] / aggregate_weekly_df_0['fourth_down_attempts']
                except:
                    aggregate_weekly_df_0['fourth_down_perc'] = 0
                aggregate_weekly_df_0['fourth_down_perc'] = aggregate_weekly_df_0['fourth_down_perc'].fillna(0)

                try:
                    aggregate_weekly_df_0['third_down_perc'] = aggregate_weekly_df_0[
                        'third_down_conversions'] / aggregate_weekly_df_0['third_down_attempts']
                except:
                    aggregate_weekly_df_0['third_down_perc'] = 0
                aggregate_weekly_df_0['third_down_perc'] = aggregate_weekly_df_0['third_down_perc'].fillna(0)
                
                aggregate_weekly_df_0 = aggregate_weekly_df_0.drop(columns = ['fourth_down_attempts',
                                                                              'fourth_down_conversions',
                                                                              'third_down_attempts',
                                                                              'third_down_conversions'])
                
                name_abb_df = aggregate_weekly_df_0[['team_name', 'team_abbr']]
                aggregate_weekly_df = aggregate_weekly_df_1.select_dtypes(exclude=['object',
                    'datetime']) * (1-m) + aggregate_weekly_df_0.select_dtypes(exclude=['object', 'datetime']) * m
                win_loss_df = win_loss_df_1.select_dtypes(exclude=['object',
                    'datetime']) * (1-m) + win_loss_df_0.select_dtypes(exclude=['object', 'datetime']) * m
                
                aggregate_weekly_df = pd.concat([name_abb_df, aggregate_weekly_df], axis=1)
                win_loss_df = pd.concat([name_abb_df, win_loss_df], axis=1)
           # print(aggregate_weekly_df['Actual Total'])
            aggregate_weekly_df = aggregate_weekly_df.drop('year', axis=1)
            aggregate_weekly_df = pd.merge(win_loss_df,aggregate_weekly_df,left_on = ['team_name',
                'team_abbr'], right_on = ['team_name', 'team_abbr'])
            away_df = pd.merge(games_df,aggregate_weekly_df,how = 'inner', left_on = ['away_name',
                'away_abbr'], right_on = ['team_name', 'team_abbr']).drop(columns = ['team_name', 'team_abbr'])
            
            list_column_change = list(away_df.columns[7:])
            list_new_away = column_name_manipulation_reverse(list_column_change, 'away')
            list_new_home = column_name_manipulation_reverse(list_column_change, 'home')

            away_df.columns = list(away_df.columns)[:7] + list_new_away
            
            home_df = pd.merge(games_df,aggregate_weekly_df,how = 'inner', left_on = ['home_name',
                'home_abbr'], right_on = ['team_name', 'team_abbr']).drop(columns = ['team_name', 'team_abbr'])
            home_df.columns = list(home_df.columns)[:7] + list_new_home
            
            aggregate_weekly_df = pd.merge(away_df,home_df,left_on = ['away_name', 'away_abbr', 'home_name',
                'home_abbr', 'winning_name', 'winning_abbr', 'week'], right_on = ['away_name', 'away_abbr',
                'home_name', 'home_abbr', 'winning_name', 'winning_abbr', 'week'])
            
            for n in range(len(list_column_change)):
                column_new = list_column_change[n] + '_' + 'dif'
                aggregate_weekly_df[column_new] = aggregate_weekly_df[list_new_away[n]] - aggregate_weekly_df[
                    list_new_home[n]]
                aggregate_weekly_df[column_new] = aggregate_weekly_df[column_new].fillna(0)
            
            aggregate_weekly_df = aggregate_weekly_df.drop(columns = list_new_away + list_new_home + [
                'fumbles_lost_dif'])
            
#             if (aggregate_weekly_df['winning_name'].isnull().values.any() and n_year == current_y and weeks_list[w
#                 ] == current_w):
#                 conditions = [aggregate_weekly_df['winning_name'] == aggregate_weekly_df['away_name'],
#                               aggregate_weekly_df['winning_name'] == aggregate_weekly_df['home_name']]
#                 choices = [1,0]
#                 aggregate_weekly_df['result'] = np.select(conditions,choices,default=np.nan)
                    
#             elif aggregate_weekly_df['winning_name'].isnull().values.any():
#                 aggregate_weekly_df = aggregate_weekly_df.dropna()
#                 aggregate_weekly_df['result'] = (aggregate_weekly_df['winning_name'] == aggregate_weekly_df['away_name'])
#                 aggregate_weekly_df['result'] = aggregate_weekly_df['result'].astype('int')
                
#             else:
#                 aggregate_weekly_df['result'] = (aggregate_weekly_df['winning_name'] == aggregate_weekly_df['away_name'])
#                 aggregate_weekly_df['result'] = aggregate_weekly_df['result'].astype('int')
#             away_df = pd.merge(games_df,over_df,how = 'inner', left_on = ['away_name',
#                 'away_abbr'], right_on = ['team_name', 'team_abbr']).drop(columns = ['team_name', 'team_abbr'])
#             home_df = pd.merge(games_df,aggregate_weekly_df,how = 'inner', left_on = ['home_name','home_abbr'], right_on = ['team_name', 'team_abbr']).drop(columns = ['team_name', 'team_abbr'])
            aggregate_weekly_df = aggregate_weekly_df.drop(columns = ['winning_name', 'winning_abbr'])
            aggregate_weekly_df['year'] = [n_year for n in range(len(aggregate_weekly_df))]
            
            new_columns_arrangement = list(aggregate_weekly_df.columns)[:5] + [
                list(aggregate_weekly_df.columns)[-1]] + list(aggregate_weekly_df.columns)[5:-1]
            aggregate_weekly_df = aggregate_weekly_df.reindex(columns=new_columns_arrangement)
            aggregate_weekly_df['year'] = aggregate_weekly_df.year.astype('int64')
            aggregate_games_df = pd.concat([aggregate_games_df, aggregate_weekly_df], axis = 0)
            aggregate_games_df = aggregate_games_df.reset_index().drop(columns = 'index')
            
    return aggregate_games_df

In [41]:
schedule_df = full_schedule
weeksgames_df = full_game_data
current_w = 6
current_y = 2022
weeksgames_df2 = weeksgames_df

In [29]:
aggregate_games_df.head(10)

NameError: name 'aggregate_games_df' is not defined

In [213]:
weeks_list

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [42]:
aggregate_games_df = aggregate_weekly_data(schedule_df, weeksgames_df, current_w, current_y, weeks_list, 0.1)
aggregate_games_df = aggregate_games_df.drop_duplicates().reset_index(drop = True)

In [89]:
full_game_data = full_game_data.drop_duplicates()

In [247]:
aggregate_games_df = pd.read_csv('agg_df_2021')

In [227]:
full_game_data=pd.read_csv('final_2010-2022_full_game_data_with_spread_no_wind_revised2.csv')

In [135]:
full_game_data=pd.read_csv('final_2010-2022_full_game_data_wk2_updatedrev2.csv').reset_index(drop = True)

In [154]:
full_game_data = full_game_data.drop(columns=['Unnamed: 0'])

In [155]:
full_game_data.to_csv('final_2010-2022_full_game_data_wk2_updatedrev3.csv')

In [165]:
aggregate_games_df=aggregate_games_df[aggregate_games_df.year!= 2017]

In [40]:
full_game_data=full_game_data[full_game_data.year!= 2010]

In [94]:
full_game_data=full_game_data.reset_index(drop = True)

In [25]:
schedule_df=schedule_df[schedule_df.week!= 18]

In [90]:
full_game_data.groupby(['year']).size()

year
2010    512
2011    512
2012    512
2013    512
2014    512
2015    512
2016    512
2017    512
2018    512
2019    512
2020    512
2021    512
2022    115
dtype: int64

In [205]:
weeks_list=list(range(1,19))

In [153]:
weeksgames_df.tail(50)

,Unnamed: 0,team_name,over_under,vegas_odds,team_abbr,score,game_won,game_lost,first_downs,fourth_down_attempts,fourth_down_conversions,fumbles,fumbles_lost,interceptions,net_pass_yards,pass_attempts,pass_completions,pass_touchdowns,pass_yards,penalties,points,rush_attempts,rush_touchdowns,rush_yards,third_down_attempts,third_down_conversions,time_of_possession,times_sacked,total_yards,turnovers,yards_from_penalties,yards_lost_from_sacks,week,spread,total,over or under,new spread,Actual Total,Actual Score Differential,Overs,Actual Covering,year,Actual Covering2,weather,wind
6158,6158,New England Patriots,46.0 (under),Miami Dolphins -3.0,nwe,7,0,1,17,1,0,2,2,1,193,30,21,1,213,3,7,22,0,78,9,4,1650,2,271,3,15,20,1,-3.0,46.0,(under),3.0,27,13,0,0,2022,NaN,NaN,NaN
6159,6159,Miami Dolphins,46.0 (under),Miami Dolphins -3.0,mia,20,1,0,18,1,1,2,0,0,242,33,23,1,270,4,20,23,0,65,14,6,1950,3,307,0,20,28,1,-3.0,46.0,(under),-3.0,27,13,0,0,2022,NaN,NaN,NaN
6160,6160,Baltimore Ravens,44.0 (under),New York Jets -6.5,rav,24,1,0,13,0,0,1,0,1,211,30,17,3,213,5,24,21,0,63,13,5,1650,2,274,1,29,2,1,-6.5,44.0,(under),6.5,33,-15,0,0,2022,NaN,NaN,NaN
6161,6161,New York Jets,44.0 (under),New York Jets -6.5,nyj,9,0,1,24,4,3,2,1,1,295,59,37,1,307,6,9,17,0,83,14,2,1950,3,378,2,81,12,1,-6.5,44.0,(under),-6.5,33,-15,0,0,2022,NaN,NaN,NaN
6162,6162,Jacksonville Jaguars,43.0 (over),Washington Commanders -3.0,jax,22,0,1,24,1,0,1,0,1,260,42,24,1,275,13,22,18,1,123,12,3,1587,2,383,1,90,15,1,-3.0,43.0,(over),3.0,50,6,1,0,2022,NaN,NaN,NaN
6163,6163,Washington Commanders,43.0 (over),Washington Commanders -3.0,was,28,1,0,26,0,0,1,1,2,305,41,27,4,313,6,28,28,0,85,10,7,2013,1,390,3,63,8,1,-3.0,43.0,(over),-3.0,50,6,1,0,2022,NaN,NaN,NaN
6164,6164,Kansas City Chiefs,54.0 (over),Arizona Cardinals -6.5,kan,44,1,0,33,1,1,5,1,0,360,39,30,5,360,3,44,27,1,128,8,5,2082,0,488,1,25,0,1,-6.5,54.0,(over),6.5,65,-23,1,0,2022,NaN,NaN,NaN
6165,6165,Arizona Cardinals,54.0 (over),Arizona Cardinals -6.5,crd,21,0,1,18,4,3,3,0,0,179,38,24,2,205,5,21,22,1,103,12,3,1518,3,282,0,49,26,1,-6.5,54.0,(over),-6.5,65,-23,1,0,2022,NaN,NaN,NaN
6166,6166,Green Bay Packers,46.0 (under),Minnesota Vikings -2.5,gnb,7,0,1,21,2,0,1,1,1,227,39,26,0,260,2,7,18,1,111,9,3,1717,4,338,2,10,33,1,-2.5,46.0,(under),2.5,30,16,0,0,2022,NaN,NaN,NaN
6167,6167,Minnesota Vikings,46.0 (under),Minnesota Vikings -2.5,min,23,1,0,19,1,1,0,0,0,269,32,23,2,277,3,23,28,0,126,13,4,1883,1,395,0,20,8,1,-2.5,46.0,(under),-2.5,30,16,0,0,2022,NaN,NaN,NaN


In [137]:
aggregate_games_df.groupby(['year']).size()

year
2011    256
2012    256
2013    256
2014    256
2015    256
2016    256
2017    256
2018    256
2019    256
2020    256
2021    256
2022     48
dtype: int64

In [128]:
full_game_data.groupby(['year']).size()

year
2010    512
2011    512
2012    512
2013    512
2014    512
2015    512
2016    512
2017    512
2018    512
2019    512
2020    512
2021    512
2022     82
dtype: int64

In [104]:
aggregate_games_df.groupby(['year']).size()

year
2011    256
2012    256
2013    256
2014    256
2015    256
2016    256
2018    256
2019    256
2020    256
2021    256
2022     32
dtype: int64

In [148]:
aggregate_games_df.groupby(['year']).size()

KeyError: 'year'

In [60]:
full_game_data['team_name']=full_game_data['team_name'].str.rstrip()

In [ ]:
aggregate_games_df[aggregate_games_df.year==2017]

In [ ]:
full_game_data[full_game_data.year==2017].dtypes

In [65]:
full_game_data[full_game_data.vegas_odds=='Tampa Bay Buccaneers -7.0']

,team_name,over_under,vegas_odds,team_abbr,score,game_won,game_lost,first_downs,fourth_down_attempts,fourth_down_conversions,fumbles,fumbles_lost,interceptions,net_pass_yards,pass_attempts,pass_completions,pass_touchdowns,pass_yards,penalties,points,rush_attempts,rush_touchdowns,rush_yards,third_down_attempts,third_down_conversions,time_of_possession,times_sacked,total_yards,turnovers,yards_from_penalties,yards_lost_from_sacks,week,spread,total,over or under,new spread,Actual Total,Actual Score Differential,Overs,Actual Covering,year,Actual Covering2
272,Carolina Panthers,37.0 (over),Tampa Bay Buccaneers -7.0,car,16,0,1,18,1,0,1,1,0,182,29,16,0,191,7,16,31,1,118,15,6,1720,2,300,1,71,9,10,-7.0,37.0,(over),7.0,47,15.0,1,0,2010,NaN
273,Tampa Bay Buccaneers,37.0 (over),Tampa Bay Buccaneers -7.0,tam,31,1,0,21,0,0,2,2,0,235,24,18,2,241,10,31,30,2,186,10,5,1880,1,421,2,80,6,10,-7.0,37.0,(over),-7.0,47,15.0,1,0,2010,NaN
1422,Philadelphia Eagles,48.0 (under),Tampa Bay Buccaneers -7.0,phi,23,1,0,23,2,2,1,1,0,338,51,32,2,381,2,23,16,1,29,18,7,1791,6,367,1,13,43,14,-7.0,48.0,(under),7.0,44,-2.0,0,0,2012,NaN
1423,Tampa Bay Buccaneers,48.0 (under),Tampa Bay Buccaneers -7.0,tam,21,0,1,18,0,0,1,0,0,178,34,14,2,189,8,21,32,1,136,16,6,1809,2,314,0,64,11,14,-7.0,48.0,(under),-7.0,44,-2.0,0,0,2012,NaN
3630,Chicago Bears,44.0 (under),Tampa Bay Buccaneers -7.0,chi,7,0,1,19,2,0,2,2,2,290,45,31,1,301,8,7,16,0,20,14,7,1609,1,310,4,59,11,2,-7.0,44.0,(under),7.0,36,22.0,0,0,2017,NaN
3631,Tampa Bay Buccaneers,44.0 (under),Tampa Bay Buccaneers -7.0,tam,29,1,0,22,0,0,1,1,0,194,31,18,1,204,4,29,34,1,117,15,8,1991,2,311,1,34,10,2,-7.0,44.0,(under),-7.0,36,22.0,0,0,2017,NaN
5518,Minnesota Vikings,52.0 (under),Tampa Bay Buccaneers -7.0,min,14,0,1,27,5,4,2,1,0,173,37,24,1,225,5,14,33,1,162,15,5,2343,6,335,1,71,52,14,-7.0,52.0,NaN,7.0,40,12.0,0,0,2020,0.0
5519,Tampa Bay Buccaneers,52.0 (under),Tampa Bay Buccaneers -7.0,tam,26,1,0,17,0,0,0,0,0,196,23,15,2,196,5,26,26,1,107,11,5,1257,0,303,0,34,0,14,-7.0,52.0,NaN,-7.0,40,12.0,0,0,2020,0.0
5756,Tampa Bay Buccaneers,49.5 (under),Tampa Bay Buccaneers -7.0,tam,19,1,0,22,0,0,0,0,0,261,43,22,0,269,7,19,30,1,120,19,9,2035,1,381,0,74,8,4,-7.0,49.5,NaN,-7.0,36,-2.0,0,0,2021,0.0
5757,New England Patriots,49.5 (under),Tampa Bay Buccaneers -7.0,nwe,17,0,1,20,0,0,1,1,1,295,42,33,2,320,8,17,8,0,1,9,2,1565,4,294,2,77,25,4,-7.0,49.5,NaN,7.0,36,-2.0,0,0,2021,0.0


In [ ]:
over_df = weeksgames_df[['over_under','Overs','Actual Total']]
aggregate_games_df = aggregate_games_df[aggregate_games_df.week != 18].reset_index()
aggregate_games_df = pd.DataFrame(np.repeat(aggregate_games_df.values, 2, axis=0)).reset_index()
#             list_column_change2 = list(over_df.columns)
#             list_new_away2 = column_name_manipulation_reverse(list_column_change2, 'away')
#             list_new_home2 = column_name_manipulation_reverse(list_column_change2, 'home')
#             aggregate_weekly_df = pd.merge(win_loss_df,aggregate_weekly_df,left_on = ['team_name','team_abbr'], right_on = ['team_name', 'team_abbr'])
#             away_df = pd.merge(over_df,aggregate_weekly_df,how = 'inner', left_on = ['week'
#                  'Actual Total'], right_on = ['week','Actual Total']).drop(columns = ['week','Actual Total'])
# over_df = over_df.iloc[32:, :].reset_index()

# aggregate_games_df.merge(over_df, how='left', left_on='index')
#aggregate_games_df = aggregate_games_df.drop(columns = 'index').drop_duplicates().reset_index()
aggregate_games_df

In [30]:
overs_df = over_df.iloc[::2].reset_index().drop(columns = ['index'])

In [426]:
weeksgames_df2['winner']= np.where(weeksgames_df2['game_won']==1, weeksgames_df2['team_abbr'],'opponent')

In [440]:
mlb_team_abbrev = {'crd': 'Arizona Cardinals',
'atl': 'Atlanta Falcons',
'rav': 'Baltimore Ravens',
'buf': 'Buffalo Bills',
'car': 'Carolina Panthers',
'chi': 'Chicago Bears',
'cin': 'Cincinnati Bengals',
'cle': 'Cleveland Browns',
'dal': 'Dallas Cowboys',
'den': 'Denver Broncos',
'det': 'Detroit Lions',
'gnb': 'Green Bay Packers',
'htx': 'Houston Texans',
'clt': 'Indianapolis Colts',
'jax': 'Jacksonville Jaguars',
'kan': 'Kansas City Chiefs',
'rai': 'Las Vegas Raiders',
'sdg': 'Los Angeles Chargers',
'ram': 'Los Angeles Rams',
'mia': 'Miami Dolphins',
'min':' Minnesota Vikings',
'nwe': 'New England Patriots',
'nor': 'New Orleans Saints',
'nyg': 'New York Giants',
'nyj': 'New York Jets',
'phi': 'Philadelphia Eagles',
'pit': 'Pittsburgh Steelers',
'sfo': 'San Francisco 49ers',
'sea': 'Seattle Seahawks',
'tam': 'Tampa Bay Buccaneers',
'oti': 'Tennessee Titans',
'was': 'Washington Football Team'}
weeksgames_df2['Team full name']= weeksgames_df2['team_abbr'].map(mlb_team_abbrev).fillna(weeksgames_df2['team_abbr'])

In [473]:
mlb_team_abbrev[2]

KeyError: 2

In [459]:
#weeksgames_df=weeksgames_df.reset_index()
lst = []
for i in range(len(weeksgames_df2)):
    #print(weeksgames_df['vegas_odds'][i][:-5])
    #print(weeksgames_df['vegas_odds'][i][:-5].replace(" ",""))
   # if weeksgames_df2['team_name'][i].replace(" ","") == weeksgames_df['vegas_odds'][i][:-5].replace(" ",""):
    lst.append(weeksgames_df2['vegas_odds'][i][:-5])
        #weeksgames_df['new spread']=((weeksgames_df['spread'][i])* 1)
    #else:
       # lst.append((weeksgames_df['spread'][i])* -1)
weeksgames_df2['favorite']=lst
    

In [ ]:
spreads_df

In [ ]:
for i in range(weeksgames_df2[])

In [26]:
over_df = weeksgames_df[['over_under','Overs','Actual Total']]

In [131]:
aggregate_games_df = aggregate_games_df[aggregate_games_df.year <= 2021]

In [113]:
aggregate_games_df = aggregate_games_df[aggregate_games_df.filter(regex='^(?!Unnamed)').columns]

In [119]:
len(weeksgames_df2[['over_under','Overs','Actual Total']])

1022

In [ ]:
weeksgames_df2

In [62]:
len(overs_df)

271

### To aggregate data for totals DF

In [ ]:
overs_df=weeksgames_df2[['over_under','Overs','Actual Total']]
overs_df = overs_df.iloc[::2].reset_index().drop(columns = ['index'])
overs_df=overs_df.iloc[256:, :].reset_index().drop(columns = ['index'])
new_df = pd.DataFrame()
new_df = pd.concat([new_df, aggregate_games_df], ignore_index=True)
new_df = new_df[new_df.year != 2020]
new_df = new_df[new_df.week != 18].reset_index()


#new_df = pd.DataFrame(np.repeat(new_df.values, 2, axis=0)).reset_index()
#             list_column_change2 = list(over_df.columns)
#             list_new_away2 = column_name_manipulation_reverse(list_column_change2, 'away')
#             list_new_home2 = column_name_manipulation_reverse(list_column_change2, 'home')
#             aggregate_weekly_df = pd.merge(win_loss_df,aggregate_weekly_df,left_on = ['team_name','team_abbr'], right_on = ['team_name', 'team_abbr'])
#             away_df = pd.merge(over_df,aggregate_weekly_df,how = 'inner', left_on = ['week'
#                  'Actual Total'], right_on = ['week','Actual Total']).drop(columns = ['week','Actual Total'])


new_df = pd.concat([new_df, overs_df], axis=1)
new_df['result']= new_df['Actual Total']
aggregate_games_df = new_df
aggregate_games_df = new_df.drop(columns = ['level_0'])
aggregate_games_df = aggregate_games_df.drop(columns = ['index'])
#new_df = new_df.drop(columns = 'index').drop_duplicates().reset_index()
aggregate_games_df

In [ ]:
aggregate_games_df.tail(50)

In [52]:
weeksgames_df2= weeksgames_df.drop(index=[796,797]).reset_index()

In [120]:
aggregate_games_df.to_csv('aggregate_totals_2021.csv')

In [150]:
aggregate_games_df = aggregate_games_df.loc[:,~aggregate_games_df.columns.duplicated()].copy()

In [135]:
aggregate_games_df.to_csv('aggregate_totals_2021_rev2.csv')

In [48]:
aggregate_games_df = pd.read_csv('agg_df_2021')

In [414]:
aggregate_games_df=aggregate_games_df[aggregate_games_df.week != 18].reset_index().drop(columns = ['index'])
aggregate_games_df = aggregate_games_df[aggregate_games_df.filter(regex='^(?!Unnamed)').columns]

In [412]:
aggregate_games_df = aggregate_games_df[aggregate_games_df.year <=2021]

In [411]:
aggregate_games_df = pd.read_csv('agg_df_2021_final_minus_wk18.csv')

In [416]:
aggregate_games_df= aggregate_games_df.drop(columns = ['level_0'])

In [417]:
aggregate_games_df.to_csv('agg_df_2021_final_minus_wk18_and_2022.csv')

### To aggregate data for spreads DF

In [462]:
lst=[]
for i in range(len(aggregate_games_df['winner'])):
    if aggregate_games_df['winner'][i] == 'opponent':
        if aggregate_games_df['team_abbr'][i] == aggregate_games_df['home_abbr'][i]:
            lst.append(aggregate_games_df['away_abbr'][i])
        else:
            lst.append(aggregate_games_df['home_abbr'][i])
    else:
        lst.append(aggregate_games_df['winner'][i])
aggregate_games_df['winner']=lst

In [463]:
mlb_team_abbrev = {'crd': 'Arizona Cardinals',
'atl': 'Atlanta Falcons',
'rav': 'Baltimore Ravens',
'buf': 'Buffalo Bills',
'car': 'Carolina Panthers',
'chi': 'Chicago Bears',
'cin': 'Cincinnati Bengals',
'cle': 'Cleveland Browns',
'dal': 'Dallas Cowboys',
'den': 'Denver Broncos',
'det': 'Detroit Lions',
'gnb': 'Green Bay Packers',
'htx': 'Houston Texans',
'clt': 'Indianapolis Colts',
'jax': 'Jacksonville Jaguars',
'kan': 'Kansas City Chiefs',
'rai': 'Las Vegas Raiders',
'sdg': 'Los Angeles Chargers',
'ram': 'Los Angeles Rams',
'mia': 'Miami Dolphins',
'min':' Minnesota Vikings',
'nwe': 'New England Patriots',
'nor': 'New Orleans Saints',
'nyg': 'New York Giants',
'nyj': 'New York Jets',
'phi': 'Philadelphia Eagles',
'pit': 'Pittsburgh Steelers',
'sfo': 'San Francisco 49ers',
'sea': 'Seattle Seahawks',
'tam': 'Tampa Bay Buccaneers',
'oti': 'Tennessee Titans',
'was': 'Washington Football Team'}
aggregate_games_df['winner full']= aggregate_games_df['winner'].map(mlb_team_abbrev).fillna(aggregate_games_df['winner'])

In [464]:
lst2=[]
for i in range(len(aggregate_games_df['winner'])):
    if aggregate_games_df['winner'][i] == aggregate_games_df['home_abbr'][i]:
        lst2.append(aggregate_games_df['away_abbr'][i])
    else:
        lst2.append(aggregate_games_df['home_abbr'][i])

aggregate_games_df['loser']=lst2

In [470]:
covers_name_list=[]
covers_diff=[]
aggregate_games_df['home or away']= np.where(((aggregate_games_df['team_abbr'])== ((aggregate_games_df['home_abbr']))),1, 0)
for i in range(len(aggregate_games_df)):
    if (aggregate_games_df['favorite'][i]== aggregate_games_df['winner full'][i]) and (abs(aggregate_games_df['Actual Score Differential'][i]) >= abs(aggregate_games_df['new spread'][i])):
        covers_name_list.append(aggregate_games_df['winner'][i])
        covers_diff.append(aggregate_games_df['Actual Score Differential'][i]-aggregate_games_df['new spread'][i])
#         if aggregate_games_df['new spread'][i]>0:
#             covers_name_list.append(aggregate_games_df['winner'][i])
#             covers_diff.append(aggregate_games_df['team_abbr'][i]-aggregate_games_df['new spread'][i])
#         elif aggregate_games_df['new spread'][i]<0 and abs(aggregate_games_df['Actual Score Differential'][i]) >= abs(aggregate_games_df['new spread'][i]):
#             covers_name_list.append(aggregate_games_df['winner'][i])
#             covers_diff.append(aggregate_games_df['Actual Score Differential'][i]+aggregate_games_df['new spread'][i])
#     else:
#         covers_name_list.append(aggregate_games_df['loser'][i])
#         covers_diff.append(aggregate_games_df['new spread'][i]-aggregate_games_df['Actual Score Differential'][i])
#     elif aggregate_games_df['team_abbr'][i]!= aggregate_games_df['winner'][i]:
#         if aggregate_games_df['new spread'][i]>0 and abs(aggregate_games_df['Actual Score Differential'][i]) <= abs(aggregate_games_df['new spread'][i]):
#             covers_name_list.append(aggregate_games_df['winner'][i])
#             covers_diff.append(aggregate_games_df['team_abbr'][i]-aggregate_games_df['new spread'][i])
#         elif aggregate_games_df['new spread'][i]<0 and abs(aggregate_games_df['Actual Score Differential'][i]) >= abs(aggregate_games_df['new spread'][i]):
#             covers_name_list.append(aggregate_games_df['winner'][i])
#             covers_diff.append(aggregate_games_df['Actual Score Differential'][i]+aggregate_games_df['new spread'][i])
    elif (aggregate_games_df['favorite'][i]!= aggregate_games_df['winner full'][i]):
        covers_name_list.append(aggregate_games_df['winner'][i])
        covers_diff.append(aggregate_games_df['Actual Score Differential'][i]-aggregate_games_df['new spread'][i])
    else:
        covers_name_list.append(aggregate_games_df['loser'][i])
        covers_diff.append(aggregate_games_df['new spread'][i]-aggregate_games_df['Actual Score Differential'][i])
aggregate_games_df['covers_name_list']  = covers_name_list
aggregate_games_df['covers_diff']  = covers_diff

In [ ]:
aggregate_games_df = pd.read_csv('agg_df_2021_final_minus_wk18_and_2022.csv')
spreads_df=weeksgames_df2[['spread','Overs','Actual Total','Actual Score Differential','winner','team_abbr','new spread','vegas_odds','Team full name','favorite']]
spreads_df = spreads_df.iloc[::2].reset_index().drop(columns = ['index'])
spreads_df=spreads_df.iloc[256:, :].reset_index().drop(columns = ['index'])
new_df = pd.DataFrame()
new_df = pd.concat([new_df, aggregate_games_df], ignore_index=True)
new_df = new_df[new_df.week != 18].reset_index()

#new_df = pd.DataFrame(np.repeat(new_df.values, 2, axis=0)).reset_index()
#             list_column_change2 = list(over_df.columns)
#             list_new_away2 = column_name_manipulation_reverse(list_column_change2, 'away')
#             list_new_home2 = column_name_manipulation_reverse(list_column_change2, 'home')
#             aggregate_weekly_df = pd.merge(win_loss_df,aggregate_weekly_df,left_on = ['team_name','team_abbr'], right_on = ['team_name', 'team_abbr'])
#             away_df = pd.merge(over_df,aggregate_weekly_df,how = 'inner', left_on = ['week'
#                  'Actual Total'], right_on = ['week','Actual Total']).drop(columns = ['week','Actual Total'])


new_df = pd.concat([new_df, spreads_df], axis=1)
new_df['result']= new_df['Actual Total']
aggregate_games_df = new_df
#aggregate_games_df = new_df.drop(columns = ['level_0'])
aggregate_games_df = aggregate_games_df.drop(columns = ['index'])
#new_df = new_df.drop(columns = 'index').drop_duplicates().reset_index()
aggregate_games_df= aggregate_games_df[aggregate_games_df.filter(regex='^(?!Unnamed)').columns]
aggregate_games_df.head(50)

In [ ]:
aggregate_games_df.head(50)

In [472]:
aggregate_games_df.to_csv('who_covers_2021.csv')

### Building aggregate to predict 2022 week 1 totals just based on 2021

In [ ]:
weeksgames_df[['over_under','Overs','Actual Total']]

In [227]:
len(weeksgames_df)

1024

In [265]:
aggregate_games_df = pd.read_csv('agg_df_2021')

In [266]:
aggregate_games_df = aggregate_games_df[aggregate_games_df.filter(regex='^(?!Unnamed)').columns]

In [267]:
aggregate_games_df= aggregate_games_df[aggregate_games_df.week != 18].reset_index()

In [232]:
aggregate_games_df.to_csv('agg_df_2021_final_minus_wk18.csv')

In [ ]:
aggregate_games_df.tail(500)

In [ ]:
over = weeksgames_df.iloc[::2].reset_index()
over.iloc[256:, :].reset_index()

In [132]:
weeksgames_df2=weeksgames_df

In [133]:
overs_df=weeksgames_df2[['over_under','Overs','Actual Total']]
overs_df = overs_df.iloc[::2].reset_index().drop(columns = ['index'])
overs_df=overs_df.iloc[256:, :].reset_index().drop(columns = ['index'])
new_df = pd.DataFrame()
new_df = pd.concat([new_df, aggregate_games_df], ignore_index=True)
new_df = new_df[new_df.year != 2020]
new_df = new_df[new_df.week != 18].reset_index()
#new_df = pd.DataFrame(np.repeat(new_df.values, 2, axis=0)).reset_index()
#             list_column_change2 = list(over_df.columns)
#             list_new_away2 = column_name_manipulation_reverse(list_column_change2, 'away')
#             list_new_home2 = column_name_manipulation_reverse(list_column_change2, 'home')
#             aggregate_weekly_df = pd.merge(win_loss_df,aggregate_weekly_df,left_on = ['team_name','team_abbr'], right_on = ['team_name', 'team_abbr'])
#             away_df = pd.merge(over_df,aggregate_weekly_df,how = 'inner', left_on = ['week'
#                  'Actual Total'], right_on = ['week','Actual Total']).drop(columns = ['week','Actual Total'])


new_df = pd.concat([new_df, overs_df], axis=1)
new_df['result']= new_df['Actual Total']
aggregate_games_df = new_df
aggregate_games_df = aggregate_games_df.drop(columns = ['index','level_0'])
aggregate_games_df = aggregate_games_df.drop(['year','week','spread_dif','total_dif','new spread_dif','Actual Total_dif','Overs_dif','Actual Score Differential_dif','Actual Covering_dif','Actual Covering2_dif','over_under','Actual Total',"Overs"], axis = 1)

In [134]:
aggregate_games_df.to_csv('agg_df_2021_to_pred_2022_wks_totals_lin_regrev1.csv')

In [ ]:
over_df = weeksgames_df[['over_under','Overs']]
over_df = over_df.iloc[32:, :].reset_index()
over_df.loc[:, "index"] = over_df["index"].apply(lambda x: x -32)
over_df['Overs']

In [564]:
new_df = pd.DataFrame()
new_df = pd.concat([new_df, aggregate_games_df], ignore_index=True)

In [580]:
current_y = 2022

In [ ]:
aggregate_games_df

In [ ]:
aggregate_games_df.drop(columns = 'index').drop_duplicates().reset_index()

In [581]:
current_w = 1

In [ ]:
full_game_data=full_game_data.drop(columns = 'index')

In [ ]:
full_game_data.head(20)

In [264]:
weeksgames_df['Overs'][0]

1

In [206]:
len(weeksgames_df.drop_duplicates())

1568

In [207]:
len(weeksgames_df[['Overs']].drop_duplicates)

TypeError: object of type 'method' has no len()

In [ ]:
current_y

### Building aggregate to predict 2022 totals  based on 2010-2021

In [196]:
weeksgames_df2=weeksgames_df[weeksgames_df.week != 18].reset_index()

In [214]:
weeksgames_df2=weeksgames_df[weeksgames_df.year != 2010].reset_index()

In [212]:
aggregate_games_df=aggregate_games_df[aggregate_games_df.year != 2010
                                     ].reset_index()

In [105]:
len(weeksgames_df)

5152

In [ ]:
weeksgames_df2[weeksgames_df2.year!= 2010].reset_index(drop=True).head(50)

In [40]:
weeksgames_df2=weeksgames_df

In [122]:
len(weeksgames_df)

6226

In [39]:
aggregate_games_df2=aggregate_games_df

In [123]:
weeksgames_df2=weeksgames_df2[weeksgames_df2.year!= 2010].reset_index(drop=True)
overs_df=weeksgames_df2[['over_under','Overs','Actual Total']]
overs_df = overs_df.iloc[::2].reset_index(drop=True)

In [124]:
len(overs_df)

2857

In [43]:
weeksgames_df2=weeksgames_df2[weeksgames_df2.year!= 2010].reset_index(drop=True)
overs_df=weeksgames_df2[['Actual Total']]
overs_df = overs_df.iloc[::2].reset_index(drop=True)
#overs_df=overs_df.iloc[256:, :].reset_index().drop(columns = ['index'])
new_df = pd.DataFrame()
new_df = pd.concat([new_df, aggregate_games_df], ignore_index=True)
#new_df = new_df[new_df.year != 2020]
#new_df = new_df[new_df.week != 18]
#new_df = pd.DataFrame(np.repeat(new_df.values, 2, axis=0)).reset_index()
#             list_column_change2 = list(over_df.columns)
#             list_new_away2 = column_name_manipulation_reverse(list_column_change2, 'away')
#             list_new_home2 = column_name_manipulation_reverse(list_column_change2, 'home')
#             aggregate_weekly_df = pd.merge(win_loss_df,aggregate_weekly_df,left_on = ['team_name','team_abbr'], right_on = ['team_name', 'team_abbr'])
#             away_df = pd.merge(over_df,aggregate_weekly_df,how = 'inner', left_on = ['week'
#                  'Actual Total'], right_on = ['week','Actual Total']).drop(columns = ['week','Actual Total'])


new_df = pd.concat([new_df, overs_df], axis=1)
new_df['result']= new_df['Actual Total']
aggregate_games_df = new_df
#aggregate_games_df = aggregate_games_df.drop(columns = ['index','level_0'])
#aggregate_games_df = aggregate_games_df.drop(['spread_dif','total_dif','new spread_dif','Actual Total_dif','Overs_dif','Actual Score Differential_dif','Actual Covering_dif','Actual Covering2_dif','over_under','Actual Total',"Overs"], axis = 1)

In [ ]:
aggregate_games_df = aggregate_games_df.drop(['spread_dif','total_dif','new spread_dif','Actual Total_dif','Overs_dif','Actual Score Differential_dif','Actual Covering_dif','Actual Covering2_dif','over_under','Actual Total',"Overs"], axis = 1)

In [55]:
aggregate_games_df = aggregate_games_df.drop(['total_dif','new spread_dif','Actual Total_dif','Actual Score Differential_dif','Actual Covering_dif','Actual Covering2_dif','Actual Total',"Overs_dif",'wind_dif'], axis = 1)

In [ ]:
aggregate_games_df.tail(150)

#### Transformation and feature engineering in the aggregate_weekly_data function
1. With this function, we create new columns and in the process of that, we also drop the unnecessary columns
2. We create 'win_perc' column from 'game_won' and 'game_lost' columns
3. We create 'fourth_down_perc' column from 'fourth_down_attempts' and 'fourth_down_conversions' columns
4. Similarly, we create 'third_down_perc' column from 'third_down_attempts' and 'third_down_conversions' columns
5. We create the target variable called 'result' column, which is coded to 1 if away team wins and coded to 0 if home team wins

In [ ]:
full_game_data.head(50)

In [101]:
a = [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]                      # [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
b = [x for i,x in enumerate(a) if x!=3] 

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

#### Calling the aggregate_weekly_data function
1. We rename the full_schedule df to schedule_df
2. We rename the full_game_data df to weeksgames_df
3. We'll have a quick look on the data information and descriptive stats

In [29]:
schedule_df = full_schedule
weeksgames_df = full_game_data


In [218]:
len(full_game_data)

1024

In [ ]:
weeksgames_df

In [220]:
len(weeksgames_df)

1024

In [492]:
current_w=

In [30]:
current_y = 2022

In [354]:
len(aggregate_games_df)

534

In [404]:
aggregate_games_df1 = aggregate_weekly_data(schedule_df, weeksgames_df, current_w, current_y, weeks_list, 0.1)

In [405]:
# Calling the aggregate games function
# Value of 'm' is taken to be '0.1' as from analyses (trial and error), we determined that it gives us highest accuracy
aggregate_games_df = aggregate_weekly_data(schedule_df, weeksgames_df, current_w, current_y, weeks_list, 0.1)
aggregate_games_df = aggregate_games_df.drop_duplicates().reset_index()

In [484]:
aggregate_games_df = aggregate_games_df.drop_duplicates().reset_index()

In [ ]:
weeksgames_df

In [ ]:
aggregate_games_df.groupby('week').count()

In [ ]:
aggregate_games_df1 = aggregate_weekly_data(schedule_df, weeksgames_df, current_w, current_y, weeks_list, 0.1)

In [ ]:
aggregate_games_df.columns

In [ ]:
aggregate_games_df.to_csv('aggregate_games_df.csv') 

In [ ]:
aggregate_games_df.loc[weeksgames_df.year ==2021]

In [ ]:
aggregate_games_df.info()

In [ ]:
aggregate_games_df_null = aggregate_games_df[aggregate_games_df['result'].isnull()]
len(aggregate_games_df_null)

In [ ]:
# Getting the descriptive stats for all feature columns
aggregate_games_df.iloc[:,6:-1].describe()

In [ ]:
full_game_data.tail(50)

1. We can see that all the features of the final dataframe doesn't have any null/missing values
2. It means, the function works correctly to aggregate the values of the features (dependent variables)
3. However, the column result, which is the target variable, has 13 NaN values (13 rows)
4. These rows are actually rows of current week games that have not been played yet so there is no result for these games
5. We will need these rows with NaN result as our prediction dataset
6. From here, the final dataframe is good to go to machine learning application

### New Linear Regression model

In [74]:
aggregate_games_df = aggregate_games_df.drop(['spread_dif','total_dif','new spread_dif','Actual Total_dif','Overs_dif','Actual Score Differential_dif','Actual Covering_dif','Actual Covering2_dif','Actual Total'], axis = 1)

In [ ]:
aggregate_games_df

In [76]:
aggregate_games_df = aggregate_games_df.drop(['over_under','Overs'], axis = 1)

In [53]:
aggregate_games_df =  aggregate_games_df[aggregate_games_df.pass_yards_dif != 0]

In [55]:
aggregate_games_df = read_csv('')

In [108]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
import numpy as np
import seaborn as sns
import math
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
pd.options.display.max_colwidth = 300
pd.options.display.max_columns = 100
## Include Libraries
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plot
from sklearn.preprocessing import StandardScaler
import numpy as np
import math

In [168]:
# We first take the dataframe where the results are not null (NaN): model_finding_df
# In other words, we only use the dataframe with completed games
model_finding_df = aggregate_games_df[aggregate_games_df.result.notna()]

# Then we select the dataset to be predicted by the best model (games that have not been played): prediction_df
prediction_df = aggregate_games_df[aggregate_games_df.result.isnull()]

In [3]:
aggregate_games_df

NameError: name 'aggregate_games_df' is not defined

In [67]:
len(aggregate_games_df)

272

In [82]:
def split_and_validate(X,y,test=.25,random=3):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test, random_state=random)
    
    # fit linear regression to training data
    lr_model = LinearRegression()
    scaler = MinMaxScaler()
    scaler.fit_transform(X_val)
    lr_model.fit(X_train, y_train)
    
    # score fit model on validation data
    val_score = lr_model.score(X_val, y_val)
    y_pred = lr_model.predict(X_val)
    #print(y_val)

    residuals = y_val - y_pred
    

    # report results
    print('\nValidation R^2 score was:', val_score)
    print('Feature coefficient results:')
#     for feature, coef in zip(X.columns, lr_model.coef_):
#         print(feature, ':', f'{coef:.2f}')
    
    return X_train, X_val, y_train, y_val, residuals, lr_model
X, y = model_finding_df.iloc[:, 6:-1].values, model_finding_df.iloc[:,-1].values

In [76]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.25, random_state = 0) ## Generated new Scores
scaler = MinMaxScaler()
#scaler = StandardScaler()
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_stdz = sc.fit_transform(X_train)
X_test_stdz = sc.transform(X_val)
model

y_pred = model.predict(X_test_stdz)
df_new = pd.DataFrame()
len(y_pred)
print(y_pred)

[[-1.80105394  1.13061534  0.65774968 ... -0.06314587 -0.59734259
  -0.22756515]
 [ 0.88499258 -0.54446776 -0.09379304 ... -1.3218308   0.98378331
   1.11279718]
 [ 1.07272747 -0.66705254  1.2740545  ...  0.12568448 -0.5982648
   0.33379801]
 ...
 [-1.40344273 -0.93165065  0.0142024  ...  1.01277436 -0.69728664
  -1.48272145]
 [-0.28030387 -0.39100548  1.05365853 ... -0.30737042  0.04798232
  -0.28645086]
 [-0.17070211  0.62170675  0.72498293 ... -0.05797791  2.13361657
   0.01315372]]


In [289]:
model_finding_df = aggregate_games_df[aggregate_games_df.result.notna()]

In [77]:
aggregate_games_df.tail(50)

,away_name,away_abbr,home_name,home_abbr,week,year,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,elo_dif,qb_dif,result
2853,Atlanta Falcons,atl,Seattle Seahawks,sea,3,2022,-0.435714,4.550000,0.478571,0.035714,16.242857,0.300000,-4.271429,-0.028571,10.885714,-0.950000,12.785714,14.378571,0.864286,77.371429,427.121429,-0.492857,93.614286,0.521429,-44.985714,-5.357143,0.456494,-0.081259,-84.395696,-12.946338,50.0
2854,Green Bay Packers,gnb,Tampa Bay Buccaneers,tam,3,2022,-0.435714,7.028571,1.400000,-0.050000,20.314286,0.564286,3.542857,0.007143,37.085714,-3.714286,-2.314286,-2.842857,0.864286,41.828571,105.157143,1.835714,62.142857,0.400000,-41.521429,16.771429,0.004977,0.104787,-85.998594,7.012767,26.0
2855,San Francisco 49ers,sfo,Denver Broncos,den,3,2022,0.014286,1.092857,0.957143,0.021429,-82.514286,-9.600000,-6.807143,-0.414286,-91.357143,-5.242857,2.828571,14.185714,1.392857,52.350000,154.900000,-0.935714,-30.164286,-0.421429,-33.900000,-8.842857,-0.090458,0.027641,112.676774,-17.408179,21.0
2856,Dallas Cowboys,dal,New York Giants,nyg,3,2022,-0.407143,-2.242857,-0.428571,-0.064286,47.200000,8.392857,0.914286,-0.757143,39.435714,-0.192857,-6.071429,-8.671429,0.035714,-71.071429,-298.635714,-1.350000,-23.871429,-0.057143,7.835714,-7.764286,-0.436335,-0.036696,133.866907,-89.490190,39.0
2857,Miami Dolphins,mia,Cincinnati Bengals,cin,4,2022,0.592308,-3.523077,-0.838462,-0.584615,39.230769,-6.430769,-2.400000,0.530769,27.730769,0.207692,4.830769,-8.769231,0.546154,-31.653846,-428.715385,-2.792308,7.576923,-1.153846,1.130769,-11.500000,0.542564,0.033770,44.225945,21.675978,42.0
2858,Minnesota Vikings,min,New Orleans Saints,nor,4,2022,0.300000,4.269231,-1.469231,-0.630769,-15.446154,1.553846,0.738462,0.330769,-26.907692,-3.800000,2.453846,0.700000,-0.007692,-7.146154,-100.884615,-2.153846,-22.592308,-1.530769,-45.415385,-11.461538,0.593750,-0.035109,25.514486,91.600340,53.0
2859,Cleveland Browns,cle,Atlanta Falcons,atl,4,2022,0.292308,2.169231,-1.838462,-0.600000,-11.923077,3.861538,3.069231,0.315385,-16.284615,1.400000,1.476923,5.607692,0.315385,35.223077,284.038462,-0.530769,23.300000,-1.823077,12.261538,-4.361538,0.145238,0.067279,72.821090,10.219648,43.0
2860,Tennessee Titans,oti,Indianapolis Colts,clt,4,2022,-0.150000,-3.392308,-0.876923,-0.253846,-17.638462,-9.753846,-6.984615,-0.069231,-34.723077,2.246154,2.807692,0.700000,0.584615,-11.876923,-107.823077,-2.261538,-29.515385,0.030769,22.823077,-17.084615,0.290643,0.012303,20.592806,41.429765,41.0
2861,Washington Commanders,was,Dallas Cowboys,dal,4,2022,-0.330769,3.823077,0.869231,0.623077,36.807692,7.284615,5.907692,1.400000,57.461538,-3.646154,4.000000,-1.084615,0.007692,-28.092308,229.653846,3.092308,8.715385,0.915385,-20.707692,20.653846,-0.304873,0.170645,-134.888185,70.972872,35.0
2862,Seattle Seahawks,sea,Detroit Lions,det,4,2022,0.021795,-2.184615,0.900000,0.253846,-16.776923,-3.115385,3.576923,-0.861538,-9.976923,1.723077,-13.930769,-9.384615,-1.461538,-85.492308,-209.407692,0.661538,-102.269231,0.823077,29.792308,6.800000,-0.064839,0.102009,74.521885,-13.348075,93.0


### PREDICT TOTALS

In [70]:
model_finding_df = aggregate_games_df[aggregate_games_df.result.notna()]

In [ ]:
model_finding_df.iloc[:, 6:-1]

In [71]:
# train = model_finding_df[['result']]
# model_finding_df = model_finding_df.drop(columns = ['away_name', 'away_abbr', 'home_name', 'home_abbr', 'week','result','over_under','Overs','Actual Total','year'])

prediction_df=aggregate_games_df[aggregate_games_df.result.isnull()]

In [72]:
# STEP 1
X = model_finding_df.iloc[:, 6:-1].values
y = model_finding_df.iloc[:, -1].values


# STEP 2
from sklearn.model_selection import train_test_split
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

# STEP 3
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_stdz = sc.fit_transform(X_train_raw)
X_test_stdz = sc.transform(X_test_raw)

from sklearn.decomposition import PCA
dimension = 7 
pca = PCA(n_components = dimension)
X_train = pca.fit_transform(X_train_stdz)
X_test = pca.transform(X_test_stdz)

# STEP 4
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
model_finding_df.tail(500)

In [73]:
def display_prediction_for_current_week_games(y_pred_prob, prediction_df):
    for t in range(len(prediction_df)):
        win_prob = np.round(y_pred_prob[t], 3)
        away_team = prediction_df.reset_index().drop(columns = 'index').loc[t,'away_name']
        home_team = prediction_df.reset_index().drop(columns = 'index').loc[t,'home_name']
        print('The {} are predicted to have a total of {} against the {}.'.format(away_team, win_prob, home_team))

In [74]:
X_pred_raw = prediction_df.iloc[:, 6:-1].values
X_pred_stdz = sc.transform(X_pred_raw)
X_pred = pca.transform(X_pred_stdz)


In [75]:
y_pred_prob=model.predict(X_pred)

In [197]:
aggregate_games_df['predicted totals']=y_pred_prob

In [151]:
aggregate_games_df.to_csv('agg_df_after_totals_pred_no_elov2.csv')

In [ ]:
prediction_df.iloc[:, 6:-4]

In [76]:
display_prediction_for_current_week_games(y_pred_prob, prediction_df)

The Las Vegas Raiders are predicted to have a total of 45.425 against the Kansas City Chiefs.
The Washington Commanders are predicted to have a total of 46.949 against the Chicago Bears.
The San Francisco 49ers are predicted to have a total of 46.458 against the Atlanta Falcons.
The New England Patriots are predicted to have a total of 47.115 against the Cleveland Browns.
The Jacksonville Jaguars are predicted to have a total of 44.501 against the Indianapolis Colts.
The New York Jets are predicted to have a total of 44.808 against the Green Bay Packers.
The Minnesota Vikings are predicted to have a total of 45.674 against the Miami Dolphins.
The Cincinnati Bengals are predicted to have a total of 47.66 against the New Orleans Saints.
The Baltimore Ravens are predicted to have a total of 46.477 against the New York Giants.
The Tampa Bay Buccaneers are predicted to have a total of 45.728 against the Pittsburgh Steelers.
The Carolina Panthers are predicted to have a total of 45.248 again

In [63]:
display_prediction_for_current_week_games(y_pred_prob, prediction_df)

The Las Vegas Raiders are predicted to have a total of 45.67 against the Kansas City Chiefs.
The Washington Commanders are predicted to have a total of 45.654 against the Chicago Bears.
The San Francisco 49ers are predicted to have a total of 45.562 against the Atlanta Falcons.
The New England Patriots are predicted to have a total of 46.493 against the Cleveland Browns.
The Jacksonville Jaguars are predicted to have a total of 44.15 against the Indianapolis Colts.
The New York Jets are predicted to have a total of 45.061 against the Green Bay Packers.
The Minnesota Vikings are predicted to have a total of 45.774 against the Miami Dolphins.
The Cincinnati Bengals are predicted to have a total of 46.119 against the New Orleans Saints.
The Baltimore Ravens are predicted to have a total of 45.718 against the New York Giants.
The Tampa Bay Buccaneers are predicted to have a total of 45.864 against the Pittsburgh Steelers.
The Carolina Panthers are predicted to have a total of 45.998 agains

In [55]:
display_prediction_for_current_week_games(y_pred_prob, prediction_df)

The Los Angeles Rams are predicted to have a total of 45.945 against the San Francisco 49ers.
The Indianapolis Colts are predicted to have a total of 47.352 against the Denver Broncos.
The New York Giants are predicted to have a total of 44.707 against the Green Bay Packers.
The Pittsburgh Steelers are predicted to have a total of 45.131 against the Buffalo Bills.
The Los Angeles Chargers are predicted to have a total of 46.365 against the Cleveland Browns.
The Houston Texans are predicted to have a total of 45.989 against the Jacksonville Jaguars.
The Chicago Bears are predicted to have a total of 47.121 against the Minnesota Vikings.
The Seattle Seahawks are predicted to have a total of 45.593 against the New Orleans Saints.
The Detroit Lions are predicted to have a total of 44.534 against the New England Patriots.
The Miami Dolphins are predicted to have a total of 45.402 against the New York Jets.
The Atlanta Falcons are predicted to have a total of 45.955 against the Tampa Bay Buc

In [158]:
display_prediction_for_current_week_games(y_pred_prob, prediction_df)

The Miami Dolphins are predicted to have a total of 42.205 against the Cincinnati Bengals.
The Minnesota Vikings are predicted to have a total of 42.874 against the New Orleans Saints.
The Cleveland Browns are predicted to have a total of 45.207 against the Atlanta Falcons.
The Tennessee Titans are predicted to have a total of 43.379 against the Indianapolis Colts.
The Washington Commanders are predicted to have a total of 48.118 against the Dallas Cowboys.
The Seattle Seahawks are predicted to have a total of 46.977 against the Detroit Lions.
The Los Angeles Chargers are predicted to have a total of 45.483 against the Houston Texans.
The Chicago Bears are predicted to have a total of 46.318 against the New York Giants.
The Jacksonville Jaguars are predicted to have a total of 46.244 against the Philadelphia Eagles.
The New York Jets are predicted to have a total of 46.375 against the Pittsburgh Steelers.
The Buffalo Bills are predicted to have a total of 46.876 against the Baltimore R

In [111]:
display_prediction_for_current_week_games(y_pred_prob, prediction_df)

The Dallas Cowboys are predicted to have a total of 44.966 against the New York Giants.
The Miami Dolphins are predicted to have a total of 43.917 against the Cincinnati Bengals.
The Minnesota Vikings are predicted to have a total of 43.97 against the New Orleans Saints.
The Cleveland Browns are predicted to have a total of 45.233 against the Atlanta Falcons.
The Tennessee Titans are predicted to have a total of 44.079 against the Indianapolis Colts.
The Washington Commanders are predicted to have a total of 47.146 against the Dallas Cowboys.
The Seattle Seahawks are predicted to have a total of 47.01 against the Detroit Lions.
The Los Angeles Chargers are predicted to have a total of 45.436 against the Houston Texans.
The Chicago Bears are predicted to have a total of 44.237 against the New York Giants.
The Jacksonville Jaguars are predicted to have a total of 46.068 against the Philadelphia Eagles.
The New York Jets are predicted to have a total of 47.349 against the Pittsburgh Steel

In [64]:
display_prediction_for_current_week_games(y_pred_prob, prediction_df)

The Dallas Cowboys are predicted to have a total of 46.012 against the New York Giants.
The Miami Dolphins are predicted to have a total of 44.279 against the Cincinnati Bengals.
The Minnesota Vikings are predicted to have a total of 44.254 against the New Orleans Saints.
The Cleveland Browns are predicted to have a total of 44.55 against the Atlanta Falcons.
The Tennessee Titans are predicted to have a total of 44.681 against the Indianapolis Colts.
The Washington Commanders are predicted to have a total of 46.935 against the Dallas Cowboys.
The Seattle Seahawks are predicted to have a total of 47.768 against the Detroit Lions.
The Los Angeles Chargers are predicted to have a total of 45.54 against the Houston Texans.
The Chicago Bears are predicted to have a total of 46.43 against the New York Giants.
The Jacksonville Jaguars are predicted to have a total of 46.73 against the Philadelphia Eagles.
The New York Jets are predicted to have a total of 46.351 against the Pittsburgh Steeler

In [50]:
display_prediction_for_current_week_games(y_pred_prob, prediction_df)

The Dallas Cowboys are predicted to have a total of 44.966 against the New York Giants.
The Miami Dolphins are predicted to have a total of 43.917 against the Cincinnati Bengals.
The Minnesota Vikings are predicted to have a total of 43.97 against the New Orleans Saints.
The Cleveland Browns are predicted to have a total of 45.233 against the Atlanta Falcons.
The Tennessee Titans are predicted to have a total of 44.079 against the Indianapolis Colts.
The Washington Commanders are predicted to have a total of 47.146 against the Dallas Cowboys.
The Seattle Seahawks are predicted to have a total of 47.01 against the Detroit Lions.
The Los Angeles Chargers are predicted to have a total of 45.436 against the Houston Texans.
The Chicago Bears are predicted to have a total of 44.237 against the New York Giants.
The Jacksonville Jaguars are predicted to have a total of 46.068 against the Philadelphia Eagles.
The New York Jets are predicted to have a total of 47.349 against the Pittsburgh Steel

In [224]:
display_prediction_for_current_week_games(y_pred_prob, prediction_df)

The Pittsburgh Steelers are predicted to have a total of 46.955 against the Cleveland Browns.
The New Orleans Saints are predicted to have a total of 48.111 against the Carolina Panthers.
The Houston Texans are predicted to have a total of 45.607 against the Chicago Bears.
The Kansas City Chiefs are predicted to have a total of 42.928 against the Indianapolis Colts.
The Buffalo Bills are predicted to have a total of 45.754 against the Miami Dolphins.
The Detroit Lions are predicted to have a total of 45.522 against the Minnesota Vikings.
The Baltimore Ravens are predicted to have a total of 43.87 against the New England Patriots.
The Cincinnati Bengals are predicted to have a total of 48.145 against the New York Jets.
The Las Vegas Raiders are predicted to have a total of 46.795 against the Tennessee Titans.
The Philadelphia Eagles are predicted to have a total of 43.025 against the Washington Commanders.
The Jacksonville Jaguars are predicted to have a total of 45.397 against the Los 

In [209]:
display_prediction_for_current_week_games(y_pred_prob, prediction_df)

The Pittsburgh Steelers are predicted to have a total of 47.915 against the Cleveland Browns.
The New Orleans Saints are predicted to have a total of 47.526 against the Carolina Panthers.
The Houston Texans are predicted to have a total of 47.597 against the Chicago Bears.
The Kansas City Chiefs are predicted to have a total of 44.975 against the Indianapolis Colts.
The Buffalo Bills are predicted to have a total of 45.037 against the Miami Dolphins.
The Detroit Lions are predicted to have a total of 45.492 against the Minnesota Vikings.
The Baltimore Ravens are predicted to have a total of 45.066 against the New England Patriots.
The Cincinnati Bengals are predicted to have a total of 46.819 against the New York Jets.
The Las Vegas Raiders are predicted to have a total of 47.187 against the Tennessee Titans.
The Philadelphia Eagles are predicted to have a total of 42.831 against the Washington Commanders.
The Jacksonville Jaguars are predicted to have a total of 45.196 against the Los

In [169]:
display_prediction_for_current_week_games(y_pred_prob, prediction_df)

The Pittsburgh Steelers are predicted to have a total of 47.915 against the Cleveland Browns.
The New Orleans Saints are predicted to have a total of 47.526 against the Carolina Panthers.
The Houston Texans are predicted to have a total of 47.597 against the Chicago Bears.
The Kansas City Chiefs are predicted to have a total of 44.975 against the Indianapolis Colts.
The Buffalo Bills are predicted to have a total of 45.037 against the Miami Dolphins.
The Detroit Lions are predicted to have a total of 45.492 against the Minnesota Vikings.
The Baltimore Ravens are predicted to have a total of 45.066 against the New England Patriots.
The Cincinnati Bengals are predicted to have a total of 46.819 against the New York Jets.
The Las Vegas Raiders are predicted to have a total of 47.187 against the Tennessee Titans.
The Philadelphia Eagles are predicted to have a total of 42.831 against the Washington Commanders.
The Jacksonville Jaguars are predicted to have a total of 45.196 against the Los

### WITH ELO

In [64]:
def get_elo():
    elo_df = pd.read_csv('nfl_elo3.csv')
    elo_df = elo_df.drop(columns = ['neutral' ,'playoff', 'elo_prob1', 'elo_prob2', 'elo1_post', 'elo2_post',
           'qbelo1_pre', 'qbelo2_pre', 'qb1', 'qb2', 'qb1_adj', 'qb2_adj', 'qbelo_prob1', 'qbelo_prob2',
           'qb1_game_value', 'qb2_game_value', 'qb1_value_post', 'qb2_value_post',
           'qbelo1_post', 'qbelo2_post', 'score1', 'score2'])
    elo_df.date = pd.to_datetime(elo_df.date)
    elo_df = elo_df[elo_df.date >= '09-8-2011']
    
    elo_df['team1'] = elo_df['team1'].replace(['KC', 'JAX', 'CAR', 'BAL', 'BUF', 'MIN', 'DET', 'ATL', 'NE', 'WSH',
           'CIN', 'NO', 'SF', 'LAR', 'NYG', 'DEN', 'CLE', 'IND', 'TEN', 'NYJ',
           'TB', 'MIA', 'PIT', 'PHI', 'GB', 'CHI', 'DAL', 'ARI', 'LAC', 'HOU',
           'SEA', 'OAK'],
            ['kan','jax','car', 'rav', 'buf', 'min', 'det', 'atl', 'nwe', 'was', 
            'cin', 'nor', 'sfo', 'ram', 'nyg', 'den', 'cle', 'clt', 'oti', 'nyj', 
             'tam','mia', 'pit', 'phi', 'gnb', 'chi', 'dal', 'crd', 'sdg', 'htx', 'sea', 'rai' ])
    elo_df['team2'] = elo_df['team2'].replace(['KC', 'JAX', 'CAR', 'BAL', 'BUF', 'MIN', 'DET', 'ATL', 'NE', 'WSH',
           'CIN', 'NO', 'SF', 'LAR', 'NYG', 'DEN', 'CLE', 'IND', 'TEN', 'NYJ',
           'TB', 'MIA', 'PIT', 'PHI', 'GB', 'CHI', 'DAL', 'ARI', 'LAC', 'HOU',
           'SEA', 'OAK'],
            ['kan','jax','car', 'rav', 'buf', 'min', 'det', 'atl', 'nwe', 'was', 
            'cin', 'nor', 'sfo', 'ram', 'nyg', 'den', 'cle', 'clt', 'oti', 'nyj', 
             'tam','mia', 'pit', 'phi', 'gnb', 'chi', 'dal', 'crd', 'sdg', 'htx', 'sea', 'rai' ])
    return elo_df

In [65]:
elo_df = get_elo()

In [66]:
def merge_rankings(agg_games_df,elo_df):
    agg_games_df = pd.merge(agg_games_df, elo_df, how = 'inner', left_on = ['home_abbr', 'away_abbr','year'], right_on = ['team1', 'team2','season']).drop(columns = ['date','team1', 'team2','season'])
    agg_games_df['elo_dif'] = agg_games_df['elo2_pre'] - agg_games_df['elo1_pre']
    agg_games_df['qb_dif'] = agg_games_df['qb2_value_pre'] - agg_games_df['qb1_value_pre']
    agg_games_df = agg_games_df.drop(columns = ['elo1_pre', 'elo2_pre', 'qb1_value_pre', 'qb2_value_pre','quality','importance','total_rating'])
    return agg_games_df

In [215]:
aggregate_games_df.to_csv('agg_df_9_21_22_elo.csv')

In [216]:
aggregate_games_df=pd.read_csv('agg_df_9_21_22_elo.csv')

In [67]:
aggregate_games_df = merge_rankings(aggregate_games_df, elo_df)

In [69]:
aggregate_games_df.tail(100)

,away_name,away_abbr,home_name,home_abbr,week,year,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,elo_dif,qb_dif,result
2803,Denver Broncos,den,Los Angeles Chargers,sdg,17,2021,-0.160000,-3.980000,0.160000,-0.300000,-57.140000,-4.460000,-3.600000,-0.940000,-54.020000,-1.740000,-7.300000,1.680000,-0.280000,5.060000,88.120000,0.500000,-52.080000,-0.240000,-15.100000,3.120000,0.043269,-0.074544,-52.356142,-121.397425,47.0
2804,Houston Texans,htx,San Francisco 49ers,sfo,17,2021,-0.240000,-4.260000,-0.400000,0.280000,-35.260000,1.680000,0.820000,-0.100000,-29.720000,1.360000,-6.240000,-4.240000,-0.780000,-34.040000,-160.260000,0.700000,-69.300000,0.000000,-1.280000,5.540000,-0.121053,0.019698,-163.770593,3.034784,30.0
2805,Los Angeles Rams,ram,Baltimore Ravens,rav,17,2021,0.180000,-2.640000,-0.320000,-0.060000,38.640000,1.480000,1.040000,0.600000,36.020000,-1.780000,1.660000,-6.960000,-0.680000,-60.540000,-248.800000,-1.240000,-21.900000,-0.080000,-15.260000,-2.620000,-0.099198,0.019805,95.092617,132.859034,39.0
2806,Detroit Lions,det,Seattle Seahawks,sea,17,2021,-0.271429,2.160000,0.160000,0.400000,16.500000,4.540000,3.780000,-0.200000,9.280000,0.960000,-1.920000,1.720000,-0.200000,5.900000,221.620000,-0.640000,22.400000,0.740000,3.820000,-7.220000,0.180000,0.017804,-169.872424,-137.477619,80.0
2807,Minnesota Vikings,min,Green Bay Packers,gnb,17,2021,-0.300000,0.560000,0.340000,0.120000,19.400000,3.220000,1.680000,-0.340000,20.040000,2.160000,0.260000,2.940000,0.100000,16.300000,-42.340000,-0.040000,35.700000,0.080000,18.500000,0.640000,0.081579,-0.073723,-165.062847,-202.331346,47.0
2808,Cleveland Browns,cle,Pittsburgh Steelers,pit,17,2021,0.070000,1.440000,0.060000,0.020000,-39.740000,-8.660000,-5.940000,-0.340000,-35.620000,0.380000,0.980000,5.540000,0.800000,62.240000,122.000000,0.540000,22.500000,-0.100000,10.900000,4.120000,-0.104000,-0.004725,19.306483,-53.092771,40.0
2809,Buffalo Bills,buf,Los Angeles Rams,ram,1,2022,-0.125000,2.312500,0.562500,0.000000,-25.687500,2.125000,0.312500,-0.250000,-28.687500,2.250000,1.250000,2.187500,0.562500,26.250000,147.875000,0.062500,0.562500,0.062500,19.875000,-3.000000,0.000000,0.040154,-0.514674,59.322645,41.0
2810,New Orleans Saints,nor,Atlanta Falcons,atl,1,2022,0.062500,-0.500000,-0.562500,-0.062500,-31.625000,-3.750000,-5.000000,0.437500,-32.000000,0.437500,2.562500,5.187500,0.125000,25.437500,86.812500,-0.062500,-6.187500,-0.312500,4.000000,-0.375000,0.000000,-0.049898,107.143329,39.536756,53.0
2811,Cleveland Browns,cle,Carolina Panthers,car,1,2022,0.125000,0.875000,-0.062500,-0.437500,7.312500,-3.750000,-1.000000,0.437500,5.562500,0.125000,2.562500,0.937500,0.125000,33.375000,10.000000,-0.125000,40.687500,-0.437500,8.437500,-1.750000,0.000000,0.023009,90.637551,27.256252,50.0
2812,San Francisco 49ers,sfo,Chicago Bears,chi,1,2022,0.187500,1.812500,-0.187500,-0.375000,60.375000,-0.812500,1.250000,0.562500,49.187500,-0.500000,6.625000,1.125000,0.437500,6.437500,52.312500,-1.312500,66.812500,-0.312500,6.312500,-11.187500,0.000000,0.035525,131.643718,8.587091,29.0


In [68]:
df1 = aggregate_games_df.pop('result')
aggregate_games_df['result']=df1

### Section for predicting 2021 to backtest

In [150]:
list1=[]
for i in range(2,len(weeks_list)):

    list1.append(run_for_each_individual_wk(i,2021))
wk1=run_for_each_individual_wk(1,2021)
wk10 =(list1[7])
del wk10[-8]

AttributeError: 'DataFrame' object has no attribute 'year'

In [128]:
spread_df_covers2 = pd.read_csv('spreads_df_covers2.csv')

In [136]:
full_game_data=full_game_data.drop(columns=['Unnamed: 0'])

In [135]:
list1 = wk1+ list1[0] + list1[1]+list1[2]+list1[3]+list1[4]+list1[5]+list1[6]+wk10+list1[8]+list1[9]+list1[10]+list1[11]+list1[12]+list1[13]+list1[14]+list1[15]

NameError: name 'wk1' is not defined

In [ ]:
schedule_df = full_schedule
weeksgames_df = full_game_data
weeks_list = list(range(1,18))

aggregate_games_df = aggregate_weekly_data(schedule_df, weeksgames_df, current_w, current_y, weeks_list, 0.1)

In [ ]:
aggregate_games_df

In [152]:
def run_for_each_individual_wk(current_w,current_y):
    schedule_df = full_schedule
    weeksgames_df = full_game_data
    weeks_list = list(range(1,18))

    aggregate_games_df = aggregate_weekly_data(schedule_df, weeksgames_df, current_w, current_y, weeks_list, 0.1)
    #aggregate_games_df = aggregate_games_df[aggregate_games_df.week !=18]
    #aggregate_games_df = aggregate_games_df[aggregate_games_df.year >=current_y-1]
    aggregate_games_df = aggregate_games_df[aggregate_games_df.filter(regex='^(?!Unnamed)').columns]
    aggregate_games_df=aggregate_games_df[aggregate_games_df.year!= 2010]
    aggregate_games_df=aggregate_games_df[aggregate_games_df.year!= 2017]
    weeksgames_df2=weeksgames_df
    overs_df=weeksgames_df2[['over_under','Overs','Actual Total']]
    overs_df = overs_df.iloc[::2].reset_index(drop = True)
    overs_df=overs_df.iloc[256:, :].reset_index(drop = True)
    new_df = pd.DataFrame()
    new_df = pd.concat([new_df, aggregate_games_df], ignore_index=True)
    new_df = new_df[new_df.year != 2020]
    new_df = new_df[new_df.week != 18].reset_index()
    #new_df = pd.DataFrame(np.repeat(new_df.values, 2, axis=0)).reset_index()
    #             list_column_change2 = list(over_df.columns)
    #             list_new_away2 = column_name_manipulation_reverse(list_column_change2, 'away')
    #             list_new_home2 = column_name_manipulation_reverse(list_column_change2, 'home')
    #             aggregate_weekly_df = pd.merge(win_loss_df,aggregate_weekly_df,left_on = ['team_name','team_abbr'], right_on = ['team_name', 'team_abbr'])
    #             away_df = pd.merge(over_df,aggregate_weekly_df,how = 'inner', left_on = ['week'
    #                  'Actual Total'], right_on = ['week','Actual Total']).drop(columns = ['week','Actual Total'])


    new_df = pd.concat([new_df, overs_df], axis=1)
    new_df['result']= new_df['Actual Total']
    aggregate_games_df = new_df
    #aggregate_games_df = aggregate_games_df.drop(columns = ['index','level_0'])
    aggregate_games_df['result'] = np.where((aggregate_games_df.year ==current_y)& (aggregate_games_df.week ==current_w), np.nan, aggregate_games_df['result'])
    aggregate_games_df = aggregate_games_df.drop(['spread_dif','total_dif','new spread_dif','Actual Total_dif','Overs_dif','Actual Score Differential_dif','Actual Covering_dif','Actual Covering2_dif','over_under','Actual Total',"Overs"], axis = 1)
   
    model_finding_df = aggregate_games_df[aggregate_games_df.result.notna()]
    prediction_df=aggregate_games_df[aggregate_games_df.result.isnull()]
    
    X = model_finding_df.iloc[:, 6:-1].values
    y = model_finding_df.iloc[:, -1].values

    # STEP 2
    from sklearn.model_selection import train_test_split
    X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

    # STEP 3
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train_stdz = sc.fit_transform(X_train_raw)
    X_test_stdz = sc.transform(X_test_raw)

    from sklearn.decomposition import PCA
    dimension = 7 
    pca = PCA(n_components = dimension)
    X_train = pca.fit_transform(X_train_stdz)
    X_test = pca.transform(X_test_stdz)

    # STEP 4
    from sklearn.linear_model import LinearRegression
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    X_pred_raw = prediction_df.iloc[:, 6:-1].values
    X_pred_stdz = sc.transform(X_pred_raw)
    X_pred = pca.transform(X_pred_stdz)
    
    y_pred_prob=model.predict(X_pred)
    list1 = y_pred_prob.tolist()
    list1 = [round(list1[x], 3) for x in range(len(list1))]

    #model_finding_df = model_finding_df.drop(columns = ['away_name', 'away_abbr', 'home_name', 'home_abbr', 'week','year'])

    return list1

In [79]:
prediction_df

,away_name,away_abbr,home_name,home_abbr,week,year,win_perc_dif,Unnamed: 0_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,result


### Section for predicting 2022 wk 1

In [296]:
# Define and transform the dataset
X_pred_raw = prediction_df.iloc[:, 6:-4].values
X_pred_stdz = sc.transform(X_pred_raw)
X_pred = pca.transform(X_pred_stdz)

In [297]:
y_pred_prob=model.predict(X_pred)

In [299]:
y_pred_prob

array([[45.42476333],
       [43.96003714],
       [44.77679302],
       [44.54714593],
       [47.1249696 ],
       [41.97206356],
       [46.78798385],
       [43.5068489 ],
       [47.62856206],
       [44.87720324],
       [44.97993277],
       [46.25467773],
       [47.05511536],
       [46.99247944],
       [46.83589192],
       [47.54204082]])

In [301]:
display_prediction_for_current_week_games(y_pred_prob, prediction_df)

The Buffalo Bills are predicted to have a total of [45.425] against the Los Angeles Rams.
The New Orleans Saints are predicted to have a total of [43.96] against the Atlanta Falcons.
The Cleveland Browns are predicted to have a total of [44.777] against the Carolina Panthers.
The San Francisco 49ers are predicted to have a total of [44.547] against the Chicago Bears.
The Pittsburgh Steelers are predicted to have a total of [47.125] against the Cincinnati Bengals.
The Philadelphia Eagles are predicted to have a total of [41.972] against the Detroit Lions.
The Indianapolis Colts are predicted to have a total of [46.788] against the Houston Texans.
The New England Patriots are predicted to have a total of [43.507] against the Miami Dolphins.
The Baltimore Ravens are predicted to have a total of [47.629] against the New York Jets.
The Jacksonville Jaguars are predicted to have a total of [44.877] against the Washington Commanders.
The Kansas City Chiefs are predicted to have a total of [44

In [125]:
display_prediction_for_current_week_games(y_pred_prob, prediction_df)

The Los Angeles Chargers are predicted to have a total of 44.103 against the Kansas City Chiefs.
The New York Jets are predicted to have a total of 47.78 against the Cleveland Browns.
The Washington Commanders are predicted to have a total of 46.862 against the Detroit Lions.
The Indianapolis Colts are predicted to have a total of 48.705 against the Jacksonville Jaguars.
The Tampa Bay Buccaneers are predicted to have a total of 45.636 against the New Orleans Saints.
The Carolina Panthers are predicted to have a total of 45.982 against the New York Giants.
The New England Patriots are predicted to have a total of 47.695 against the Pittsburgh Steelers.
The Miami Dolphins are predicted to have a total of 48.505 against the Baltimore Ravens.
The Atlanta Falcons are predicted to have a total of 43.378 against the Los Angeles Rams.
The Seattle Seahawks are predicted to have a total of 48.049 against the San Francisco 49ers.
The Cincinnati Bengals are predicted to have a total of 48.965 agai

In [117]:
aggregate_games_df=aggregate_games_df.drop(columns=['Unnamed: 0_dif','index'])

In [118]:
aggregate_games_df.tail(100)

,away_name,away_abbr,home_name,home_abbr,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,result
2492,San Francisco 49ers,sfo,Seattle Seahawks,sea,0.185455,4.638182e+00,0.265455,0.285455,34.780000,1.220000,0.345455,-0.041818,21.392727,0.307273,5.749091,7.178182,0.552727,34.027273,355.445455,-1.165455,68.807273,0.794545,17.394545,-13.387273,0.380952,0.047416,53.0
2493,Washington Commanders,was,Las Vegas Raiders,rai,-0.061818,7.090909e-01,0.225455,0.101818,-77.003636,-3.320000,-2.396364,-0.161818,-75.672727,-2.269091,-3.414545,4.836364,-0.263636,30.889091,64.176364,0.038182,-46.114545,0.287273,-19.589091,1.330909,-0.005162,0.042552,32.0
2494,Baltimore Ravens,rav,Pittsburgh Steelers,pit,0.284545,5.058182e+00,-0.081818,0.450909,-5.392727,-6.085455,-3.550909,0.020000,-0.987273,-0.352727,4.930909,10.421818,0.609091,78.792727,262.740000,1.001818,73.400000,0.185455,5.540000,4.405455,0.369963,-0.026303,39.0
2495,Jacksonville Jaguars,jax,Los Angeles Rams,ram,-0.469091,-2.672727e+00,0.694545,0.020000,-71.803636,-0.150909,-2.663636,-1.392727,-73.774545,2.325455,-10.909091,-1.654545,0.185455,13.098182,-86.410909,0.325455,-58.705455,0.469091,15.778182,-1.970909,0.089841,-0.061564,44.0
2496,Denver Broncos,den,Kansas City Chiefs,kan,-0.141818,-5.189091e+00,-0.960000,-0.347273,-65.494545,-8.560000,-5.003636,-0.838182,-58.345455,-1.265455,-4.556364,1.338182,0.101818,4.436364,30.845455,0.880000,-61.058182,-0.858182,-10.300000,7.149091,0.027273,-0.126290,31.0
2497,New England Patriots,nwe,Buffalo Bills,buf,-0.032727,-2.390000e+00,-0.572727,-0.340000,-46.940000,-6.415000,-3.292273,-0.895455,-39.854545,-1.538636,-3.572727,1.991818,0.008182,0.004545,-54.693182,0.649091,-47.085455,-0.264091,-12.160455,7.085455,0.221299,-0.056447,24.0
2498,Pittsburgh Steelers,pit,Minnesota Vikings,min,0.115909,-2.925000e+00,-0.200000,0.375000,-32.650000,1.575000,-0.650000,-0.625000,-33.525000,-0.625000,-5.525000,-3.950000,-0.100000,-31.075000,-47.600000,0.275000,-63.725000,0.475000,-15.675000,-0.875000,-0.266713,0.032033,64.0
2499,Atlanta Falcons,atl,Carolina Panthers,car,-0.025000,3.552714e-15,0.425000,-0.150000,32.975000,2.100000,4.125000,0.650000,32.200000,-1.275000,-1.125000,-4.350000,-0.575000,-23.500000,-122.300000,-0.475000,9.475000,-0.125000,-10.525000,-0.775000,-0.168571,0.068051,50.0
2500,San Francisco 49ers,sfo,Cincinnati Bengals,cin,-0.100000,1.775000e+00,0.150000,-0.350000,1.050000,-1.025000,-1.775000,-0.350000,-7.325000,1.775000,-1.675000,2.025000,0.050000,13.850000,-46.275000,-0.850000,14.900000,0.025000,27.450000,-8.375000,0.059583,-0.033345,49.0
2501,Baltimore Ravens,rav,Cleveland Browns,cle,0.200000,3.500000e+00,-0.275000,0.625000,21.625000,3.175000,2.625000,0.400000,21.725000,-0.800000,3.350000,4.250000,-0.275000,14.050000,238.950000,0.575000,35.675000,0.375000,-11.550000,0.100000,0.297570,0.024450,46.0


### Grabbing 2020 agg data

In [374]:
aggregate_games_df=pd.read_csv('aggregate_games_df_original.csv')

In [375]:
aggregate_games_df = aggregate_games_df.loc[aggregate_games_df.year == 2020].reset_index()

In [376]:
aggregate_games_df = aggregate_games_df[aggregate_games_df.filter(regex='^(?!Unnamed)').columns].drop(columns = 'index')

In [377]:
aggregate_games_df= aggregate_games_df.loc[aggregate_games_df.week != 18].reset_index().drop(columns = 'index')

In [379]:
aggregate_games_df.to_csv('agg_games_2020.csv')

In [380]:
aggregate_games_df2 = pd.read_csv('agg_df_2021.csv')

In [381]:
aggregate_games_df2= aggregate_games_df2.loc[aggregate_games_df2.week != 18].reset_index().drop(columns = 'index')

In [383]:
aggregate_games_df2= aggregate_games_df2[aggregate_games_df2.filter(regex='^(?!Unnamed)').columns]

In [385]:
aggregate_games_df2 = aggregate_games_df2.loc[aggregate_games_df2.year == 2021].reset_index()

In [388]:
aggregate_games_df2=aggregate_games_df2.drop(columns = 'index')

In [154]:
lst = []
#weeksgames_df=weeksgames_df.reset_index()
for i in range(len(aggregate_games_df)):
    lst.append(aggregate_games_df['over_under'][i][:-7])
    
lst = [x.strip(' ') for x in lst]
lst = [float(x) for x in lst]
aggregate_games_df['over_under'] =lst

In [199]:
aggregate_games_df['Result_prediction']=np.where(((aggregate_games_df['predicted totals']>=aggregate_games_df['over_under'])),1,0)

In [162]:
from sklearn.metrics import confusion_matrix

In [166]:
tpr = tp/(tp+fn)
print("Percentage of totals correctly predicted to go over:",tpr)

NameError: name 'tp' is not defined

In [200]:
def c_m_analysis(true,pred):
    tn, fp, fn, tp =confusion_matrix(true,pred).ravel()
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    fpr = fp/(fp+tn)
    tpr = tp/(tp+fn)
    f_score = 2*precision*tpr/(precision+tpr)
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    print("Precision:\t\t\t%1.2f identified overs actually went over"%(precision))
    print("Recall/TPR:\t\t\t%1.2f proportion of actual overs predicted"%(recall))
    print("False Positive Rate:\t\t%1.2f proportion of unders predicted as overs"%fpr)
    print("f-score:\t\t\t%1.2f tradeoff between precision and recall"%(f_score))
    print("Accuracy:\t\t\t%1.2f how well the model has classified"%(accuracy))
c_m_analysis(aggregate_games_df['Overs'],aggregate_games_df['Result_prediction'])

Precision:			0.54 identified overs actually went over
Recall/TPR:			0.50 proportion of actual overs predicted
False Positive Rate:		0.36 proportion of unders predicted as overs
f-score:			0.52 tradeoff between precision and recall
Accuracy:			0.58 how well the model has classified


In [ ]:
aggregate_games_df['predicted totals']-aggregate_games_df['over_under']

In [ ]:
tn, fp, fn, tp =confusion_matrix(true,pred).ravel()

### Testing out thresholds

In [ ]:
testing_predictions = aggregate_games_df['predicted totals']-aggregate_games_df['over_under']
def get_classification(predictions,threshold):
    #y_pred = df['Totals_predict']-df['Closing Total']
    classes = np.zeros_like(testing_predictions)
    for i in range(len(classes)):
        #print(predictions[i])
        if predictions[i] > threshold:
            classes[i] = 1
    return classes

get_classification(testing_predictions,3)

In [220]:
def c_m_analysis(true,pred,threshold):
    tn, fp, fn, tp = confusion_matrix(true,get_classification(pred,threshold)).ravel()
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    fpr = fp/(fp+tn)
    tpr = tp/(tp+fn)
    f_score = 2*precision*tpr/(precision+tpr)
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    print("Precision:\t\t\t%1.2f identified overs actually went over"%(precision))
    print("Recall/TPR:\t\t\t%1.2f proportion of actual overs predicted"%(recall))
    print("False Positive Rate:\t\t%1.2f proportion of unders predicted as overs"%fpr)
    print("f-score:\t\t\t%1.2f tradeoff between precision and recall"%(f_score))
    print("Accuracy:\t\t\t%1.2f how well the model has classified"%(accuracy))

c_m_analysis(aggregate_games_df["Overs"],testing_predictions,-3)

Precision:			0.46 identified overs actually went over
Recall/TPR:			0.70 proportion of actual overs predicted
False Positive Rate:		0.69 proportion of unders predicted as overs
f-score:			0.56 tradeoff between precision and recall
Accuracy:			0.49 how well the model has classified


In [209]:
confusion_matrix(aggregate_games_df["result"],get_classification(testing_predictions,1.9)).ravel()

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [88]:
display_prediction_for_current_week_games(y_pred_prob, model_finding_df)

TypeError: type numpy.ndarray doesn't define __round__ method

In [2]:
aggregate_weekly_games

NameError: name 'aggregate_weekly_games' is not defined

### ADDING IN ELO 

In [31]:
def get_elo():
    elo_df = pd.read_csv('nfl_elo_latest (1).csv')
    elo_df = elo_df.drop(columns = ['neutral' ,'playoff', 'elo_prob1', 'elo_prob2', 'elo1_post', 'elo2_post',
           'qbelo1_pre', 'qbelo2_pre', 'qb1', 'qb2', 'qb1_adj', 'qb2_adj', 'qbelo_prob1', 'qbelo_prob2',
           'qb1_game_value', 'qb2_game_value', 'qb1_value_post', 'qb2_value_post',
           'qbelo1_post', 'qbelo2_post', 'score1', 'score2'])
    elo_df.date = pd.to_datetime(elo_df.date)
    elo_df = elo_df[elo_df.date >= '09-10-2020']
    
    elo_df['team1'] = elo_df['team1'].replace(['KC', 'JAX', 'CAR', 'BAL', 'BUF', 'MIN', 'DET', 'ATL', 'NE', 'WSH',
           'CIN', 'NO', 'SF', 'LAR', 'NYG', 'DEN', 'CLE', 'IND', 'TEN', 'NYJ',
           'TB', 'MIA', 'PIT', 'PHI', 'GB', 'CHI', 'DAL', 'ARI', 'LAC', 'HOU',
           'SEA', 'OAK'],
            ['kan','jax','car', 'rav', 'buf', 'min', 'det', 'atl', 'nwe', 'was', 
            'cin', 'nor', 'sfo', 'ram', 'nyg', 'den', 'cle', 'clt', 'oti', 'nyj', 
             'tam','mia', 'pit', 'phi', 'gnb', 'chi', 'dal', 'crd', 'sdg', 'htx', 'sea', 'rai' ])
    elo_df['team2'] = elo_df['team2'].replace(['KC', 'JAX', 'CAR', 'BAL', 'BUF', 'MIN', 'DET', 'ATL', 'NE', 'WSH',
           'CIN', 'NO', 'SF', 'LAR', 'NYG', 'DEN', 'CLE', 'IND', 'TEN', 'NYJ',
           'TB', 'MIA', 'PIT', 'PHI', 'GB', 'CHI', 'DAL', 'ARI', 'LAC', 'HOU',
           'SEA', 'OAK'],
            ['kan','jax','car', 'rav', 'buf', 'min', 'det', 'atl', 'nwe', 'was', 
            'cin', 'nor', 'sfo', 'ram', 'nyg', 'den', 'cle', 'clt', 'oti', 'nyj', 
             'tam','mia', 'pit', 'phi', 'gnb', 'chi', 'dal', 'crd', 'sdg', 'htx', 'sea', 'rai' ])
    return elo_df

In [32]:
elo_df = get_elo()

In [33]:
def merge_rankings(agg_games_df,elo_df):
    agg_games_df = pd.merge(agg_games_df, elo_df, how = 'inner', left_on = ['home_abbr', 'away_abbr','year'], right_on = ['team1', 'team2','season']).drop(columns = ['date','team1', 'team2','season'])
    agg_games_df['elo_dif'] = agg_games_df['elo2_pre'] - agg_games_df['elo1_pre']
    agg_games_df['qb_dif'] = agg_games_df['qb2_value_pre'] - agg_games_df['qb1_value_pre']
    agg_games_df = agg_games_df.drop(columns = ['elo1_pre', 'elo2_pre', 'qb1_value_pre', 'qb2_value_pre','quality','importance','total_rating'])
    return agg_games_df


In [34]:
aggregate_games_df= merge_rankings(aggregate_games_df,elo_df)

In [176]:
aggregate_games_df.to_csv('agg_df_totals_with_elo_rev2.csv')

In [175]:
aggregate_games_df= aggregate_games_df.drop(columns = ['predicted totals','Result_prediction'])

In [35]:
aggregate_games_df.tail(100)

,away_name,away_abbr,home_name,home_abbr,week,year,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,result,elo_dif,qb_dif
160,Houston Texans,htx,Tennessee Titans,oti,11,2021,-0.662857,-6.777143e+00,0.684286,0.351429,-2.995714,1.675714,1.048571,-0.141429,-3.252857,0.527143,-13.048571,-9.378571,-1.197143,-59.911429,-298.798571,0.161429,-62.907143,0.634286,-0.810000,-0.257143,-0.161948,-0.018835,35.0,-311.559715,-54.469219
161,New Orleans Saints,nor,Philadelphia Eagles,phi,11,2021,0.197143,-2.114286e-01,-0.627143,-0.092857,-2.454286,-0.267143,-0.440000,0.730000,-2.414286,-0.571429,0.212857,1.165714,-0.288571,-19.435714,168.761429,-0.030000,-21.890000,0.022857,-1.405714,0.040000,0.244444,-0.055217,69.0,148.694835,-74.087017
162,Cincinnati Bengals,cin,Las Vegas Raiders,rai,11,2021,0.000000,-2.328571e+00,-0.385714,0.271429,-44.842857,-6.528571,-4.328571,0.528571,-40.357143,-3.285714,1.571429,1.771429,-0.057143,11.700000,17.928571,0.528571,-33.142857,0.228571,-29.742857,4.485714,0.041958,0.002809,45.0,6.065577,-41.499542
163,Dallas Cowboys,dal,Kansas City Chiefs,kan,11,2021,0.131429,-1.567143e+00,-0.244286,-0.314286,-9.347143,-6.052857,-2.784286,-0.078571,-6.805714,0.988571,4.877143,5.325714,0.345714,26.550000,26.790000,-0.030000,17.202857,-0.657143,11.344286,2.541429,-0.154326,-0.076354,28.0,-93.812539,16.993555
164,Arizona Cardinals,crd,Seattle Seahawks,sea,11,2021,0.362857,3.911429e+00,0.977143,0.348571,46.401429,2.580000,3.125714,0.091429,41.061429,0.821429,7.372857,6.088571,0.754286,20.477143,274.735714,-0.768571,66.878571,0.461429,11.180000,-5.340000,0.457143,0.076636,36.0,75.434931,-53.283848
165,Pittsburgh Steelers,pit,Los Angeles Chargers,sdg,11,2021,0.033929,-3.257143e+00,0.642857,-0.157143,-31.142857,-0.057143,-0.400000,-0.785714,-33.842857,-0.185714,-4.385714,1.528571,-0.228571,-7.357143,166.614286,0.028571,-38.500000,0.271429,-12.828571,-2.700000,-0.261234,-0.040477,78.0,12.663171,-34.675099
166,New York Giants,nyg,Tampa Bay Buccaneers,tam,11,2021,-0.316667,-4.066667e+00,0.516667,-0.050000,-84.250000,-7.266667,-5.900000,-2.050000,-77.616667,-0.716667,-11.850000,3.266667,-0.033333,7.550000,-57.333333,0.983333,-76.700000,-0.066667,-15.716667,6.633333,-0.029060,-0.096213,40.0,-192.654841,-128.711909
167,Chicago Bears,chi,Detroit Lions,det,12,2021,0.303333,-1.306667e+00,0.540000,0.090000,-41.150000,-8.046667,-6.603333,-0.073333,-37.173333,-0.560000,0.786667,4.456667,0.000000,21.443333,0.666667,0.743333,-19.706667,0.040000,-1.533333,3.976667,-0.027376,0.005150,30.0,146.190723,1.927580
168,Las Vegas Raiders,rai,Dallas Cowboys,dal,12,2021,-0.196667,-3.636667e+00,-0.086667,0.140000,12.520000,0.243333,-0.333333,-0.556667,14.373333,0.003333,-6.500000,-5.663333,-0.040000,-45.823333,-94.123333,0.220000,-33.303333,-0.086667,3.800000,1.853333,0.004582,-0.080934,69.0,-95.585615,-39.040283
169,Buffalo Bills,buf,New Orleans Saints,nor,12,2021,0.123333,3.403333e+00,0.843333,0.253333,72.143333,7.613333,7.063333,0.043333,71.136667,1.143333,4.660000,-3.526667,0.136667,-3.606667,51.513333,-0.280000,68.536667,0.303333,13.140000,-1.006667,-0.255000,0.102598,37.0,22.731979,131.385591


## 4. Model Selection

### 4.1. Checking the balance of the data and existence of multicolinearity

#### Preparing the data before applying the machine learning (ML) models
1. Before going to the steps of building model, we have to make sure our data is ready
2. We split the data into two datasets: data for the model (that later will be split again into test and train dataset), and data to be predicted
3. We'll plot the histogram of each dependent variables to see the overall distribution of each feature
4. We'll check the balance of the target variable
5. We'll confirm again the existence of multicolinearity

In [389]:
# We first take the dataframe where the results are not null (NaN): model_finding_df
# In other words, we only use the dataframe with completed games
model_finding_df = aggregate_games_df[aggregate_games_df.result.notna()]
model_finding_df2 = aggregate_games_df2[aggregate_games_df2.result.notna()]
# Then we select the dataset to be predicted by the best model (games that have not been played): prediction_df
prediction_df = aggregate_games_df[aggregate_games_df.result.isnull()]

In [ ]:
# Creating histogram of the features to visualize the distribution of the data
def create_histograms(model_finding_df, no_of_cols):
    features_df = model_finding_df.iloc[:,6:-1]
    no_of_rows = (len(features_df.columns)//no_of_cols)+1
    fig = plt.figure(figsize=(20,25))
    for i, col in enumerate(features_df.columns):
        ax = fig.add_subplot(no_of_rows,no_of_cols, i+1)
        features_df[col].hist(bins=50, ax=ax, facecolor='midnightblue', grid=False)
        ax.set_title('Distribution of '+col, color='firebrick')
        ax.set_ylabel('Counts')
    fig.tight_layout()  
    plt.show()

create_histograms(model_finding_df, 4)

1. We can see that almost all the data are looking like normal distribution
2. There are some extreme values, but since these numbers are game stats (facts from every game), we believe that we should include all the data to build the models
3. We'll now check the balance of the data because we are building a classification model

In [ ]:
# Checking the balance of the target variable
def create_result_frequency(model_finding_df):
    import seaborn as sns
    fig,axes=plt.subplots(1 , 2, figsize=(12,6), dpi=80)
    model_finding_df['result'].value_counts().plot.pie(explode=[0,0.1], autopct='%1.1f%%', ax=axes[0], colormap='Paired')
    axes[0].set_title('Result Frequency (in Percentage)')
    axes[0].set_ylabel('')
    sns.countplot('result', data=model_finding_df, ax=axes[1], palette=['#A6CEE3', '#B15928'])
    axes[1].set_title('Result Frequency')
    axes[1].set_ylabel('Counts')
    axes[1].set_xlabel('Target')
    axes[1].set_xticklabels(['0.0: Home Team Wins','1.0: Away Team Wins'])
    plt.show()

create_result_frequency(model_finding_df)

1. The ratio of the target variable (result) is 55.6 : 44.4
2. We think this is a balanced dataset
2. Since it's balanced, we can proceed to check the multicolinearity

In [ ]:
# Checking the correlation matrix
def create_correlation_matrix(model_finding_df):
    features_df = model_finding_df.iloc[:, 6:-1]
    import seaborn as sns
    fig, ax = plt.subplots(figsize=(10, 8), dpi=80)  
    corr_mat = features_df.corr()
    sns.heatmap(corr_mat[(corr_mat >= 0.5) | (corr_mat <= -0.5)], 
            cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True)
    ax.set_title('Features Matrix Correlation', fontsize=15)

create_correlation_matrix(model_finding_df)

1. We can see that there are some features that are highly correlated with each other
2. We will perform PCA to reduce the dimensions (data reductions) to handle this

### 4.2. Building the ML models

#### Steps to build the classification model
1. Separate the data into independent variables (X) and dependent variables (y). Here, 'result' is the dependent variable and rest are independent variables.
2. Splitting the data into train set and test set. Model will learn from the train set and its performance and effectiveness will be tested on the test set.
3. Using StandardScaler to standardise the values corresponding to each independent variable and bring them all in a particular range. We'll perform PCA inside this step too.
4. Importing the model and training the model on the test set
5. Testing the model built on the test set
6. We will be using cross validation to test the ability of our machine learning model to predict new data. It can also help to flag problems like overfitting or selection bias and gives insights on how the model will generalize to an independent dataset.
7. Printing the report (overall effectiveness) including recall, precision, etc of the model

In [ ]:
model_finding_df

In [390]:
# STEP 1
# Separating the model_finding_df to X and y

X = model_finding_df.iloc[:, 6:-1].values 
y = model_finding_df2.iloc[:, -1].values

In [391]:
# STEP 2 
# Splitting the data into train and test

from sklearn.model_selection import train_test_split
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [392]:
# STEP 3
# Standardizing the data and performing PCA

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_stdz = sc.fit_transform(X_train_raw)
X_test_stdz = sc.transform(X_test_raw)

print(X_train_stdz.shape)
print(X_test_stdz.shape)

(191, 22)
(64, 22)


In [393]:
# Performing PCA

from sklearn.decomposition import PCA
dimension = 7 
pca = PCA(n_components = dimension)
X_train = pca.fit_transform(X_train_stdz)
X_test = pca.transform(X_test_stdz)
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))
print('Total variance explained: {}'.format(pca.explained_variance_ratio_.sum()))

Explained variation per principal component: [0.32095532 0.20113637 0.0942349  0.08115136 0.06823364 0.04638349
 0.04028429]
Total variance explained: 0.8523793702205467


In [394]:
# Checking the eigenvalues

eigenvalues = pca.explained_variance_
eigenvalues

array([7.09818019, 4.44828963, 2.0840793 , 1.79472643, 1.50904082,
       1.02580755, 0.89091879])

1. The number of components that we have chosen is 7
2. The reason is that the eigenvalues for 7 components are all > 0.9 (which is close to 1.0)
3. Also, we want to make sure that the total variance explained is at least 80%

In [ ]:
# Defining a function to highlight large correlation in the factor loadings table
def highlight_background(val):
    threshold = 0.35
    color = ''
    if (val > threshold) or (val < -1*threshold):
        color = 'wheat'
    return 'background-color: %s' % color

def highlight_font(val):
    threshold = 0.35
    color = ''
    if (val > threshold) or (val < -1*threshold):
        if val > 0:
            color = 'royalblue'
        else:
            color = 'firebrick'
    return 'color: %s' % color

In [ ]:
# Using the functions above, we can check which variables explain each component
# Usually, correlation larger than 0.3 (absolute value) is large enough, but we use 0.35 as our threshold
# Column = PCA, Row = Original features

components_df = pd.DataFrame(pca.components_, index=[
    'PCA%i' % i for i in range(dimension)])
components_df = components_df.T.set_index(model_finding_df.iloc[:, 6:-1].columns)
components_style = components_df.style.applymap(highlight_background).applymap(highlight_font)
components_style

1. Component 1 (PCA0): total_yards (+)
2. Component 2 (PCA1): pass_attempts (-), rush_attempts (+), rush_yards (+)
3. Component 3 (PCA2): penalties (-), yards_from_penalties (-)
4. Component 4 (PCA3): fumbles (-), interceptions (-), turnovers (-)
5. Component 5 (PCA4): times_sacked (+), yards_lost_from_sacks (+)
6. Component 6 (PCA5): fourth_down_perc (+)
7. Component 7 (PCA6): rush_touchdowns (+), time_of_possession (-)

In [ ]:
# Creating the dataframe for the PCA of training dataset to check the matrix correlation again

pca_X_train_df = pd.DataFrame(data = X_train, columns=[
    'PCA%i' % i for i in range(dimension)])

In [ ]:
# Creating the correlation matrix for the principal components
def create_pca_correlation_matrix(pca_X_train_df):
    import seaborn as sns
    fig, ax = plt.subplots(figsize=(10, 8), dpi=80)  
    corr_mat = pca_X_train_df.corr()
    sns.heatmap(corr_mat, mask=np.zeros_like(corr_mat, dtype=np.bool),
                cmap='viridis', square=True, ax=ax)
    ax.set_title('Components Matrix Correlation', fontsize=15)

create_pca_correlation_matrix(pca_X_train_df)

1. We can see that we have handled the multicollinearity
2. We can continue with applying several ML models to the datasets (steps 4 to 7 for each  model)

#### Now we are using several machine learning algorithms such as Decision Tree, Logistic Regression , Random Forest Classifier, etc

Steps 1 to 3 (preprocessing steps) as seen above remain same for all the models and we will be building all the models on that dataset

#### Model 1 : Decision Tree

In [ ]:
# STEP 4
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

In [ ]:
# STEP 5 & STEP 6
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report as report
y_pred = classifier.predict(X_test)
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

In [ ]:
# STEP 7
print(report(y_test, y_pred))

#### Model 2 : Logistic Regression

In [398]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [399]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report as report
y_pred = classifier.predict(X_test)
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 50.26 %
Standard Deviation: 6.70 %


In [ ]:
print(report(y_test, y_pred))

#### Model 3 : Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report as report
y_pred = classifier.predict(X_test)
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

In [ ]:
print(report(y_test, y_pred))

#### Model 4 : Kernel SVM (Support Vector Machine)

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report as report
y_pred = classifier.predict(X_test)
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

In [ ]:
print(report(y_test, y_pred))

#### Model 5 : Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report as report
y_pred = classifier.predict(X_test)
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

In [ ]:
print(report(y_test, y_pred))

#### Model 6 : KNN (K Nearest Neighbours)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report as report
y_pred = classifier.predict(X_test)
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

In [ ]:
print(report(y_test, y_pred))

#### Model 7 : XGBoost

In [ ]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report as report
y_pred = classifier.predict(X_test)
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

In [ ]:
print(report(y_test, y_pred))

#### Model 8 : Artifical Neural Networks

In [ ]:
import tensorflow as tf
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=7, activation='relu'))
for n in range(2):
    ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 32, epochs = 300)

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
print(report(y_test, y_pred))

### 4.3. Final model selection

#### We choose Logistic Regression Classifier because it has high accuracy and comparable to our models with high accuracy
1. Based on the results above, we can see that Logistic Regression, Kernel SVM, Naive Bayes models and Neural Network have recorded the highest accuracy
2. These four models are also comparable to each other based on the other metrics (precision, recall, etc)
3. We choose Logistic Regression as our model for further analysis as Logistic Regression model gives us accuracy comparable/higher to others. It is also faster to run, efficient to implement compared to other models.
4. With the selected model, we will now take a look at several things: feature importance, confusion matrix, and ROC-AUC
5. Before that, we will run and define the classifier again to make sure we are using the right model

In [ ]:
# STEP 1
X = model_finding_df.iloc[:, 6:-1].values
y = model_finding_df.iloc[:, -1].values

# STEP 2
from sklearn.model_selection import train_test_split
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

# STEP 3
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_stdz = sc.fit_transform(X_train_raw)
X_test_stdz = sc.transform(X_test_raw)

from sklearn.decomposition import PCA
dimension = 7 
pca = PCA(n_components = dimension)
X_train = pca.fit_transform(X_train_stdz)
X_test = pca.transform(X_test_stdz)

# STEP 4
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

In [ ]:
def show_feature_importances(X_test, y_test):
    from yellowbrick.model_selection import feature_importances
    X_test_df = pd.DataFrame(X_test, columns = ['PCA%i' % i for i in range(dimension)])
    y_test_df = pd.DataFrame(y_test, columns = [list(model_finding_df.columns)[-1]])
    classifier = LogisticRegression(C = 0.01, penalty = 'l2', solver = 'liblinear', random_state = 0)
    feature_importances(classifier, X_test_df, y_test_df)
    plt.rcParams["figure.dpi"] = 80
    plt.show()

show_feature_importances(X_test, y_test)

#### Feature importance after PCA
1. As we can see above, PCA0, PCA1, and PCA3 are the top 3 in the feature importance graph
2. If we take a look again at the factor loadings, we know that PCA0 is highly correlated with total_yards (+)
3. PCA1 is highly correlated with pass_attempts (-), rush_attempts (+), and rush_yards (+)
4. And last but not least, PCA3 is highly explained by fumbles (-), interceptions (-), and turnovers (-)
5. The threshold number for absolute correlation between original varibales and the PCA components is 0.35

In [ ]:
y_test_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_test_pred)
print('Confusion Matrix:')
print(cm)
print('\nAccuracy: ' + f'{100 * accuracy_score(y_test, y_test_pred):.2f}%')

In [ ]:
def show_confusion_matrix(classifier, X_test, y_test):
    from sklearn.metrics import plot_confusion_matrix
    matrix = plot_confusion_matrix(classifier, X_test, y_test,
                                 cmap=plt.cm.Blues,
                                 normalize=None)
    plt.grid(visible=False)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show(matrix)
    plt.rcParams["figure.dpi"] = 80
    plt.show()

show_confusion_matrix(classifier, X_test, y_test)

#### Interpretation of the confusion matrix
1. True negative (tn): model predicts away team lost (0) and away team lost = 295
2. False positive (fp): model predicts away team won (1) but away team actually lost = 92
3. False negative (fn): model predicts away team lost but away team actually won = 171
4. True positive (tp): model predicts away team won and away team won = 128

In [ ]:
def show_confusion_matrix_analysis(classifier, X_test, y_test):
    y_test_pred = classifier.predict(X_test)
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_test_pred)
    tn = cm[0][0]
    fp = cm[0][1]
    fn = cm[1][0]
    tp = cm[1][1]
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    fpr = fp/(fp+tn)
    f_score = 2*precision*recall/(precision+recall)
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    
    print("Precision:\t\t\t%1.2f"%(100*precision) + "% identified as away teams' victory are away teams' victory")
    print("Recall/TPR:\t\t\t%1.2f"%(100*recall) + "% of away teams' victory are identified")
    print("False Positive Rate:\t\t%1.2f"%(100*fpr) + "% of away team's defeat identified as away team's victory")
    print("f-score:\t\t\t%1.2f"%(100*f_score) + "% tradeoff between precision and recall")
    print("Accuracy:\t\t\t%1.2f"%(100*accuracy) + "% how well the model has classified")

show_confusion_matrix_analysis(classifier, X_test, y_test)

In [ ]:
def show_roc_curve(classifier, X_test, y_test):
    import sklearn.metrics as metrics
    y_test_pred_prob = classifier.predict_proba(X_test)
    y_test_pred_prob = y_test_pred_prob[:,1]
    fpr, tpr, threshold = metrics.roc_curve(y_test, y_test_pred_prob)
    roc_auc = metrics.auc(fpr, tpr)

    import matplotlib.pyplot as plt
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.rcParams["figure.dpi"] = 80
    plt.show()

show_roc_curve(classifier, X_test, y_test)

1. The area under curve (AUC) is 0.68
2. The AUC is a measure of how stable the model is
3. If the AUC is higher, then the better the model is at predicting 0 classes as 0 and 1 classes as 1
4. We consider the 0.68 value is good enough for this use-case

#### Feature importance with original full features
1. We have obtained the feature importance from the model after PCA
2. However, to be able to compare it the previous hypothesis of good features and bad features, we need the feature importance with original full features instead
3. To do that, we run the best model (Logistic Regression) on the dataset without performing PCA so the dimension is not reduced
4. When building this particular model, we add 'F' or 'f' to the variable or model name to indicate 'full features', and so that it won't change the value of pre-defined variables that have been used before

In [ ]:
# STEP 1
XF = model_finding_df.iloc[:, 6:-1].values
yf = model_finding_df.iloc[:, -1].values

# STEP 2
from sklearn.model_selection import train_test_split
XF_train_raw, XF_test_raw, yf_train, yf_test = train_test_split(XF, yf, test_size = 0.25, random_state = 0)

# STEP 3
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
XF_train = sc.fit_transform(XF_train_raw)
XF_test = sc.transform(XF_test_raw)

# STEP 4
from sklearn.linear_model import LogisticRegression
classifier_f = LogisticRegression(random_state = 0)
classifier_f.fit(XF_train, yf_train)

#### GridSearchCV method is used for hyperparameter tuning so that the best parameters are selected for the Logistic Regression

In [ ]:
from sklearn.model_selection import GridSearchCV
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
# define grid search
parameters = dict(solver=solvers,penalty=penalty,C=c_values)
# parameters = {'var_smoothing': np.logspace(0,-9, num=100)}
grid_search = GridSearchCV(estimator = classifier_f,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search.fit(XF_train, yf_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Parameters:", best_parameters)

In [ ]:
def show_feature_importances_full_features(XF_test, yf_test):
    from yellowbrick.model_selection import feature_importances
    XF_test_df = pd.DataFrame(XF_test, columns = list(model_finding_df.columns)[6:-1])
    yf_test_df = pd.DataFrame(yf_test, columns = [list(model_finding_df.columns)[-1]])
    classifier_f = LogisticRegression(C = 0.01, penalty = 'l2', solver = 'liblinear', random_state = 0)
    feature_importances(classifier_f, XF_test_df, yf_test_df)
    plt.rcParams["figure.dpi"] = 80
    plt.show()

show_feature_importances_full_features(XF_test, yf_test)

1. If we compare with our previous hypothesis, there are five features where feature importance shows different results
2. Feature importance shows that penalties has positive importance while we hypothesized that this feature is a bad feature instead
3. Feature importance shows that total_yards, net_pass_yards, pass_yards, and rush_attempts has negative importance while we hypothesized that they are good features instead
4. Based on these results, we believe that we still can accept our hypothesis
5. We can say that our intuition about good attributes (features) and bad attributes is in line with the feature importance results, even though there are some exceptions

## 5. NFL Games Prediction

### 5.1. Predicting probabilities for the current week games that have not been played yet

1. Using the selected classifier, we can now predict the outcome of forthcoming games in the current week that have not been played
2. We first define a function to display the prediction results
3. After that, we define the dataset that will be used from the prediction dataframe we have created before
4. The dataset is then standardized and transformed according to the PCA results
5. Finally, we can compute the winning probability of the away team and display the results

In [ ]:
# Function to display the prediction results
def display_prediction_for_current_week_games(y_pred_prob, prediction_df):
    for t in range(len(y_pred_prob)):
        win_prob = round(y_pred_prob[t], 3)
        away_team = prediction_df.reset_index().drop(columns = 'index').loc[t,'away_name']
        home_team = prediction_df.reset_index().drop(columns = 'index').loc[t,'home_name']
        print('The {} have a probability of {} of defeating the {}.'.format(away_team, win_prob, home_team))

In [ ]:
# Define and transform the dataset
X_pred_raw = prediction_df.iloc[:, 6:-1].values
X_pred_stdz = sc.transform(X_pred_raw)
X_pred = pca.transform(X_pred_stdz)

In [ ]:
# Predict the results (winning probability)
y_pred_prob = classifier.predict_proba(X_pred)
y_pred_prob = y_pred_prob[:,1]

In [ ]:
display_prediction_for_current_week_games(y_pred_prob, prediction_df)

### 5.2. Simple text mining (sentiment analysis) vs. machine learning model
1. We know that NFL is the number 1 sport in the US
2. Fans, sports analysts, or media in general will report a news, share an analysis, or just give a support about the NFL teams
3. One medium where they'll do it is Twitter
4. We will gather the recent tweets from Twitter using each team's name as the search term
5. After that we will perform sentiment analysis on each team's search results
6. For each pair of two teams that will play each other in the current week, we'll compute the net sentiment score (net_sentiment_dif), which is the difference of the net sentiment of the away team minus that of the home team
7. If the net sentiment is greater than 0, then we'll just simply conclude that the away team is more favoured, and vice versa
8. We can then compare it with the probability model results since we can know which team is more favoured to win the game based on the winning probability of the away team

In [ ]:
# Function to do the positive vs. negative sentiment analysis
def do_pos_neg_sentiment_analysis(text_list, debug=False):
    import nltk
    def get_pos_neg_words():
        
        def get_words(url):
            import requests
            words = requests.get(url).content.decode('latin-1')
            word_list = words.split('\n')
            index = 0
            while index < len(word_list):
                word = word_list[index]
                if ';' in word or not word:
                    word_list.pop(index)
                else:
                    index+=1
            return word_list

        p_url = 'http://ptrckprry.com/course/ssd/data/positive-words.txt'
        n_url = 'http://ptrckprry.com/course/ssd/data/negative-words.txt'
        positive_words = get_words(p_url)
        negative_words = get_words(n_url)
        return positive_words, negative_words
    
    positive_words, negative_words = get_pos_neg_words()
    from nltk import word_tokenize
    sentiment_results = list()
    for text in text_list:
        cpos = cneg = lpos = lneg = 0
        for word in word_tokenize(text[1]):
            if word in positive_words:
                if debug:
                    print("Positive", word)
                cpos+=1
            if word in negative_words:
                if debug:
                    print("Negative", word)
                cneg+=1
        sentiment_results.append((text[0], cpos/len(word_tokenize(text[1])),
                                  cneg/len(word_tokenize(text[1]))))
    return sentiment_results

In [ ]:
# Function to get the recent tweets that will be used as an input to the sentiment analysis function
def get_recent_tweets(search_term):
    
    # Change the twitter API input
    consumer_key = 'nfTyujbaEl1fDF22xVlMYz6ga'
    consumer_secret = 'NinKRYzhB5LQevBg7a5H38SZIoDdDFHsvqgQWFdBi5Ir2Bz6qP'
    access_token = '1456748024533856263-3TENqcDwMAamVNd4wEKpa8jxNgEsEf'
    access_token_secret = 'u8F5lNFuPfqQ2gZtS81mLSLkoAQikBeFUA7JNy4Gfqvg9'
    
    # Change this to your computer directory
    file_location = 'C:/Data/03_MSBA/05_Courses_Term/01_Fall_2021/IEOR_E_4523_DA/Group_Project/Notebook_Files/Tweets/'
    
    import tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)

    tweet_results = api.search_tweets(q=search_term,
                                      lang='en',
                                      result_type='recent',
                                      count=1000)

    for i in range(len(tweet_results)):
        filename = search_term+'.'+str(len(tweet_results)-i)
        with open(file_location+filename, 'w', encoding='utf-8') as f:
            f.write(tweet_results[i]._json['text']+'\n')

In [ ]:
def compare_probability_model_and_sentiment(prediction_df, y_pred_prob):
    prediction_df = prediction_df.reset_index().drop(columns='index')

    probability_final_list = list()
    
    for i in prediction_df.index:
        if y_pred_prob[i] > 0.5:
            probability_winner = prediction_df['away_name'][i]
        elif y_pred_prob[i] < 0.5:
            probability_winner = prediction_df['home_name'][i]
        else:
            probability_winner = 'Draw'

        probability_final_list.append([prediction_df['away_name'][i],
                                   prediction_df['home_name'][i],
                                   f'{y_pred_prob[i]:.2f}',
                                   probability_winner])
    
    sentiment_final_list = list()

    for i in prediction_df.index:
        get_recent_tweets(prediction_df['away_name'][i])
        get_recent_tweets(prediction_df['home_name'][i])
    
        # Change this to your computer directory where the tweets data are stored
        tweets_root = 'C:/Data/03_MSBA/05_Courses_Term/01_Fall_2021/IEOR_E_4523_DA/Group_Project/Notebook_Files/Tweets'
    
        import nltk
        from nltk.corpus import PlaintextCorpusReader
        
        away_files = prediction_df['away_name'][i]+'.*'
        home_files = prediction_df['home_name'][i]+'.*'
        away_data = PlaintextCorpusReader(tweets_root, away_files)
        home_data = PlaintextCorpusReader(tweets_root, home_files)
    
        sentiment_list = do_pos_neg_sentiment_analysis([[prediction_df['away_name'][i],
                                                         away_data.raw()],
                                                        [prediction_df['home_name'][i],
                                                         home_data.raw()]])
    
        away_net = (sentiment_list[0][1] - sentiment_list[0][2])
        home_net = (sentiment_list[1][1] - sentiment_list[1][2])
        net_sentiment_dif = 100 * (away_net - home_net)
    
        if net_sentiment_dif > 0:
            sentiment_winner = prediction_df['away_name'][i]
        elif net_sentiment_dif < 0:
            sentiment_winner = prediction_df['home_name'][i]
        else:
            sentiment_winner = 'Draw'
    
        sentiment_final_list.append([sentiment_list[0][0],
                                     sentiment_list[1][0],
                                     f'{net_sentiment_dif:.2f}%',
                                     sentiment_winner])
    
    probability_df = pd.DataFrame(probability_final_list, columns = ['away_name',
                                                                     'home_name',
                                                                     'away_win_prob',
                                                                     'favourable_team_prob'])
    
    sentiment_df = pd.DataFrame(sentiment_final_list, columns = ['away_name',
                                                                 'home_name',
                                                                 'away_minus_home_net_sentiment',
                                                                 'favourable_team_sentiment'])
    
    prob_model_vs_sentiment_df = pd.merge(probability_df, sentiment_df,
                                          left_on=['away_name', 'home_name'],
                                          right_on=['away_name', 'home_name'])
   
    return prob_model_vs_sentiment_df

In [ ]:
# Calling the main function to create the comparison dataframe
prob_model_vs_sentiment_df = compare_probability_model_and_sentiment(prediction_df, y_pred_prob)

1. The favourable_team_prob is equal to away_name if the away_win_prob is greater than 0.5
2. The favourable_team_prob is equal to home_name if the away_win_prob is smaller than 0.5
3. The favourable_team_sentiment is equal to away_name if the away_minus_home_net_sentiment is greater than 0%
4. The favourable_team_sentiment is equal to home_name if the away_minus_home_net_sentiment is smaller than 0%

In [ ]:
prob_model_vs_sentiment_df

#### In this analysis and comparison, we have some assumptions
1. We assume the recent tweets will reflect the upcoming game that a particular team will play, meaning that the tweets leading up to the match will be about the match: who will win and who will lose (while in reality, the tweets are not always about the upcoming game)
2. We assume that greater net sentiment means that there are more favourable tweets on that team to win the game against the other team, even though both teams have recorded positive net sentiment

In [ ]:
prob_model_vs_sentiment_df[prob_model_vs_sentiment_df['favourable_team_prob']
                           == prob_model_vs_sentiment_df['favourable_team_sentiment']]

1. We can filter the dataframe to select all the rows where both the favourable teams (one is based on the model and the other one is based on the sentiment analysis) are equal
2. It is also important to note that the tweets data are always being updated since new tweets appear from time to time, and thus the sentiment analysis results may change accordingly